In [ ]:
# import pandas as pd
# import numpy as np
# import yfinance as yf
# import torch
# from datetime import datetime
# import matplotlib.pyplot as plt


# def calculate_indicators_ver3(data, interval=None, seq_len=None):
#     # Adjust parameters based on frequency
#     if interval == '1wk':  # Weekly data
#         period = 6
#         rsi_period = 9
#         short_ema_period = 8
#         long_ema_period = 16
#         signal_period = 5
#         vol_period = 14
#         bband_window_size = 12
#     elif interval == '1mo':  # Monthly data
#         period = 4
#         rsi_period = 7
#         short_ema_period = 6
#         long_ema_period = 12
#         signal_period = 4
#         vol_period = 10
#         bband_window_size = 8
#     elif interval == '1d':  # Monthly data
#         period = 7
#         rsi_period = 14
#         short_ema_period = 12
#         long_ema_period = 26
#         signal_period = 9
#         vol_period = 26
#         bband_window_size = 20
#     elif interval == None:  # not specified
#         period = 5
#         rsi_period = 7
#         short_ema_period = 6
#         long_ema_period = 13
#         signal_period = 4
#         vol_period = 13
#         bband_window_size = 10

#     if data.shape[0] < long_ema_period + seq_len:
#         return None
    
#     # Check if 'Close' column exists in the data
#     if 'Close' in data.columns:
#         # Calculate daily returns
#         data['Return'] = data['Close'].pct_change()
#         # Calculate daily Direction
#         data["Direction_Signal"] = np.where((data['Return']<-0.001), 1, np.where((data['Return']>0.001), 3, 2))
#         # Calculate VPT (Volume-Price Trend)
#         data['VPT'] = (data['Volume'] * data['Return']).cumsum()
#         # Price Range
#         data['Price Range'] = data['High'] - data['Low']
#         # Candle Body Size
#         data['Body Size'] = data['Open'] - data['Close']
#         # Upper Shadow Size
#         data['Upper Shadow'] = data.apply(lambda row: row.High - max(row.Open, row.Close), axis=1)
#         # Lower Shadow Size
#         data['Lower Shadow'] = data.apply(lambda row: min(row.Open, row.Close) - row.Low, axis=1)

#         # Calculate ZLEMA
#         lag = (period - 1) // 2  # Ensure integer division
#         ema = data['Close'].ewm(span=10, adjust=False).mean()
#         data['ZLEMA'] = ema + (ema - ema.shift(lag))

#         # lag = (period - 1) // 2
#         # # Calculate adjusted price
#         # adjusted_price = 2 * data['Close'] - data['Close'].shift(lag)

#         # # Calculate ZLEMA directly using adjusted price
#         # zlema = adjusted_price.ewm(span=period).mean()
#         # data['ZLEMA'] = zlema

#         # Define tolerance as a percentage of the close price
#         tolerance = 0.015

#         # Initialize column with 0s
#         data['ZLEMA_Signal'] = 2

#         # Buy signal when Close is above (1 + tolerance) * ZLEMA
#         data.loc[data['Close'] > data['ZLEMA'] * (1 + tolerance), 'ZLEMA_Signal'] = 3

#         # Sell signal when Close is below (1 - tolerance) * ZLEMA
#         data.loc[data['Close'] < data['ZLEMA'] * (1 - tolerance), 'ZLEMA_Signal'] = 1

#         # Calculate the first exponential moving average (EMA1).
#         ema1 = data['Close'].ewm(span=period, min_periods=period).mean()
#         # Calculate the second exponential moving average (EMA2).
#         ema2 = ema1.ewm(span=period, min_periods=period).mean()
#         # Calculate the third exponential moving average (EMA3).
#         ema3 = ema2.ewm(span=period, min_periods=period).mean()
#         # Calculate the TEMA.
#         tema = 3 * ema1 - 3 * ema2 + ema3

#         # Convert the TEMA to a Pandas Series.
#         data['TEMA'] = tema

#         # Initialize column with 0s
#         data['TEMA_Signal'] = 2

#         # Buy signal when Close is above (1 + tolerance) * TEMA
#         data.loc[data ['Close']> data ['TEMA']* (1 + tolerance), 'TEMA_Signal']= 3

#         # Sell signal when Close is below (1 - tolerance) * TEMA
#         data.loc[data ['Close']< data ['TEMA']*(1-tolerance), 'TEMA_Signal']= 1

#         # Calculate RSI and its signal
#         delta = data['Close'].diff()
#         up, down = delta.copy(), delta.copy()

#         up[up < 0] = 0
#         down[down > 0] = 0

#         average_gain = up.rolling(window=rsi_period).mean()
#         average_loss = abs(down.rolling(window=rsi_period).mean())

#         epsilon = 1e-10  # A small constant to prevent division by zero
#         rs = average_gain / (average_loss + epsilon)

#         data['RSI'] =100 - (100 / (1 + rs))
#         data["RSI_Signal"] =2
#         data.loc[data.RSI < 30,"RSI_Signal"] =3
#         data.loc[data.RSI > 70,"RSI_Signal"] =1

#         # # Calculate MACD Line: (12-day EMA - 26-day EMA)
#         EMA_short = data['Close'].ewm(span=short_ema_period).mean()
#         EMA_long = data['Close'].ewm(span=long_ema_period).mean()
#         data['MACD_Line'] = EMA_short - EMA_long

#         # Calculate Signal Line: a n-day MA of MACD Line
#         data['Signal_Line'] = data["MACD_Line"].ewm(span=signal_period).mean()

#         # Generate MACD signals based on crossovers
#         data["MACD_Signal"] = 2 # Neutral
#         data.loc[(data.MACD_Line > data.Signal_Line) & (data.MACD_Line.shift() < data.Signal_Line.shift()),"MACD_Signal"] = 3 # Buy
#         data.loc[(data.MACD_Line < data.Signal_Line) & (data.MACD_Line.shift() > data.Signal_Line.shift()),"MACD_Signal"] = 1 # Sell

#         # Generate EMA signals based on crossovers
#         data["EMA_Short"]=EMA_short
#         data["EMA_Long"]=EMA_long

#         # Define a small percentage as a threshold
#         ema_diff_threshold = 0.01 # 1% difference

#         # Calculate absolute percent difference between short and long EMA
#         data['EMA_Diff'] = (EMA_short - EMA_long) / EMA_long

#         data["EMA_Signal"] = np.where((data["EMA_Short"] > data["EMA_Long"]) & (abs(data["EMA_Diff"]) > ema_diff_threshold), 3, 
#                              np.where((data["EMA_Short"] < data["EMA_Long"]) & (abs(data["EMA_Diff"]) > ema_diff_threshold), 1, 2))

#         # Calculate Volatility as rolling standard deviation of log returns
#         data["Log_Return"] =(np.log(data.Close).diff())
#         data["Volatility"] =(data.Log_Return.rolling(window=vol_period, center=False).std())

#         # Generate Volatility signals based on high or low volatility
#         high_vol_threshold = data["Volatility"].quantile(0.85)
#         low_vol_threshold = data["Volatility"].quantile(0.15)
        
#         data['Volatility_Signal'] = 2
#         data.loc[data.Volatility > high_vol_threshold, 'Volatility_Signal'] = 3
#         data.loc[data.Volatility < low_vol_threshold, 'Volatility_Signal'] = 1

#         # Calculate Bollinger Bands
#         data["Middle_Band"] = data["Close"].rolling(window=bband_window_size, center=False).mean()
#         data["Std_Dev"] = data["Close"].rolling(window=bband_window_size, center=False).std()
#         data["Upper_Band"] = data["Middle_Band"] + data["Std_Dev"] * 2
#         data["Lower_Band"] = data["Middle_Band"] - data["Std_Dev"] * 2

#         # Identify shifting points
#         data["BBand_Signal"] = 2  # neutral

#         # Create a copy of the "Close" column
#         close_shifted = data["Close"].copy()

#         # Check for crossing Upper Band
#         cross_upper_band = (close_shifted.shift(1) <= data["Upper_Band"].shift(1)) & (data["Close"] > data["Upper_Band"])
#         data.loc[cross_upper_band, "BBand_Signal"] = 3  # buy

#         # Check for crossing Lower Band
#         cross_lower_band = (close_shifted.shift(1) >= data["Lower_Band"].shift(1)) & (data["Close"] < data["Lower_Band"])
#         data.loc[cross_lower_band, "BBand_Signal"] = 1  # sell

#         # Drop unnecessary columns
#         data = data.drop(columns=["Middle_Band", "Std_Dev", 'Log_Return', 'Open', 'Low', 'High', 'Volume'])

#     else:
#         print("'Close' column is not present in the input dataframe.")

#     return data

# def fetch_data(symbol_list, start_date, interval, target_only=False):
#     # Initialize an empty DataFrame
#     all_data = []

#     # Keep 'today' as a datetime object for datetime operations
#     today_datetime = datetime.now()
#     # Convert to string only when needed for string operations
#     today_str = today_datetime.strftime('%Y-%m-%d')

#     for symbol in symbol_list:
#         try:
#             data = yf.download(symbol, start=start_date, end=today_str, interval=interval)
#         except KeyError:
#             print(f"Data for {symbol} not found.")
#             continue

#         data = data.copy()

#         # Convert the index to datetime
#         data.index = pd.to_datetime(data.index)

#         # If the 'Date' column is expected to be the index after fetching data
#         if 'Date' not in data.columns and data.index.name == 'Date':
#             data.reset_index(inplace=True)
#         elif 'Date' not in data.columns:
#             print("Date column is still missing. Please check the DataFrame source.")

#         if interval.endswith('mo'):
#             # Check if the last data point is from the current month and year
#             if data.index[-1].month == today_datetime.month and data.index[-1].year == today_datetime.year:
#                 # Count the number of days in the current month up to today
#                 days_this_month_so_far = today_datetime.day

#             # If the number of days in the last month is less than 20, drop the last month's data
#             if days_this_month_so_far < 20:
#                 data = data.iloc[:-1].copy()

#         elif interval.endswith('wk'):
#             weekdays_till_today = (today_datetime.weekday() + 1) % 7
#             # If the number of days in the last week is less than 3, drop the last week's data
#             if weekdays_till_today < 3:  # 0: Monday, ..., 6: Sunday
#                 data = data.iloc[:-1].copy()

#         data.dropna(inplace=True)
#         data = data[data['Volume'] != 0].copy()

#         # If "Close" only = true, drop all columns except "Close"
#         if target_only:
#             data.drop(['Open', 'High', 'Low', 'Volume', 'Adj Close'], axis=1, inplace=True)
#         else:
#             data.drop(['Adj Close'], axis=1, inplace=True)

#         # Add a column to identify the symbol
#         data['Symbol'] = symbol
        
#         # Append the data to the all_data list
#         all_data.append(data)

#     # Concatenate all_data into a single DataFrame 
#     all_data_df = pd.concat(all_data)
    
#     return all_data_df

# # def normalize_data(features, target):
# #     """ Normalize features and target separately. """
# #     feature_mean = features.mean()
# #     feature_std = features.std()
# #     target_mean = target.mean()
# #     target_std = target.std()

# #     normalized_features = (features - feature_mean) / feature_std
# #     normalized_target = (target - target_mean) / target_std

# #     return normalized_features, normalized_target, feature_mean, feature_std, target_mean, target_std

# # def normalize_data(features, target):
# #     """ Normalize the dataframe using mean and standard deviation, assumes features are all numeric. """
# #     # Exclude datetime and other non-numeric columns from normalization
# #     numeric_features = features.select_dtypes(include=[np.number])
    
# #     feature_mean = numeric_features.mean()
# #     feature_std = numeric_features.std()

# #     # Apply normalization only to numeric features
# #     normalized_features = (numeric_features - feature_mean) / feature_std

# #     # Normalize the target if it's numeric
# #     target_mean = target.mean() if pd.api.types.is_numeric_dtype(target) else None
# #     target_std = target.std() if pd.api.types.is_numeric_dtype(target) else None
# #     normalized_target = (target - target_mean) / target_std if pd.api.types.is_numeric_dtype(target) else target

# #     return normalized_features, normalized_target, feature_mean, feature_std, target_mean, target_std

# def normalize_data(features, target):
#     """ Assumes features are already filtered to numeric types. """
#     feature_mean = features.mean()
#     feature_std = features.std()
#     normalized_features = (features - feature_mean) / feature_std

#     target_mean = target.mean()
#     target_std = target.std()
#     normalized_target = (target - target_mean) / target_std

#     return normalized_features, normalized_target, target_mean, target_std

# def inverse_normalize(prediction, target_mean, target_std):
#     """ Revert the normalization for a prediction. """
#     return prediction * target_std + target_mean

# def create_lagged_rows(data, num_lags, target_col, time_column='Date', scale_params=None):
#     """
#     Create lagged rows for all columns except the specified target column,
#     and include the current time from the specified time column.
#     Optionally use scale parameters if provided.
#     """
#     # Create a list to hold the lagged data rows
#     lagged_data_rows = []
    
#     # Determine which columns are features based on exclusion of the target and time columns
#     feature_columns = [col for col in data.columns if col != target_col and col != time_column]

#     for i in range(num_lags, len(data)):
#         row = {}
        
#         # Create lags for each feature
#         for feature in feature_columns:
#             for lag in range(0, num_lags + 1):
#                 row[f'{feature}_lag_{lag}'] = data[feature].iloc[i - lag]
        
#         # Create lags for the target column
#         for lag in range(1, num_lags + 1):
#             row[f'{target_col}_lag_{lag}'] = data[target_col].iloc[i - lag]
        
#         # Include the target column as 'Target' for lag_0
#         row['Target'] = data[target_col].iloc[i]
        
#         # Optionally append scale parameters if needed for inverse transformations later
#         if scale_params:
#             row['scale_params'] = scale_params  # assuming scale_params is a dictionary or similar structure

#         lagged_data_rows.append(row)
    
#     return pd.DataFrame(lagged_data_rows)


# def split_data(X, y, train_frac=0.7, valid_frac=0.15, random_state=False):
#     """ Split the data into train, validation, and test sets. """
#     total_count = X.shape[0]
#     train_size = int(total_count * train_frac)
#     valid_size = int(total_count * valid_frac)
#     # test_size = total_count - train_size - valid_size

#     if random_state:

#         indices = np.random.permutation(total_count)
#         train_indices = indices[:train_size]
#         valid_indices = indices[train_size:train_size + valid_size]
#         test_indices = indices[train_size + valid_size:]

#         X_train, y_train = X[train_indices], y[train_indices]
#         X_valid, y_valid = X[valid_indices], y[valid_indices]
#         X_test, y_test = X[test_indices], y[test_indices]

#     else:
#         X_train, y_train = X[:train_size], y[:train_size]
#         X_valid, y_valid = X[train_size:train_size + valid_size], y[train_size:train_size + valid_size]
#         X_test, y_test = X[train_size + valid_size:], y[train_size + valid_size:]

#     return X_train, y_train, X_valid, y_valid, X_test, y_test

# def convert_to_tensors(*arrays):
#     tensors = []
#     for array in arrays:
#         if isinstance(array, pd.DataFrame):
#             # Convert all entries to numeric, coercing errors to NaN, then fill with zeros
#             array = array.apply(pd.to_numeric, errors='coerce').fillna(0)
#         elif isinstance(array, pd.Series):
#             array = pd.to_numeric(array, errors='coerce').fillna(0)
#         tensor = torch.tensor(array.values, dtype=torch.float32)
#         tensors.append(tensor)
#     return tuple(tensors)

# # def prepare_time_series_data(data=None, group_col=None, sort_col=None, target_col=None, num_lags=None, symbol_list=None, 
# #                              start_date=None, interval=None, train_frac=None, valid_frac=None):
# #     all_features = []
# #     all_labels = []
# #     all_scale_params = {}  # To store scaling parameters for each group

# #     # Check if data is properly initialized
# #     if data is None:
# #         print("Fetching data because no data was provided...")
# #         data = fetch_data(symbol_list=symbol_list, start_date=start_date, interval=interval)
# #     else:
# #         print(f"Data provided. Type: {type(data)}")  # Debug the type of data

# #     # Ensure data is a DataFrame
# #     if not isinstance(data, pd.DataFrame):
# #         print("Error: Data is not a DataFrame. Check the data input.")
# #         return None

# #     for group_value in data[group_col].unique():
# #         group_data = data[data[group_col] == group_value]
# #         group_data = group_data.sort_values(sort_col)
# #         group_data = group_data.drop(columns=[group_col])  # Remove only the grouping column

# #         # Normalize data
# #         if target_col in group_data.columns:
# #             target = group_data[target_col]
# #             features = group_data.drop(columns=[target_col])

# #             normalized_features, normalized_target, feature_mean, feature_std, target_mean, target_std = normalize_data(features, target)
# #             all_scale_params[group_value] = {
# #                 'feature_mean': feature_mean, 'feature_std': feature_std,
# #                 'target_mean': target_mean, 'target_std': target_std
# #             }

# #             normalized_data = normalized_features
# #             normalized_data[target_col] = normalized_target  
# #         else:
# #             print("Target column missing from data")
# #             continue

# #         if sort_col in group_data.columns:
# #             lagged_data = create_lagged_rows(normalized_data, num_lags, target_col, sort_col, all_scale_params[group_value])
# #         else:
# #             print(f"'{sort_col}' column not found in data.")
# #             continue

# #         features = lagged_data.drop(columns=['Target'])  # All but the target column as features
# #         labels = lagged_data['Target']  # Target column

# #         all_features.append(features)
# #         all_labels.append(labels)

# #     all_features = pd.concat(all_features)
# #     all_labels = pd.concat(all_labels)

# #     X_train, y_train, X_valid, y_valid, X_test, y_test = split_data(all_features, all_labels, train_frac=train_frac, valid_frac=valid_frac)

# #     X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, X_test_tensor, y_test_tensor = convert_to_tensors(X_train, y_train, X_valid, y_valid, X_test, y_test)

# #     return X_train_tensor, y_train_tensor, X_valid_tensor, y_valid_tensor, X_test_tensor, y_test_tensor, all_scale_params

# def generate_future_prediction_data(latest_data, last_used_data, num_lags, target_feature):
#     future_data = {}
#     # Set lag_0 for non-target features directly from latest_data
#     for feature in latest_data.index:
#         if feature != target_feature:
#             future_data[f'{feature}_lag_0'] = latest_data[feature]

#     # Handle the target feature separately
#     # Shift remaining lags for non-target features
#     for feature in latest_data.index:
#         if feature != target_feature:
#             for lag in range(1, num_lags + 1):
#                 future_data[f'{feature}_lag_{lag}'] = last_used_data.get(f'{feature}_lag_{lag - 1}', None)

#     # Target feature: replace lag_1 with the target from latest_data and shift other lags
#     future_data[f'{target_feature}_lag_1'] = latest_data[target_feature]
#     for lag in range(2, num_lags + 1):
#         future_data[f'{target_feature}_lag_{lag}'] = last_used_data.get(f'{target_feature}_lag_{lag - 1}', None)

#     return future_data


# # def prepare_time_series_data(data=None, group_col=None, sort_col=None, target_col=None, num_lags=None, symbol_list=None, 
# #                              start_date=None, interval=None, train_frac=0.7, valid_frac=0.15):
# #     if data is None:
# #         data = fetch_data(symbol_list=symbol_list, start_date=start_date, interval=interval)

# #     if not isinstance(data, pd.DataFrame):
# #         return None  # Handle non-DataFrame input robustly

# #     all_features = []
# #     all_labels = []
# #     all_scale_params = {}
# #     future_data = {}

# #     for group_value in data[group_col].unique():
# #         group_data = data[data[group_col] == group_value].sort_values(sort_col).drop(columns=[group_col])
# #         if target_col in group_data.columns:
# #             target = group_data[target_col]
# #             features = group_data.drop(columns=[target_col])
# #             normalized_features, normalized_target, feature_mean, feature_std, target_mean, target_std = normalize_data(features, target)
# #             all_scale_params[group_value] = {
# #                 'feature_mean': feature_mean, 'feature_std': feature_std,
# #                 'target_mean': target_mean, 'target_std': target_std
# #             }
# #             normalized_data = normalized_features
# #             normalized_data[target_col] = normalized_target
# #             lagged_data = create_lagged_rows(normalized_data, num_lags, target_col, sort_col, all_scale_params[group_value])
            
# #             # Generate future data for predictions using the latest data point
# #             if len(lagged_data) > num_lags:
# #                 latest_data = group_data.iloc[-1]  # Assume the latest raw data is needed
#                 # last_used_data = lagged_data.iloc[-(num_lags + 1)]  # Last used lagged data point
#                 # future_prediction_data = generate_future_prediction_data(latest_data, last_used_data, num_lags)
# #                 future_data[group_value] = future_prediction_data
# #                 lagged_data = lagged_data.iloc[:-num_lags]  # Use all but the last 'num_lags' entries for training

# #             features = lagged_data.drop(columns=['Target'])
# #             labels = lagged_data['Target']
# #             all_features.append(features)
# #             all_labels.append(labels)
# #         else:
# #             continue  # Skip if no target column

# #     all_features = pd.concat(all_features)
# #     all_labels = pd.concat(all_labels)

# #     X_train, y_train, X_valid, y_valid, X_test, y_test = split_data(all_features, all_labels, train_frac, valid_frac)

# #     return X_train, y_train, X_valid, y_valid, X_test, y_test, all_scale_params, future_data

# def prepare_time_series_data(data=None, group_col=None, sort_col=None, target_col=None, num_lags=None, symbol_list=None, 
#                              start_date=None, interval=None, train_frac=0.7, valid_frac=0.15):
#     if data is None:
#         data = fetch_data(symbol_list=symbol_list, start_date=start_date, interval=interval)

#     if not isinstance(data, pd.DataFrame):
#         return None  # Ensure only DataFrame inputs are processed

#     all_features = []
#     all_labels = []
#     all_scale_params = {}
#     future_data = {}

#     for group_value in data[group_col].unique():
#         group_data = data[data[group_col] == group_value].sort_values(sort_col).drop(columns=[group_col])
#         if target_col in group_data.columns:
#             target = group_data[target_col]
#             numeric_features = group_data.drop(columns=[target_col])

#         normalized_features, normalized_target, target_mean, target_std = normalize_data(numeric_features, target)
#         all_scale_params[group_value] = {'target_mean': target_mean,'target_std': target_std}
        
#         # Reconstruct normalized data including datetime columns for proper indexing
#         normalized_data = normalized_features.copy()
#         normalized_data[target_col] = normalized_target
#         normalized_data[sort_col] = group_data[sort_col]  # Preserve datetime column for sorting and future use

#         lagged_data = create_lagged_rows(normalized_data, num_lags, target_col, sort_col, all_scale_params[group_value])

#         # Generate future data for predictions using the latest data point
#         if len(lagged_data) > num_lags:
#             latest_data = normalized_data.iloc[-1]
#             last_used_data = lagged_data.iloc[-(num_lags + 1)]  # Last used lagged data point
#             future_prediction_data = generate_future_prediction_data(latest_data, last_used_data, num_lags, target_col)
#             future_data[group_value] = future_prediction_data
#             # lagged_data = lagged_data.iloc[:-num_lags]  # Use all but the last 'num_lags' entries for training

#         features = lagged_data.drop(columns=['Target'])
#         labels = lagged_data['Target']
#         all_features.append(features)
#         all_labels.append(labels)

#     all_features = pd.concat(all_features)
#     all_labels = pd.concat(all_labels)

#     X_train, y_train, X_valid, y_valid, X_test, y_test = split_data(all_features, all_labels, train_frac, valid_frac)

#     return X_train, y_train, X_valid, y_valid, X_test, y_test, all_scale_params, future_data



In [29]:
import pandas as pd
import numpy as np
import yfinance as yf
import torch
from datetime import datetime
import matplotlib.pyplot as plt


def calculate_indicators_ver3(data, interval=None):
    # Adjust parameters based on frequency
    if interval == '1wk':  # Weekly data
        period = 6
        rsi_period = 9
        short_ema_period = 8
        long_ema_period = 16
        signal_period = 5
        vol_period = 14
        bband_window_size = 12
    elif interval == '1mo':  # Monthly data
        period = 4
        rsi_period = 7
        short_ema_period = 6
        long_ema_period = 12
        signal_period = 4
        vol_period = 10
        bband_window_size = 8
    elif interval == '1d':  # Monthly data
        period = 7
        rsi_period = 14
        short_ema_period = 12
        long_ema_period = 26
        signal_period = 9
        vol_period = 26
        bband_window_size = 20
    elif interval == None:  # not specified
        period = 5
        rsi_period = 7
        short_ema_period = 6
        long_ema_period = 13
        signal_period = 4
        vol_period = 13
        bband_window_size = 10

    # if data.shape[0] < long_ema_period:
    #     return None
    
    # Check if 'Close' column exists in the data
    if 'Close' in data.columns:
        # Calculate daily returns
        data['Return'] = data['Close'].pct_change()
        # Calculate daily Direction
        data["Direction_Signal"] = np.where((data['Return']<-0.001), 1, np.where((data['Return']>0.001), 3, 2))
        # Calculate VPT (Volume-Price Trend)
        data['VPT'] = (data['Volume'] * data['Return']).cumsum()
        # Price Range
        data['Price Range'] = data['High'] - data['Low']
        # Candle Body Size
        data['Body Size'] = data['Open'] - data['Close']
        # Upper Shadow Size
        data['Upper Shadow'] = data.apply(lambda row: row.High - max(row.Open, row.Close), axis=1)
        # Lower Shadow Size
        data['Lower Shadow'] = data.apply(lambda row: min(row.Open, row.Close) - row.Low, axis=1)

        # Calculate ZLEMA
        lag = (period - 1) // 2  # Ensure integer division
        ema = data['Close'].ewm(span=10, adjust=False).mean()
        data['ZLEMA'] = ema + (ema - ema.shift(lag))

        # Define tolerance as a percentage of the close price
        tolerance = 0.015

        # Initialize column with 0s
        data['ZLEMA_Signal'] = 2

        # Buy signal when Close is above (1 + tolerance) * ZLEMA
        data.loc[data['Close'] > data['ZLEMA'] * (1 + tolerance), 'ZLEMA_Signal'] = 3

        # Sell signal when Close is below (1 - tolerance) * ZLEMA
        data.loc[data['Close'] < data['ZLEMA'] * (1 - tolerance), 'ZLEMA_Signal'] = 1

        # Calculate the first exponential moving average (EMA1).
        ema1 = data['Close'].ewm(span=period, min_periods=period).mean()
        # Calculate the second exponential moving average (EMA2).
        ema2 = ema1.ewm(span=period, min_periods=period).mean()
        # Calculate the third exponential moving average (EMA3).
        ema3 = ema2.ewm(span=period, min_periods=period).mean()
        # Calculate the TEMA.
        tema = 3 * ema1 - 3 * ema2 + ema3

        # Convert the TEMA to a Pandas Series.
        data['TEMA'] = tema

        # Initialize column with 0s
        data['TEMA_Signal'] = 2

        # Buy signal when Close is above (1 + tolerance) * TEMA
        data.loc[data ['Close']> data ['TEMA']* (1 + tolerance), 'TEMA_Signal']= 3

        # Sell signal when Close is below (1 - tolerance) * TEMA
        data.loc[data ['Close']< data ['TEMA']*(1-tolerance), 'TEMA_Signal']= 1

        # Calculate RSI and its signal
        delta = data['Close'].diff()
        up, down = delta.copy(), delta.copy()

        up[up < 0] = 0
        down[down > 0] = 0

        average_gain = up.rolling(window=rsi_period).mean()
        average_loss = abs(down.rolling(window=rsi_period).mean())

        epsilon = 1e-10  # A small constant to prevent division by zero
        rs = average_gain / (average_loss + epsilon)

        data['RSI'] =100 - (100 / (1 + rs))
        data["RSI_Signal"] =2
        data.loc[data.RSI < 30,"RSI_Signal"] =3
        data.loc[data.RSI > 70,"RSI_Signal"] =1

        # # Calculate MACD Line: (12-day EMA - 26-day EMA)
        EMA_short = data['Close'].ewm(span=short_ema_period).mean()
        EMA_long = data['Close'].ewm(span=long_ema_period).mean()
        data['MACD_Line'] = EMA_short - EMA_long

        # Calculate Signal Line: a n-day MA of MACD Line
        data['Signal_Line'] = data["MACD_Line"].ewm(span=signal_period).mean()

        # Generate MACD signals based on crossovers
        data["MACD_Signal"] = 2 # Neutral
        data.loc[(data.MACD_Line > data.Signal_Line) & (data.MACD_Line.shift() < data.Signal_Line.shift()),"MACD_Signal"] = 3 # Buy
        data.loc[(data.MACD_Line < data.Signal_Line) & (data.MACD_Line.shift() > data.Signal_Line.shift()),"MACD_Signal"] = 1 # Sell

        # Generate EMA signals based on crossovers
        data["EMA_Short"]=EMA_short
        data["EMA_Long"]=EMA_long

        # Define a small percentage as a threshold
        ema_diff_threshold = 0.01 # 1% difference

        # Calculate absolute percent difference between short and long EMA
        data['EMA_Diff'] = (EMA_short - EMA_long) / EMA_long

        data["EMA_Signal"] = np.where((data["EMA_Short"] > data["EMA_Long"]) & (abs(data["EMA_Diff"]) > ema_diff_threshold), 3, 
                             np.where((data["EMA_Short"] < data["EMA_Long"]) & (abs(data["EMA_Diff"]) > ema_diff_threshold), 1, 2))

        # Calculate Volatility as rolling standard deviation of log returns
        data["Log_Return"] =(np.log(data.Close).diff())
        data["Volatility"] =(data.Log_Return.rolling(window=vol_period, center=False).std())

        # Generate Volatility signals based on high or low volatility
        high_vol_threshold = data["Volatility"].quantile(0.85)
        low_vol_threshold = data["Volatility"].quantile(0.15)
        
        data['Volatility_Signal'] = 2
        data.loc[data.Volatility > high_vol_threshold, 'Volatility_Signal'] = 3
        data.loc[data.Volatility < low_vol_threshold, 'Volatility_Signal'] = 1

        # Calculate Bollinger Bands
        data["Middle_Band"] = data["Close"].rolling(window=bband_window_size, center=False).mean()
        data["Std_Dev"] = data["Close"].rolling(window=bband_window_size, center=False).std()
        data["Upper_Band"] = data["Middle_Band"] + data["Std_Dev"] * 2
        data["Lower_Band"] = data["Middle_Band"] - data["Std_Dev"] * 2

        # Identify shifting points
        data["BBand_Signal"] = 2  # neutral

        # Create a copy of the "Close" column
        close_shifted = data["Close"].copy()

        # Check for crossing Upper Band
        cross_upper_band = (close_shifted.shift(1) <= data["Upper_Band"].shift(1)) & (data["Close"] > data["Upper_Band"])
        data.loc[cross_upper_band, "BBand_Signal"] = 3  # buy

        # Check for crossing Lower Band
        cross_lower_band = (close_shifted.shift(1) >= data["Lower_Band"].shift(1)) & (data["Close"] < data["Lower_Band"])
        data.loc[cross_lower_band, "BBand_Signal"] = 1  # sell

        # Drop unnecessary columns
        data = data.drop(columns=["Middle_Band", "Std_Dev", 'Log_Return', 'Open', 'Low', 'High', 'Volume'])

    else:
        print("'Close' column is not present in the input dataframe.")

def fetch_data(symbol_list, start_date, end_date=None, interval='1d', target_only=False):
    # Initialize an empty DataFrame
    all_data = []

    # Keep 'today' as a datetime object for datetime operations
    today_datetime = datetime.now()
    # Convert to string only when needed for string operations
    today_str = today_datetime.strftime('%Y-%m-%d')

    if end_date is None:
        end_date = today_str

    for symbol in symbol_list:
        try:
            data = yf.download(symbol, start=start_date, end=end_date, interval=interval)
        except KeyError:
            print(f"Data for {symbol} not found.")
            continue

        data = data.copy()

        # Convert the index to datetime
        data.index = pd.to_datetime(data.index)

        # If the 'Date' column is expected to be the index after fetching data
        if 'Date' not in data.columns and data.index.name == 'Date':
            data.reset_index(inplace=True)
        elif 'Date' not in data.columns:
            print("Date column is still missing. Please check the DataFrame source.")

        if interval.endswith('mo'):
            # Check if the last data point is from the current month and year
            if data.index[-1].month == today_datetime.month and data.index[-1].year == today_datetime.year:
                # Count the number of days in the current month up to today
                days_this_month_so_far = today_datetime.day

            # If the number of days in the last month is less than 20, drop the last month's data
            if days_this_month_so_far < 20:
                data = data.iloc[:-1].copy()

        elif interval.endswith('wk'):
            weekdays_till_today = (today_datetime.weekday() + 1) % 7
            # If the number of days in the last week is less than 3, drop the last week's data
            if weekdays_till_today < 3:  # 0: Monday, ..., 6: Sunday
                data = data.iloc[:-1].copy()

        data.dropna(inplace=True)
        data = data[data['Volume'] != 0].copy()

        # If "Close" only = true, drop all columns except "Close"
        if target_only:
            data.drop(['Open', 'High', 'Low', 'Volume', 'Adj Close'], axis=1, inplace=True)
        else:
            data.drop(['Adj Close'], axis=1, inplace=True)

        # Add a column to identify the symbol
        data['Symbol'] = symbol
        
        # Append the data to the all_data list
        all_data.append(data)

    # Concatenate all_data into a single DataFrame 
    all_data_df = pd.concat(all_data)

    return all_data_df

def normalize_data(features, target):
    """ Normalize the dataframe using mean and standard deviation, assumes features are all numeric. """
    # Exclude datetime and other non-numeric columns from normalization
    numeric_features = features.select_dtypes(include=[np.number])
    
    feature_mean = numeric_features.mean()
    feature_std = numeric_features.std()

    # Apply normalization only to numeric features
    normalized_features = (numeric_features - feature_mean) / feature_std

    # Normalize the target if it's numeric
    target_mean = target.mean() if pd.api.types.is_numeric_dtype(target) else None
    target_std = target.std() if pd.api.types.is_numeric_dtype(target) else None
    normalized_target = (target - target_mean) / target_std if pd.api.types.is_numeric_dtype(target) else target

    return normalized_features, normalized_target, target_mean, target_std

def inverse_normalize(prediction, target_mean, target_std):
    """ Revert the normalization for a prediction. """
    return prediction * target_std + target_mean
import pandas as pd

def create_lagged_rows(data, num_lags, target_col, time_column='Date', scale_params=None):
    """
    Create lagged rows for all columns except the specified target column,
    and include the current time from the specified time column.
    Optionally use scale parameters if provided.
    """
    # Create a list to hold the lagged data rows
    lagged_data_rows = []
    
    # Determine which columns are features based on exclusion of the target and time columns
    feature_columns = [col for col in data.columns if col != target_col and col != time_column]

    for i in range(num_lags, len(data)):
        row = {}
        
        # Create lags for each feature
        for feature in feature_columns:
            for lag in range(0, num_lags + 1):
                row[f'{feature}_lag_{lag}'] = data[feature].iloc[i - lag]
        
        # Create lags for the target column
        for lag in range(1, num_lags + 1):
            row[f'{target_col}_lag_{lag}'] = data[target_col].iloc[i - lag]
        
        # Include the target column as 'Target' for lag_0
        row['Target'] = data[target_col].iloc[i]
        
        # Optionally append scale parameters if needed for inverse transformations later
        # if scale_params:
        #     row['scale_params'] = scale_params  # assuming scale_params is a dictionary or similar structure

        lagged_data_rows.append(row)
    
    return pd.DataFrame(lagged_data_rows)

def impute_nulls(df):
    """
    Impute null values in a Pandas DataFrame based on the data type of each column.
    - For float columns, impute with the mean.
    - For integer columns, impute with the median.
    - For object columns, impute with the mode.
    - For datetime columns, impute with the most recent or most frequent date.
    - For timedelta columns, impute with the mode.
    - For bool columns, impute with the mode.
    - For category columns, impute with the mode.
    - For complex columns, impute with the mean.
    """
    # Get data types of all columns
    dtypes = df.dtypes

    # Iterate over all columns
    for col in df.columns:
        # Check if column contains null values
        if df[col].isnull().sum() > 0:
            # Get data type of column
            dtype = dtypes[col]
            # Impute null values based on data type
            if dtype == 'float64' or dtype == 'float32' or dtype == 'float16':
                df[col] = df[col].fillna(df[col].mean())
            elif dtype == 'int64' or dtype == 'int32' or dtype == 'int16' or dtype == 'int8':
                df[col] = df[col].fillna(df[col].median())
            elif dtype == 'object':
                df[col] = df[col].fillna(df[col].mode()[0])
            elif dtype == 'datetime64':
                # For datetime, choosing to backfill, but can also forward fill or use a specific value
                df[col] = df[col].fillna(method='bfill')
            elif dtype == 'timedelta64':
                df[col] = df[col].fillna(df[col].mode()[0])
            elif dtype == 'bool':
                df[col] = df[col].fillna(df[col].mode()[0])
            elif dtype.name == 'category':
                df[col] = df[col].fillna(df[col].mode()[0])
            elif dtype == 'complex64' or dtype == 'complex128':
                df[col] = df[col].fillna(df[col].mean())
    return df


def split_data(X, y, train_frac=None, valid_frac=None, random_state=False):
    """ Split the data into train, validation, and test sets. """
    total_count = X.shape[0]
    train_size = int(total_count * train_frac)
    valid_size = int(total_count * valid_frac)
    # test_size = total_count - train_size - valid_size

    if random_state:

        indices = np.random.permutation(total_count)
        train_indices = indices[:train_size]
        valid_indices = indices[train_size:train_size + valid_size]
        test_indices = indices[train_size + valid_size:]

        X_train, y_train = X[train_indices], y[train_indices]
        X_valid, y_valid = X[valid_indices], y[valid_indices]
        X_test, y_test = X[test_indices], y[test_indices]

    else:
        X_train, y_train = X[:train_size], y[:train_size]
        X_valid, y_valid = X[train_size:train_size + valid_size], y[train_size:train_size + valid_size]
        X_test, y_test = X[train_size + valid_size:], y[train_size + valid_size:]

    return X_train, y_train, X_valid, y_valid, X_test, y_test

def convert_to_tensors(*arrays):
    tensors = []
    for array in arrays:
        if isinstance(array, pd.DataFrame):
            # Convert all entries to numeric, coercing errors to NaN, then fill with zeros
            array = array.apply(pd.to_numeric, errors='coerce').fillna(0)
        elif isinstance(array, pd.Series):
            array = pd.to_numeric(array, errors='coerce').fillna(0)
        tensor = torch.tensor(array.values, dtype=torch.float32)
        tensors.append(tensor)
    return tuple(tensors)

# def generate_future_prediction_data(latest_data, last_used_data, num_lags, target_feature):
#     future_data = {}
    
#     # We will start by setting up the lag_1 data from the latest_data
#     for feature in latest_data.index:
#         if feature != 'Date':  # We're not including 'Date' in the future_data
#             # For lag_1, we use the latest data available for all features including the target
#             future_data[f'{feature}_lag_1'] = latest_data[feature]

#     # Then we set up lag_2 to num_lags data from the last_used_data
#     for lag in range(2, num_lags + 1):
#         for feature in latest_data.index:
#             if feature != 'Date':  # Again, skipping 'Date'
#                 # Retrieve the last used data for the previous lag, if it exists, otherwise default to None
#                 future_data[f'{feature}_lag_{lag}'] = last_used_data.get(f'{feature}_lag_{lag - 1}', None)
    
#     # Order the future_data to ensure that the sequence is Open_lag_1, Open_lag_2, etc.
#     ordered_future_data = {}
#     for feature in latest_data.index:
#         if feature != 'Date':
#             for lag in range(1, num_lags + 1):
#                 ordered_future_data[f'{feature}_lag_{lag}'] = future_data.get(f'{feature}_lag_{lag}')

#     return ordered_future_data

def generate_future_prediction_data(latest_data, last_used_data, num_lags, target_feature):
    future_data = {}
    
    # Set up lag_0 with latest data for all features except the target column
    for feature in latest_data.index:
        if feature != 'Date' and feature != target_feature:  
            future_data[f'{feature}_lag_0'] = latest_data[feature]
    
    # Set up lag_1 with latest data for all features including the target column
    for feature in latest_data.index:
        if feature != 'Date':  
            future_data[f'{feature}_lag_1'] = latest_data[feature]
    
    # Set up lag_2 to num_lags data from the last_used_data
    for lag in range(2, num_lags + 1):
        for feature in latest_data.index:
            if feature != 'Date':  
                # Retrieve the last used data for the previous lag, if it exists, otherwise default to None
                future_data[f'{feature}_lag_{lag}'] = last_used_data.get(f'{feature}_lag_{lag - 1}', None)
    
    # Order the future_data to ensure that the sequence is Open_lag_1, Open_lag_2, etc.
    ordered_future_data = {}
    for feature in latest_data.index:
        if feature != 'Date':
            for lag in range(0, num_lags + 1):
                if not (lag == 0 and feature == target_feature):  # Exclude target feature for lag 0
                    ordered_future_data[f'{feature}_lag_{lag}'] = future_data.get(f'{feature}_lag_{lag}')

    return ordered_future_data


def prepare_time_series_data(data=None, group_col=None, sort_col=None, target_col=None, num_lags=None, symbol_list=None, 
                             start_date=None, interval=None, train_frac=0.7, valid_frac=0.15, indicator_use=True):
    if data is None:
        data = fetch_data(symbol_list=symbol_list, start_date=start_date, interval=interval)

    if not isinstance(data, pd.DataFrame):
        return None  # Ensure only DataFrame inputs are processed

    all_features = []
    all_labels = []
    all_scale_params = {}
    future_data = {}

    for group_value in data[group_col].unique():
        group_data = data[data[group_col] == group_value].sort_values(sort_col).drop(columns=[group_col])

        if indicator_use:
            group_data = calculate_indicators_ver3(group_data, interval=interval)        
            # Handling NaNs before proceeding
            group_data.dropna(inplace=True) 

        if target_col in group_data.columns:
            target = group_data[target_col]
            numeric_features = group_data.drop(columns=[target_col])

        normalized_features, normalized_target, target_mean, target_std = normalize_data(numeric_features, target)
        all_scale_params[group_value] = {'target_mean': target_mean,'target_std': target_std}

        # Reconstruct normalized data including datetime columns for proper indexing
        normalized_data = normalized_features.copy()
        normalized_data[target_col] = normalized_target
        # normalized_data[sort_col] = group_data[sort_col]  # Preserve datetime column for sorting and future use

        lagged_data = create_lagged_rows(normalized_data, num_lags, target_col, sort_col, all_scale_params[group_value])

        # Generate future data for predictions using the latest data point
        if len(lagged_data) > num_lags:
            latest_data = normalized_data.iloc[-1]
            # print(latest_data)
            last_used_data = lagged_data.iloc[-1]  # Last used lagged data point
            # print(last_used_data)
            future_prediction_data = generate_future_prediction_data(latest_data, last_used_data, num_lags, target_col)
            future_data[group_value] = future_prediction_data
            future_data_df = pd.DataFrame.from_dict(future_data)
            future_data_impute = impute_nulls(future_data_df)
            # lagged_data = lagged_data.iloc[:-num_lags]  # Use all but the last 'num_lags' entries for training

        features = lagged_data.drop(columns=['Target'])
        labels = lagged_data['Target']
        all_features.append(features)
        all_labels.append(labels)

    all_features = pd.concat(all_features)
    all_labels = pd.concat(all_labels)

    X_train, y_train, X_valid, y_valid, X_test, y_test = split_data(all_features, all_labels, train_frac, valid_frac)

    return X_train, y_train, X_valid, y_valid, X_test, y_test, all_scale_params, future_data_impute


In [30]:
import torch
import os

data=None
group_col='Symbol'
sort_col='Date'
target_col='Close'
num_lags=10
symbol_list=ETF_symbol_list
start_date='2001-01-01'
end_date='2024-04-18'
interval='1d'
train_frac=0.70
valid_frac=0.20
indicator_use=False

X_train, y_train, X_valid, y_valid, X_test, y_test, scale_params, future_data = prepare_time_series_data(data=data, group_col=group_col, sort_col=sort_col, 
                                                                    target_col=target_col, num_lags=num_lags, symbol_list=symbol_list, start_date=start_date, 
                                                                    interval=interval, train_frac=train_frac, valid_frac=valid_frac, indicator_use=indicator_use)

output_file_path = r"C:\Users\young78703\Documents\GitHub\Machine-Learning-Projects\Data_save\Time_Series_Lag\stock_ETF_100_list_lag10_split_0.7_future_daily_04182024"
# output_file_path = "/home/predict_price/stock_price/data_save/daily/ver5.5/test/test"

def reformat_scale_params(scale_params):
    reformatted_params = {
        'Symbol': [],
        'Target_Mean': [],
        'Target_Std': []
    }

    # Loop through each symbol in the scale_params
    for symbol, params in scale_params.items():
        reformatted_params['Symbol'].append(symbol)
        
        # Assuming 'target_mean' and 'target_std' are single numeric values, append them directly
        reformatted_params['Target_Mean'].append(params['target_mean'])
        reformatted_params['Target_Std'].append(params['target_std'])

    return pd.DataFrame(reformatted_params)

def reformat_future_data(future_data):
    reformatted_data = []
    for symbol, data in future_data.items():
        row = {'Symbol': symbol}
        row.update(data)  # Assuming data is a dictionary
        reformatted_data.append(row)
    return pd.DataFrame(reformatted_data)

# Create the directory if it does not exist
if not os.path.exists(output_file_path):
    os.makedirs(output_file_path)
# Apply the reformatting function to scale_params
scale_params_df = reformat_scale_params(scale_params)
scale_params_df.to_csv(os.path.join(output_file_path, 'scale_params.csv'), index=False)
future_data_df = reformat_future_data(future_data)
future_data_df.to_csv(os.path.join(output_file_path, 'future_data.csv'), index=False)

# try:
#     torch.save(X_train, os.path.join(output_file_path, 'X_train.pt'))
#     torch.save(y_train, os.path.join(output_file_path, 'y_train.pt'))
#     torch.save(X_valid, os.path.join(output_file_path, 'X_valid.pt'))
#     torch.save(y_valid, os.path.join(output_file_path, 'y_valid.pt'))
#     torch.save(X_test, os.path.join(output_file_path, 'X_test.pt'))
#     torch.save(y_test, os.path.join(output_file_path, 'y_test.pt'))
# except Exception as e:
#     print(f"Failed to save files due to {e}")

try:
    X_train.to_csv(os.path.join(output_file_path, 'X_train.csv'), index=False)
    y_train.to_csv(os.path.join(output_file_path, 'y_train.csv'), index=False)
    X_valid.to_csv(os.path.join(output_file_path, 'X_valid.csv'), index=False)
    y_valid.to_csv(os.path.join(output_file_path, 'y_valid.csv'), index=False)
    X_test.to_csv(os.path.join(output_file_path, 'X_test.csv'), index=False)
    y_test.to_csv(os.path.join(output_file_path, 'y_test.csv'), index=False)
except Exception as e:
    print(f"Failed to save files due to {e}")




[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

In [ ]:
output_file_path = r"C:\Users\young78703\Documents\GitHub\Machine-Learning-Projects\Data_save\Time_Series_Lag\stock_SP_list_lag10_with_future_daily"
# output_file_path = "/home/predict_price/stock_price/data_save/daily/ver5.5/test/test"


def reformat_scale_params(scale_params):
    reformatted_params = {
        'Symbol': [],
        'Target_Mean': [],
        'Target_Std': []
    }

    # Loop through each symbol in the scale_params
    for symbol, params in scale_params.items():
        reformatted_params['Symbol'].append(symbol)
        
        # Assuming 'target_mean' and 'target_std' are single numeric values, append them directly
        reformatted_params['Target_Mean'].append(params['target_mean'])
        reformatted_params['Target_Std'].append(params['target_std'])

    return pd.DataFrame(reformatted_params)

def reformat_future_data(future_data):
    reformatted_data = []
    for symbol, data in future_data.items():
        row = {'Symbol': symbol}
        row.update(data)  # Assuming data is a dictionary
        reformatted_data.append(row)
    return pd.DataFrame(reformatted_data)

# Create the directory if it does not exist
if not os.path.exists(output_file_path):
    os.makedirs(output_file_path)
# Apply the reformatting function to scale_params
scale_params_df = reformat_scale_params(scale_params)
scale_params_df.to_csv(os.path.join(output_file_path, 'scale_params.csv'), index=False)
future_data_df = reformat_future_data(future_data)
future_data_df.to_csv(os.path.join(output_file_path, 'future_data.csv'), index=False)

# Example for exception handling during file operations
try:
    torch.save(X_train, os.path.join(output_file_path, 'X_train.pt'))
    torch.save(y_train, os.path.join(output_file_path, 'y_train.pt'))
    torch.save(X_valid, os.path.join(output_file_path, 'X_valid.pt'))
    torch.save(y_valid, os.path.join(output_file_path, 'y_valid.pt'))
    torch.save(X_test, os.path.join(output_file_path, 'X_test.pt'))
    torch.save(y_test, os.path.join(output_file_path, 'y_test.pt'))
except Exception as e:
    print(f"Failed to save files due to {e}")


In [ ]:
import torch
import os

output_file_path = r"C:\Users\young78703\Documents\GitHub\Machine-Learning-Projects\Data_save\Time_Series_Lag\stock_SP_list_lag10"
# output_file_path = "/home/predict_price/stock_price/data_save/daily/ver5.5/test/test"

# Create the directory if it does not exist
if not os.path.exists(output_file_path):
    os.makedirs(output_file_path)

# Save tensors
torch.save(X_train, os.path.join(output_file_path, 'X_train.pt'))
torch.save(y_train, os.path.join(output_file_path, 'y_train.pt'))
torch.save(X_valid, os.path.join(output_file_path, 'X_valid.pt'))
torch.save(y_valid, os.path.join(output_file_path, 'y_valid.pt'))
torch.save(X_test, os.path.join(output_file_path, 'X_test.pt'))
torch.save(y_test, os.path.join(output_file_path, 'y_test.pt'))


In [2]:
All_valid_symbol_list=[
             "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "BRK-B", "TSLA", "LLY", "V", "TSM", "UNH", "NVO", "JPM", "WMT", "XOM", "AVGO", "MA", "JNJ", "PG", "ORCL", 
             "HD", "ADBE", "ASML", "CVX", "MRK", "TM", "COST", "KO", "ABBV", "BAC", "PEP", "FMX", "CRM", "SHEL", "ACN", "NFLX", "BABA", "MCD", "LIN", "AZN", "CSCO", "AMD", 
             "NVS", "INTC", "TMO", "ABT", "SAP", "DIS", "TMUS", "CMCSA", "PFE", "NKE", "TTE", "INTU", "DHR", "WFC", "BHP", "VZ", "PDD", "HSBC", "HDB", "AMGN", "QCOM", "PM", 
             "IBM", "TXN", "NOW", "UNP", "COP", "MS", "AMAT", "GE", "CAT", "SPGI", "BX", "HON", "UPS", "BA", "SBUX", "RY", "BUD", "UL", "LOW", "SNY", "AXP", "RTX", "NEE", 
             "T", "UBER", "GS", "TD", "SYK", "LMT", "RIO", "BKNG", "DE", "ELV", "SONY", "ISRG", "BLK", "BMY", "PLD", "PBR", "TJX", "MUFG", "MDT", "BP", "SCHW", "MMC", 
             "MDLZ", "PBR-A", "ADP", "EQNR", "PGR", "GILD", "AMT", "LRCX", "CB", "ETN", "ADI", "VRTX", "CVS", "SHOP", "C", "REGN", "MU", "CI", "UBS", "SNPS", "ZTS", "ABNB",
             "BSX", "DEO", "IBN", "SO", "CME", "PANW", "FI", "CNI", "SLB", "MELI", "KLAC", "KKR", "EQIX", "CDNS", "NTES", "MO", "ITW", "INFY", "GSK", "EOG", "NOC", "ENB",
             "BTI", "CNQ", "DUK", "RELX", "WM", "SHW", "BDX", "GD", "AON", "CP", "ANET", "VALE", "MCO", "VMW", "HCA", "SMFG", "FDX", "RACE", "SAN", "HUM", "ICE", "CL", "TRI",
             "CSX", "PYPL", "STLA", "CHTR", "APD", "ITUB", "WDAY", "MCK", "TGT", "MAR", "CMG", "USB", "ORLY", "SCCO", "BMO", "MNST", "EPD", "CTAS", "ROP", "BN", "AMX",
             "MPC", "PH", "TDG", "PXD", "APH", "OXY", "LULU", "SNOW", "BNS", "AJG", "MMM", "ECL", "MSI", "BBVA", "E", "DELL", "NXPI", "TT", "PNC", "FCX", "EMR", "RSG", "PSX",
             "HMC", "APO", "CRWD", "BSBR", "TEAM", "ING", "PCAR", "AFL", "CPRT", "MRVL", "NSC", "AZO", "NGG", "WELL", "ADSK", "MET", "PSA", "SRE", "SPG", "CCI", "PCG", "AIG",
             "MCHP", "KDP", "CARR", "JD", "ABEV", "EL", "HLT", "ODFL", "STZ", "HES", "DHI", "TAK", "PLTR", "WMB", "MFG", "SU", "PAYX", "TFC", "CRH", "VLO", "KHC", "MSCI", "ET",
             "LNG", "KMB", "STM", "TEL", "DLR", "EW", "AEP", "ROST", "SGEN", "HSY", "COF", "DXCM", "F", "FTNT", "COR", "GWW", "ADM", "TRV", "EXC", "D", "CNC", "LI", "IDXX",
             "NUE", "WDS", "BIDU", "OKE", "DASH", "GIS", "GM", "O", "LVS", "KMI", "IQV", "NU", "EA", "TRP", "DOW", "CM", "BCE", "YUM", "LEN", "BK", "SYY", "AME",
             "MPLX", "DDOG", "AMP", "JCI", "LHX", "ALL", "WCN", "FAST", "SQ", "VRSK", "OTIS", "CTSH", "MFC", "ALC", "SPOT", "PRU", "BBD", "FERG", "CSGP", "BKR", "XEL", "A", "CTVA",
             "HAL", "BIIB", "LYG", "CVE", "IT", "IBKR", "TTD", "URI", "FIS", "PPG", "QSR", "PEG", "CMI", "ORAN", "PUK", "ED", "ROK", "ACGL", "LYB", "GFS", "ARES",
             "NDAQ", "IMO", "KR", "DD", "ON", "VICI", "NEM", "SLF", "MRNA", "GPN", "CDW", "ARGX", "CHT", "MLM", "EXR", "ELP", "IR", "DVN", "CPNG", "NTR", "VMC", "VEEV", "BF-A",
             "CQP", "HPQ", "GOLD", "MDB", "CCEP", "ZS", "PKX", "FANG", "PWR", "RCL", "TTWO", "ANSS", "BCS", "RYAAY", "WEC", "DG", "MPWR", "TU", "FICO", "SBAC", "AWK", "WST", "CAH",
             "SPLK", "EIX", "WBD", "WTW", "DLTR", "VOD", "EFX", "XYL", "WIT", "EC", "GLW", "AEM", "AVB", "GIB", "BNTX", "DB", "CBRE", "FTV", "NET", "RBLX", "KEYS", "HUBS", "ZBH",
             "FNV", "COIN", "WY", "TEF", "HIG", "MTD", "CHD", "DAL", "TCOM", "GRMN", "TLK", "APTV", "RCI", "ICLR", "TSCO", "NWG", "RMD", "TROW", "TW", "SE", "RJF", "FCNCA", "EBAY",
             "STT", "VRSN", "BR", "DTE", "MOH", "FE", "HWM", "ALNY", "PINS", "DFS", "EQR", "LIFW", "ETR", "GMAB", "BRO", "WPM", "HEI-A", "HEI", "IX", "WAB", "INVH", "LYV", "MTB",
             "ULTA", "NVR", "HPE", "AEE", "STE", "ES", "FTS", "BGNE", "SYM", "CTRA", "SNAP", "TS", "SIRI", "MT", "ZTO", "NOK", "CCL", "GPC", "DOV", "BEKE", "ROL", "PPL", "PHG",
             "OWL", "ZM", "CCJ", "IFF", "UMC", "YUMC", "DRI", "EBR", "PHM", "TRGP", "AIU", "FITB", "JBHT", "TDY", "ASX", "MKL", "STLD", "PTC", "IRM", "TECK", "ARE", "EXPE", "BAX",
             "WRB", "VTR", "WBA", "LH", "K", "HRL", "KOF", "HOLX", "RKT", "MKC", "FDS", "TYL", "UWMC", "PBA", "CNP", "EXPD", "DKNG", "EG", "CLX", "VIV", "TSN", "FLT", "PFG",
             "ATO", "AKAM", "COO", "CMS", "J", "JBL", "LPLA", "BAH", "AXON", "BEP", "FSLR", "EQT", "CHKP", "BMRN", "WLK", "BALL", "VRT", "WMG", "ERIC", "BLDR", "NTAP", "RPRX", "SUI",
             "CINF", "DECK", "STX", "KB", "HBAN", "WAT", "BSY", "RS", "MGA", "ALGN", "HUBB", "ENTG", "NTRS", "RIVN", "BG", "CF", "OMC", "AVY", "DGX", "WSO", "RF", "ILMN", "TXT", "DT",
             "SMCI", "AMH", "L", "IEX", "WDC", "SWKS", "FWONA", "EPAM", "ERIE", "SNA", "TME", "BBY", "MRO", "RTO", "LUV", "LII", "LDOS", "ALB", "SHG", "AER", "IOT", "MAA", "MGM",
             "SUZ", "XPEV", "JHX", "ESS", "PKG", "TER", "FOXA", "CE", "LW", "SWK", "ELS", "MANH", "SSNC", "APP", "SQM", "POOL", "GGG", "CAG", "AMCR", "CBOE", "MAS", "LOGI",
             "CSL", "CNHI", "NDSN", "NIO", "RPM", "DPZ", "AVTR", "GEN", "GDDY", "UAL", "VST", "ROKU", "XP", "TAP", "LNT", "CFG", "BIP", "IHG", "GRAB", "ENPH", "TPL", "NICE",
             "NMR", "AGR", "CPB", "MOS", "LBRDK", "PODD", "WPC", "BEN", "GLPI", "NWSA", "INCY", "FNF", "ACI", "ACM", "LKQ", "HST", "OVV", "SYF", "RYAN", "UDR", "H", "KIM",
             "IP", "AZPN", "LEGN", "GFI", "EVRG", "MORN", "ZI", "OC", "FLEX", "HTHT", "IPG", "OKTA", "EDU", "SJM", "AES", "CG", "KEY", "FMS", "PSTG", "AOS", "RBA", "ARCC", "JKHY",
             "SNN", "TWLO", "LECO", "GL", "GFL", "REG", "U", "ZBRA", "EXAS", "RDY", "WSM", "VTRS", "APA", "TRU", "RVTY", "SAIA", "BCH", "CNA", "NRG", "RNR", "NI", "UTHR", "CHK",
             "TRMB", "NBIX", "WES", "CDAY", "CASY", "USFD", "RGA", "PNR", "AEG", "OTEX", "PAG", "PATH", "KMX", "UHAL", "YPF", "CCK", "TFX", "XPO", "FIVE", "PAYC",
             "CX", "TFII", "REXR", "BAP", "EME", "LAMR", "CZR", "TEVA", "DKS", "WYNN", "DOX", "LCID", "CRL", "EMN", "COTY", "WRK", "TECH", "PFGC", "FFIV", "CHRW", "EQH", "FND",
             "DINO", "NTNX", "Z", "WPP", "CPT", "WMS", "WTRG", "HII", "UHS", "ALLE", "BLD", "AFG", "TOL", "QRVO", "SBS", "PEAK", "BJ", "BRKR", "QGEN", "CHDN", "HSIC", "DBX", "TTC",
             "CLH", "ITT", "CUBE", "SCI", "BSAC", "BIO", "CHE", "SNX", "NLY", "KEP", "FTI", "ESLT", "NVT", "GGB", "EWBC", "TTEK", "AGCO", "BXP", "DOCU", "DVA", "DUOL", "MEDP", "ETSY",
             "ONON", "SKM", "CLF", "CHWY", "PEN", "MTCH", "JNPR", "MKTX", "MTN", "RGEN", "PARA", "RHI", "TIMB", "AIZ", "MNSO", "BURL", "ALV", "DLB", "ATR", "PCTY", "PSO", "FBIN",
             "KNX", "FUTU", "VIPS", "PNW", "UNM", "ALLY", "KNSL", "MNDY", "LSCC", "RRC", "CW", "WWD", "ORI", "PCOR", "SSL", "SKX", "AAL", "BWA", "GWRE", "RL", "STVN", "CYBR",
             "STN", "FCN", "EGP", "LNW", "LEA", "GLOB", "OHI", "SRPT", "MUSA", "CNM", "COLD", "OLED", "JAZZ", "X", "SEIC", "AR", "GRFS", "PR", "RRX", "TOST", "FRT", "WCC", "BERY",
             "RGLD", "WSC", "GTLB", "YMM", "AFRM", "SWN", "HESM", "EDR", "WEX", "WF", "NOV", "LAD", "TXRH", "JLL", "VOYA", "WBS", "JEF", "PRI", "NYT", "ESTC", "HOOD", "DCI", "ARMK",
             "NFE", "HLI", "AU", "BZ", "TX", "CACI", "TREX", "CTLT", "CCCS", "RMBS", "GNTX", "NNN", "APPF", "MSTR", "IEP", "INFA", "BWXT", "KBR", "ROIV", "OGE", "UI", "RBC", "CSAN",
             "GNRC", "IONS", "FIX", "TPR", "DSGX", "PAC", "APG", "BBWI", "GPK", "ASR", "ACHC", "INGR", "UFPI", "FHN", "AXTA", "GXO", "FSV", "TPX", "CAR", "ARW",
             "WING", "MAT", "MTDR", "VFC", "EXEL", "KGC", "FMC", "CAE", "WFRD", "ONTO", "DAR", "MUR", "LBTYK", "SSD", "CIEN", "MIDD", "PSN", "NYCB", "CIB", "CIG", "HRB", "COKE",
             "BILL", "MSA", "CHRD", "ICL", "TKO", "GMED", "BRX", "FLR", "QLYS", "WTS", "SOFI", "SWAV", "WFG", "STAG", "EHC", "AMKR", "CIVI", "WH", "KRTX", "STWD", "KT",
             '^GSPC','^DJI','^RUT','^NYA','^STI','^IXIC','^FTSE','^FCHI','^GDAXI','^IBEX','^N100','^N225','^KS11',
             'ES=F','YM=F','NQ=F','ZB=F','ZN=F','ZF=F','ZT=F','GC=F','HG=F','SI=F','CL=F','NG=F','BZ=F','ZC=F','ZO=F','KE=F','ZM=F','ZL=F','ZS=F','GF=F','HE=F','HO=F','LE=F','CC=F','KC=F','CT=F','SB=F',
                          
]
# ETF_symbol_list = ['LSPU.L', 'ISEU.L', 'SC0J.DE', 'SPY4.L', 'HPRO.L', 'INRG.L', 'H50E.L', 'CMR.TO', 'RSP', 'FUTY', 'ACWE.MI', 'VNRA.L', 'TDTT', 'AGHG.L', 'CEU2.L', 'XMMO', 'IWMO.L', 'ECOG.L', 'SPGP', 'XDEQ.MI', 'SDHY.L', 'VECA.L', 'MSED.L', 'DAX.PA', 'IDOG', 'XMWO.SW', 'IGLS.L', 'GOAI.DE', 'OM3X.DE', 'DVY.MX', 'XISR.JK', 'XAR', 'VONE', 'JMUB', 'LDCU.L', 'GBSS.L', 'DBAW', 'IYW', 'SMLF', 'XLF', 'G500.L', 'EELV', 'VDEM.L', 'FLXU.L', 'CNKY.L', 'OMFL', 'ZPDF.DE', 'IFRA', 'VERX.DE', 'VDIV.DE', 'SXLU.L', 'IDIN.L', 'AOA', 'ITWN.MI', 'VGIT', 'BTP13.MI', 'WTV', 'XMCX.L', 'UIM4.DE', 'QCH.NE', 'ISP6.L', 'VHT', 'VTV.MX', 'TOF.AS', 'WTAI.MI', 'CSBGE7.SW', 'VDE.MX', 'IUSA.DE', 'TDIV.SW', 'OPER', 'HMEU.L', 'EWC.MX', 'ZTM.NE', 'XUCS.L', 'XD9U.SW', 'QTUM', 'TRET.AS', 'VPC', 'USAI', 'IWDE.AS', 'VRP', 'VEQT.TO', 'IGSU.L', 'SXRQ.DE', '1656.T', 'NANR', 'EMDD.L', 'VERE.DE', 'WTCH.AS', 'PSCD', 'ESGE.DE', 'LEAD.DE', 'VGVE.DE', 'IDP6.L', 'CSBGE7.MI', 'IBGS.L', 'ZGQ.TO', 'JPST.L', 'XMWD.L', 'FDVV', 'GLD', '1343.T', 'XMID.L', '4BRZ.DE', '1484.T', 'JREU.MI', 'SPY4.DE', 'VRIG', 'DJSC.AS', 'SMMU', 'SPYN.DE', 'TRD3.DE', 'DGRO', 'IPKW', 'RDVY', '2517.T', 'FCOM', 'XDWI.L', 'XWEH.DE', 'HMWO.L', 'XSHC.L', 'VVL.TO', 'DBEZ', 'VEVE.L', 'ACWI.L', 'NUDM', 'MOAT.AX', 'PZT', 'IVV', 'VECP.DE', 'BIGT.L', 'XSFN.L', 'LYY0.DE', 'MWRD.MI', 'HYHG', 'XWEH.MI', 'MMIT', 'VFEM.DE', 'OMXS.L', 'LSST', 'DGS', 'SPX5.L', 'PULS', 'IJPE.L', 'CSM', 'C73.MI', 'IUS3.DE', 'EXSA.MI', 'VTS.AX', 'SXLI.L', 'IGB.AX', 'JPUS', 'UIMM.DE', 'EL4E.DE', 'H4ZA.DE', 'XUKX.L', 'EL4C.DE', 'IJH', 'VFEA.DE', '1320.T', 'XIEE.DE', 'UPVL.L', 'SPYV', 'SPY.MX', '00661.TW', 'JRGE.DE', 'VWCG.MI', 'IVOV', 'CE71.L', 'IS0R.DE', 'SCHF', 'EQL', 'ISCF', 'FLMI', 'SPPW.DE', 'CDL', 'IEFM.L', 'XZWE.DE', 'DBEF', 'RISE.L', 'BCHN.L', 'DJAD.DE', 'FDRR', 'IS0P.DE', '1489.T', 'SXLP.MI', 'EUDG', 'IS3U.DE', 'URTH', 'IWLD.AX', 'XDWD.SW', 'XDWE.L', 'EWT', 'UESG.MI', 'IWRD.MI', 'FYX', 'LGLV', 'CUKX.L', 'VNRT.DE', 'PTH', 'SXRL.DE', 'USDV.SW', 'SP5C.PA', 'VDET.SW', 'FRUE.L', 'MTE.PA', 'SPXE', 'INDS', 'U10H.MI', 'XWD.TO', 'WLD.PA', '1698.T', 'AMEQ.DE', 'IEGA.AS', 'WDSC.L', 'PXC.TO', 'VEUR.MI', 'XSX6.DE', 'VUKE.L', '4BRZ.SW', 'STNX.MI', 'VIG', 'ANGL', 'VERX.AS', 'ERNE.MI', 'DIVB', '0053.TW', 'FENY', 'C40.PA', 'EDOG', 'TUR', 'CBU7.L', 'SPXD.L', 'ZLU.TO', 'VFEM.AS', 'HTWN.L', 'FFIU', 'FDL', 'DBJP', 'XLY.MX', '1397.T', 'ESRU.L', 'HEZU', 'HYGH', '1657.T', 'CSX5.L', 'WCOS.AS', 'IDTW.L', 'ZPRC.DE', 'RNSC', 'UB39.L', 'IS3Q.DE', 'SPXS.L', 'DJMC.L', 'VIOV', 'SUJA.L', 'HXQ.TO', 'FV', 'VCSH.MX', 'ENGY.L', 'FLLA', 'SEGA.L', 'FYLD', 'VGS.AX', 'C001.DE', 'C73.PA', 'IJPE.AS', 'GRNB', 'EXI2.DE', 'IEMU.L', 'DGRW.L', 'PSR', 'IXN', 'XLV', 'EUSRI.PA', 'ERNE.AS', 'WTI2.DE', 'VWCG.DE', 'XZMU.L', 'IBGX.AS', 'IASH.L', 'SEMB.L', 'WTEM.DE', 'XQQ.TO', 'DGT', 'CBON', 'PHSP.L', 'VAF.AX', 'SPXD.MI', 'TRS3.SW', 'S500.MI', 'VHY.AX', '2510.T', 'BBAX', 'ZPRS.DE', 'IESE.SW', 'IS3M.DE', 'XACTC25.CO', 'QCEU.PA', 'FNDX', 'PFXF', 'XUTC.DE', 'XUTD.L', 'PDC.TO', 'SMLK.DE', 'VGAD.AX', 'ICSH', 'FLRN', 'XMWO.MI', 'IUSV', 'VWRD.L', 'DJSC.MI', '1653.T', 'DIVO', 'STYC.L', 'REZ', 'FDIS', 'REGL', 'SDCI', 'XDEW.L', 'XSCS.L', 'NUSC', 'TFRN.L', 'FCUQ.TO', 'EL4B.DE', 'EL4F.DE', 'ECH', 'ERNS.L', 'D5BM.DE', 'WHEA.L', 'IBTS.MI', 'DRSK', 'ZPAL.SW', 'PTNQ', 'IUSK.DE', '5MVL.DE', 'VGVF.DE', 'GOAU', 'IEU.AX', 'CALF', 'XCV.TO', 'HEFA', 'IUS7.DE', 'XDWD.DE', 'SPY4.PA', 'ONEQ', 'EUE.L', 'IYH', 'MIVB.DE', '2513.T', 'UIND.L', 'ZSILEU.SW', 'PSCC', 'QGRO', 'VECA.MI', 'PFFD', 'PRF', 'ACDC.AX', 'IYE.MX', 'PHAU.MI', 'UST.MI', 'SMH.L', 'SPTS', 'IH2O.L', 'SMMD', 'XUHC.DE', 'RPV', 'IGSB', 'VHVG.L', 'IOO', 'IBCN.DE', 'IBTS.AS', 'EES', 'VEVE.SW', 'PJUL', 'SXRT.DE', 'IUSA.L', 'NUMG', 'EL4N.DE', 'MOGB.L', 'IUSQ.DE', 'XDN0.L', 'MGK', 'XDWD.L', 'ATMP', 'SXLV.MI', 'EMTL', 'XSX6.L', 'ZTM-U.NE', 'IAI', 'CD8.PA', 'FLIN', 'IMCV', 'G2X.DE', 'EXSC.DE', 'FALN', 'MTD.PA', 'XYP1.DE', 'SBIO.MI', 'XLF.MX', 'HUKX.L', 'IEFQ.L', 'IHYU.L', 'UIFS.L', 'EXH7.DE', 'SPF1.DE', 'GGRW.L', 'IUKP.L', 'IHCU.L', '1311.T', 'MTI.PA', 'S6X0.DE', 'VGEU.DE', 'HXH.TO', 'XDWC.MI', 'VV', 'XMU.TO', 'IB01.SW', 'VISM.AX', 'VECA.DE', 'IBCA.DE', 'JRGD.DE', 'HTAB', 'IJJ', 'LYYA.DE', 'DGRC.TO', 'VDE', 'EMVL.L', 'JMRE.L', 'GLUX.MI', 'XSMO', 'XDW0.DE', 'FSTA', 'DIVI', 'DXJ', 'XMAW.L', 'HXQ-U.TO', 'GLAU.L', 'VSDA', 'VUSA.SW', '1319.T', 'DIA', 'ZPDM.DE', 'WSRI.MI', 'VMBS', 'GDX', '1655.T', 'SXPIEX.SW', 'IUSA.MI', '00645.TW', 'IWDG.L', 'IS3K.DE', 'JPSA.L', 'ISDW.L', 'IWDE.MI', 'ACWD.L', 'VYMI', 'SLYG', 'EWT.MX', 'UBUT.DE', '0057.TW', 'IQIN', 'QQC-F.TO', 'USTB', 'ESGS', 'VO', 'UCAP.L', 'VERG.L', 'XWLD.L', 'LEAD', 'LQQ.PA', '1550.T', 'PSCT', 'VWRL.L', 'ESGD', 'ENGW.L', 'IWQU.L', 'TP05.L', 'IEI', 'EL4Y.DE', 'FNDA', 'CACC.PA', 'BCHS.L', 'VDEA.MI', 'XTL', 'S250.L', 'ZPDT.DE', 'XUCD.L', 'GBS.L', 'US10.L', '2515.T', 'EWM', 'IMCG', 'XDWU.L', 'CU71.L', 'NBTK.DE', 'SXLC.L', 'VFH', 'IJPE.MI', 'TDTF', 'XLB', 'VDEA.L', 'NDQ.AX', 'CDC', 'CZA', 'NACP', 'MWRD.L', 'XLU', '1545.T', 'PBUS', 'KIE', 'CIBR', 'XMV.TO', 'TRET.DE', 'MGT.PA', 'MWSH.DE', 'XNKY.DE', 'ZGRO.TO', 'IQD.TO', 'IS0Q.DE', 'KORP', 'DX2D.DE', 'TRE3.L', 'LVHI', 'TPYP', 'VCIT', 'USML.L', 'JPIB', 'UC13.L', 'HXE.TO', 'VEUR.L', 'IGSG.AS', 'XMAW.DE', 'EEMS', 'USHY', 'ZTS.NE', 'RWJ', 'SWRD.AS', 'CSBGU3.MI', 'SPSB', 'XD5E.DE', 'EUN.MI', 'XUEN.DE', 'PSI', 'XDN0.DE', 'IHYU.MI', 'SPY', 'XUTC.L', 'JGRE.L', 'XHB', 'WESG.L', 'SLX', 'CEMU.AS', 'ETHI.TO', 'WTDM.DE', 'USRIH.MI', 'MINT.MI', 'IWRD.L', 'MXUK.L', 'IAK', '2520.T', 'EXS1.MI', 'IEFA.MX', 'VDY.TO', 'DXJS', 'XUT3.L', 'VTIP.MX', 'XEIN.MI', 'U13G.L', 'MFE.PA', 'FLAM.NE', 'CSSPX.MI', 'PSC', 'UQLTD.MI', 'EUNK.DE', 'ITB', 'XESC.SW', 'IQLT', 'BCD', 'DALI', 'DAXX.MI', 'GBS.MI', 'XLY', '1680.T', 'IMIE.MI', 'JHMM', 'IUSA.AS', 'JPME', 'IWFQ.L', 'GCVB.L', 'MUST', 'XIU.TO', 'IWQU.MI', 'MFDX', 'EXS1.DE', 'BJUL', 'SPXN', 'PHAU.L', '1329.T', 'XLI', '1651.T', 'FEMS', 'IWMO.MI', 'MBB', 'TDT.AS', 'FNDB', 'CHIE', 'DJSC.L', 'SPTM', 'O87.SI', 'SP5EUY.MI', 'GHYC.SW', 'EUEA.AS', 'STEA.L', 'XUCS.DE', 'QQQ.MX', '2514.T', 'VGSH', 'EWX', 'JRUD.DE', 'IEFV.L', 'CHSPI.SW', 'VUAA.L', 'C003.DE', 'MTC.PA', 'PSRU.L', 'EM710.MI', 'VNRG.L', 'UIMP.AS', 'DBX5.DE', '10AI.DE', '1557.T', 'CBU0.L', 'LBRE.DE', 'LGOV', 'XY4P.DE', 'IWRD.AS', 'MGT.MI', 'CSSMI.SW', 'TDIV.AS', 'VFMO', 'JRGD.L', 'VUAA.DE', 'FCOR', 'SPLG', 'IBTM.L', 'HMUS.L', 'PFFA', 'STIP', 'FTCS', 'VEE.TO', 'ERND.L', 'PTF', 'CSBGE3.MI', 'RAVI', 'VLC.AX', 'PICK', 'IJS', 'EUN4.DE', 'BNXG.DE', 'UC99.L', 'IUS', 'ONEV', 'XUCD.DE', 'CS51.L', 'VUKG.L', 'MXWS.L', 'XDWC.DE', 'ITWN.L', 'PDBC', 'IYE', 'GCVC.SW', 'XDUS.L', 'GSSC', 'IYW.MX', '1658.T', 'IDJG.AS', 'DBXW.DE', 'SLQD', 'US13.L', 'XZW0.DE', 'ZEB.TO', 'FLO5.L', 'TDIV.MI', 'FNDF', 'GGRG.L', 'SPFU.DE', 'IBCG.DE', 'USSC.L', 'HYDB', 'SUSC.L', 'DGRW', 'GLUX.DE', 'VMO.TO', 'US37.L', 'SXLU.AS', 'USDV.L', 'U500.AS', 'SMSWLD.MI', 'SXLE.L', 'JVAL', 'XDWL.SW', 'XDEW.SW', 'XLU.MX', 'VECP.L', 'IEMB.MI', 'IQQW.DE', 'SC0H.DE', 'FYT', 'XZMU.DE', 'FGD', 'QUAL.AX', 'RIGS', 'CNDX.L', 'IAF.AX', 'FCIQ.TO', 'XCEM', 'IB01.L', 'XUU.TO', 'FNX', 'SXLF.AS', 'DBEU', 'CEU1.L', 'VEVE.AS', 'WOMN', '00692.TW', 'PEY', 'WFIN.AS', 'QQQ', 'HSPD.L', 'INAA.AS', 'TDIV.L', 'QQXT', 'ICSU.L', 'HAZ.TO', 'LYPS.DE', 'EMDL.L', 'VFQY', 'SC0D.DE', 'ISMD', 'VDET.L', 'SPYI.DE', 'DXU.TO', 'UNCA.MI', 'ESGU', 'BOCT', 'QCN.TO', 'SXLU.MI', 'XESC.L', 'NASL.L', 'XLFS.L', 'HEAW.L', 'ZUQ.TO', 'ZPRE.DE', 'FHLC', 'EWC', 'BOUT', 'ELFA.DE', 'ISFR.L', 'USDV.MI', '1478.T', 'STN.PA', 'DJMC.AS', 'VEMT.MI', 'IDWR.L', 'FINW.L', 'MFLX', '10AL.DE', 'IBCL.DE', 'ENFR', 'NEAR', 'YLD', 'XESC.MI', 'XLV.MX', 'JPST', 'CEU.MI', 'SXLY.L', 'PXE', 'AFRN.PA', 'IJR', 'VUSA.MI', 'CBU3.L', 'TRE3.MI', 'PUST.PA', 'L8I7.DE', 'HMOP', 'VXC.TO', 'EMLD.MI', 'ITOT', 'THD', 'CHIQ', 'TOK', 'QUU.TO', 'IGEB', 'QLD', 'SXRW.DE', '1321.T', 'EMIM.L', 'EUNA.AS', 'ZTS-U.NE', 'VB', 'RSP.MX', 'EMCR.L', 'FPEI', '5HED.L', '1325.T', 'STHE.L', 'XYP1.MI', 'GLCB.L', 'XLG', 'PXF', 'GOAI.PA', 'IHOO.AX', '2522.T', 'HXS-U.TO', 'JQUA', 'FXL', 'S600.L', 'EQL.TO', 'VDEV.L', 'FLLV', 'GSJY', 'FXG', 'ICF', 'LSPX.L', 'OEF', 'CE2D.L', 'WNRG.L', 'VOO.MX', 'VFEA.L', 'CQQQ', 'XLKS.L', 'JPEM', 'E50EUA.SW', 'PSRM.L', 'IEAG.MI', 'SPHB', 'WUTI.L', 'SCHO', 'IBGS.MI', 'STHY.MI', 'DJSXE.SW', 'SPSM', 'XEH.TO', 'CEU.PA', 'VNLA', '2528.T', 'VGRO.TO', 'FCAL', 'HSRT', 'EUSRI.MI', 'CW8.PA', '0051.TW', 'ACWI.MI', 'SKOR', 'UIMM.AS', 'CEUG.DE', 'EMLP', 'XSTC.L', 'IUSL.DE', 'IEF.MX', 'CG1G.DE', 'EVX', 'HBRL.L', 'VEUD.L', 'HXX.TO', 'S500H.MI', 'WOOD.L', 'XDEM.L', '5HEE.MI', 'KGRN', 'VOOG', 'EL4Z.DE', 'ECH.MX', 'CCOR', 'ZQQ.TO', 'RUSG.L', 'LIT', 'VMID.DE', 'XSPX.L', 'EMCB', '1345.T', 'XDWT.DE', 'XDAX.L', 'EWW', 'VWRL.MI', 'HHH.PA', 'SXLV.L', 'JREU.L', 'IYF', 'EL4A.DE', 'FPE', 'WNRG.AS', 'PSP5.PA', 'XRMU.L', 'XRSS.L', 'XMTW.L', 'EUSC', 'HEDJ', 'IS0Y.DE', 'IVV.AX', 'HIDD.L', 'XMUS.L', 'XMVM', 'ACU7.DE', 'IEXF.MI', 'CSPX.AS', 'GLUX.PA', 'ULST', 'RESD', 'MDBG.L', 'VGT.MX', '00660.TW', 'SC0C.DE', 'IBTS.L', 'ZNQ.TO', 'WESG.PA', 'WLD.MI', 'EGOV.MI', 'XUTC.MI', '1547.T', 'VE.TO', 'ZPDU.DE', 'UIMP.DE', 'FNDE', 'JREG.L', 'FLTR', 'VWCE.DE', 'ESGG', 'HFXI', 'IBZL.L', 'VAS.AX', 'IAGG', 'IQQS.DE', 'BTP10.MI', 'U13H.MI', 'IVW', '006203.TW', 'EUSR.L', 'EM57.MI', 'UC96.L', 'IWDE.SW', 'MFUS', 'GDX.MI', '6TVM.DE', 'ISAE.AS', 'VGT', 'SHYG', 'IQQM.DE', 'US10.PA', 'VI.TO', 'VFMF', 'CNXT', 'KX1G.DE', 'LWCR.MI', 'NFTY', 'IAU', 'VLU', 'DGRA.L', 'WAT.MI', 'SPVM', 'SBIO.L', 'S500.PA', 'FIVG', 'ESGE.PA', 'SPY5.MI', '00713.TW', 'LQDH.L', 'IESE.AS', 'XLKS.MI', 'BFIT', 'EXSA.DE', 'LTAM.L', 'JREE.DE', 'IGSG.L', '1546.T', 'WTEC.MI', 'ETFSP500.WA', 'TLTD', 'ZPDI.DE', 'PPA', '006208.TW', 'SXR8.DE', 'IAU.MX', 'GGRP.L', 'IUSF.L', 'IBTM.MI', 'ZST-L.TO', 'ZPRV.DE', 'SCHD', 'XDWI.DE', 'PID', 'VHVE.MI', 'VFEM.SW', 'XDEQ.DE', 'QQEW', '100H.PA', 'VPL', 'IYG', 'ZSILHC.SW', 'FTLS', 'HTWD.L', 'ISAG.MI', 'XLKQ.L', 'FIW', 'EXSI.DE', 'MGV', 'RLY', 'BIV', 'WRDU.L', 'BCHN.MI', 'WFIN.L', 'C6E.PA', 'XSPX.SW', 'NRAM.MI', 'ZGI.TO', 'OUSM', 'VIOO', 'BRES.MI', 'SPXP.L', 'MLPX', 'CEFS', 'XMEU.L', 'VTIP', 'EXH2.DE', '0052.TW', 'ZCON.TO', 'SECT', 'WNDU.L', 'XDWS.L', 'WLDH.PA', 'XUS.TO', 'PSRF.L', 'MXWO.L', 'TIPX', 'AUTP.MI', 'XDEQ.L', 'SDVY', 'SUES.L', 'PHGP.L', 'VUKE.MI', 'FXO', 'HSPX.L', 'SPY4.MI', 'XDWS.MI', 'FLQM', 'MINT.L', 'XZMU.MI', 'DDWM', 'CSBGU7.MI', 'EZM', 'ZST.TO', 'XD9E.DE', 'MFEM', 'IDX', 'CSBGU0.SW', 'US10.MI', 'YPS', '2511.T', 'IS3F.DE', 'IEAG.AS', 'IESE.MI', 'GMVM.DE', 'LMBS', '00735.TW', 'RWK', '2521.T', 'XDJE.DE', 'CACX.L', 'TMFC', 'GAGU.L', 'VOO', 'JCPB', 'CBU7.AS', 'HSCZ', 'DWAT', 'EWL', 'EUNE.SW', 'RUD.TO', 'IBGX.MI', 'XZW0.L', 'FTXN', 'XDWI.MI', 'IITU.L', 'CONV.MI', 'VAP.AX', 'SP5H.PA', 'WSRI.PA', 'RZV', 'GGRA.L', 'GAGG.MI', 'FCTR', 'SQLV', 'ALAG.L', 'QCEU.MI', 'JREU.DE', 'EPI', 'JREG.MI', 'JPSE', 'KBWP', 'SPXH.MI', 'MOTG', 'JRUD.L', 'PRN', 'SHYL', 'UST.PA', 'EMLI.MI', 'IH2O.MI', 'DLN', 'HXS.TO', 'IQDY', 'IUSM.DE', 'QEFA', 'XLK.MX', 'VONG', 'IWDA.AS', 'FVAL', 'EXXX.DE', 'ICDU.L', 'ZPDS.DE', 'SXRV.DE', 'KBA', 'XMUS.MI', 'ILCG', 'UBTP.L', 'UIMR.DE', 'EM15.MI', 'WTAI.L', 'SMEA.L', 'FILL', 'IEI.MX', 'IUSG', 'C5E.PA', 'PSET', 'MOAT.MI', 'SPEM', 'TRTY', 'XZW0.MI', 'XD9U.MI', 'ISAG.L', 'IVE', 'VFEM.MI', 'WOSC.L', 'IVV.MX', 'KBND', 'QBB.TO', 'SPYY.DE', 'VWRL.AS', '1369.T', 'UKGBPB.MI', 'VMIG.L', 'IEFA', '1488.T', '5HEE.DE', 'LWCR.DE', 'VHVE.L', 'STHY.L', 'WLDD.L', 'NURE', 'VAW', 'MEUD.PA', 'CSBGE3.SW', 'XAMB.DE', 'PHPD.MI', 'AE50.DE', 'XMED.L', 'EIMI.L', 'VCE.TO', 'EXSI.MI', 'SPTI', 'JMBS', 'HYZD', 'SDY', 'MGC', 'VEF.TO', 'IWB', 'XMEU.MI', 'EUN1.DE', 'WTEC.L', 'EXW3.DE', 'MAGA', 'VGG.TO', 'ESTX.AX', 'ZPDD.DE', 'XLE', 'UC64.L', 'WHEA.AS', '1659.T', 'IGRO', 'ONEY', 'EQAL', 'BIZD', 'FLOT', 'EUNR.DE', 'EMCR.MI', 'MEUS.L', 'DX2G.DE', 'GLT5.L', 'GTO', 'XDJP.L', 'JMST', 'LQDI', 'GXG', 'EPU', 'IBGL.AS', 'VFEA.MI', 'IBGX.L', 'UTES', 'ZSILUS.SW', 'SXR7.DE', 'VCR', 'MUUSAS.SW', '00733.TW', 'UKX.PA', 'UQLTE.SW', 'FLTW', 'SP5.PA', 'LYQ7.DE', 'HAWX', 'SYLD', 'HTUS', 'SDG', 'VEUR.SW', 'VNRA.DE', 'OMFS', 'DSI', 'IS3B.DE', '1330.T', 'SLYV', 'XFN.TO', 'EXSD.DE', 'EUNH.DE', 'IMAE.AS', 'LYY7.DE', 'XMUD.L', 'SWRD.L', 'SPYG', 'MWRD.PA', 'TDGB.L', 'SSPY', 'XD5E.MI', 'EWJV', 'FBND', 'PSRW.L', 'WRDU.AS', 'VAMO', 'UBU7.DE', '2527.T', 'CSNDX.MI', 'T3GB.L', 'X1G.MI', 'IDTM.L', 'PBTP', 'QMOM', 'SXLI.AS', 'BFOR', 'PKW', 'CHIS', 'CNRG', 'IQQN.DE', 'MFED.PA', 'SOXX', 'XDEM.MI', 'FLOT.MX', 'IHF', 'GILS.L', 'RFV', 'GAGH.MI', 'VNRT.L', 'AGZ', 'JU13.L', 'CSPX.L', 'MXUS.L', 'JRGE.MI', 'GDX.L', 'CSX5.AS', 'S500H.PA', 'ERNU.L', 'WRDUSA.SW', 'IWF', 'DEM.L', 'FLJH', '7USH.DE', 'EM1015.MI', 'PEXL', 'VFV.TO', 'DXG.TO', 'SXLI.MI', 'XDWH.L', 'XSPU.L', 'BSJP', 'XESX.L', 'VBR', 'XZWE.MI', 'GCVE.MI', 'CAC.PA', 'SCHX', '1346.T', 'JERE.L', 'XDWH.MI', 'EUNL.DE', 'FGQD.L', 'LYP6.DE', 'SPY5.DE', 'EWD', 'WAT.PA', 'SXLB.L', 'IETC', 'SPYD.DE', 'GENE.L', 'XLRE', 'MEUD.MI', 'MXUK.DE', 'MEU.PA', '1326.T', 'CBE3.L', 'AMEW.DE', 'SPMB', 'XDWL.L', 'VFVA', 'SCHP', 'PCEF', 'CSSX5E.MI', 'RWL', '00757.TW', 'JREE.L', 'XDAX.MI', 'RDIV', 'AFRN.MI', 'CSEMU.MI', 'UIM1.DE', 'IHI', 'XD9U.DE', '1654.T', 'SPMO', 'E500.DE', 'SXLK.AS', 'XDUK.L', 'BAB', 'FLOS.L', 'H4ZF.DE', 'LQDH', 'URTH.MX', 'ZSP-U.TO', 'XLK', 'FCUD.TO', 'SDOG', 'GAA', 'MOAT', 'SJNK', 'DNL', 'VEUA.L', 'SXLF.MI', 'X35E.MI', 'XD5E.L', 'FUSI.L', 'QHAL.AX', 'S100.L', 'GOAI.MI', 'XDWD.MI', 'XSVM', 'SWDA.L', 'VOOV', 'IDHQ', 'GGRA.MI', 'LYBRE.SW', 'DVY', 'SLG-OMXH25.HE', 'USRI.MI', 'FUMB', 'IMEU.L', 'XDW0.L', 'UBUT.AS', '00762.TW', 'XDDX.DE', 'SMEA.MI', 'HXT.TO', 'FTSL', 'IWDA.L', '1673.T', 'D500.DE', 'ZSIL.SW', 'MDBU.L', 'ICOW', 'DTD', 'ZUS-U.TO', 'SYBW.DE', 'IS3R.DE', 'HMWD.L', 'IESU.L', 'XRSM.DE', 'IWY', 'DXC.TO', 'GDMA', 'FDHY', 'SPY5.L', '10AH.DE', 'SMICHA.SW', 'IQQT.DE', 'VFEM.L', 'QARP', 'EUE.MI', 'EMI.MI', 'IHDG', 'KCE', 'VUSA.L', 'COM', 'IWL', 'BBJP', 'IVOO', 'IDUS.L', 'CWS', 'UBTS.L', 'VGWL.DE', 'CSUK.L', 'CWB', 'QWLD', 'EURO.MI', 'UDVD.L', 'SXLF.L', 'XDWL.DE', '10AM.DE', 'HEEM', 'PHPD.L', 'IDNA.L', 'IISU.L', 'SP5C.L', 'VUKE.DE', 'PHO', 'EUN.L', 'IFRE.AS', 'VUG', 'EZA.MX', 'IWDP.L', 'DSTL', 'UB0A.L', 'IRCP.L', 'LYMS.DE', 'AUT.PA', 'XMTD.L', 'EXH6.DE', 'VUSD.L', 'EMXC', 'SPX.MI', 'HDEF', 'CU2U.L', 'SUSA', 'STHE.MI', 'SDHY.MI', 'XDWH.DE', 'IBGL.L', 'F500.DE', 'CEMB', 'QTIP.NE', 'VERE.L', 'DH2O.L', 'SPHY', 'CSP1.L', 'MOAT.L', 'WNRG.MI', 'FUQA.L', 'ZSP.TO', 'JREG.DE', 'CSEMU.SW', 'CNDX.AS', 'EUNS.DE', 'XDWC.L', 'JURE.L', 'US13.MI', 'EBBB.MI', 'E500.MI', 'VUAA.MI', 'LESW.DE', 'SPXS.MI', 'VECP.MI', 'HEWB.TO', 'ACWE.PA', 'JRDG.L', 'LYM8.DE', 'VUSA.AS', 'XLPE.MI', 'ERNE.L', 'ECN.PA', 'IMIE.PA', 'MDYV', 'SEGA.MI', 'UBXX.L', 'SXLK.MI', 'T3RE.DE', 'LYPG.DE', '5HED.DE', 'VCSH', 'GXLK.L', 'DBXD.DE', 'SRIEC.PA', 'CEU2.MI', 'SEMC.L', 'FTXL', 'HNDQ.AX', 'IWDE.L', 'CEW', 'CSB', 'XDWS.DE', 'VCN.TO', 'IYM', 'SXLB.MI', 'L100.L', 'HYS.MI', '50E.PA', 'IGM', 'VEMT.L', 'AME6.DE', 'IQQQ.DE', 'ZLB.TO', 'TPU.TO', 'IDBT.L', 'VDGR.AX', 'XMAW.MI', '1678.T', 'UB06.L', 'EXV6.DE', 'VWCG.L', 'TR3G.L', 'NASD.L', 'WXOZ.AX', 'VECP.AS', 'IWFM.L', 'XUTE.DE', 'VERX.L', 'XDEW.MI', 'CSA', 'IDJG.L', 'FCQH.TO', 'EUREUA.SW', 'IMSU.L', 'XDWT.L', 'AIRR', 'VUAG.L', 'CU31.L', 'NOBL', 'SHYU.L', 'XDEW.DE', 'HSPD.MI', 'S5USAS.SW', 'VTV', 'XSPX.MI', 'WESG.MI', 'PRIW.L', '006204.TW', 'FXZ', 'ESG', 'GAGG.PA', 'EQDS.L', 'IEO', 'VWRP.L', 'H50E.MI', 'XSD', 'VUN.TO', 'QDIV', 'BBVAE.MC', 'XIN.TO', 'CNYA', 'UTRN', 'CHDVD.SW', 'CSSX5E.SW', 'GDIG.L', 'IPAC', 'ESGE.MI', 'SPY4.SW', 'BJAN', 'VUSA.DE', 'NULG', 'GSY', 'XMHQ', 'SXLP.L', 'UINC.L', 'MVE.AX', 'ASRR.DE', 'PAVE', 'VOE', 'FLN', 'VGEM.DE', 'CEU2.PA', 'XDW0.MI', 'SPX4.L', 'MMIN', 'IUSU.DE', 'INFR.L', 'CNX1.L', 'EQWL', 'INAA.L', 'XUHC.L', 'POCT', 'XD9U.L', 'XDEM.DE', 'MEUD.L', 'FTEC', 'IWR', 'UITB', 'SXLY.AS', 'RAAX', 'EBBB.PA', 'SUUS.L', 'XUEN.L', 'XSEN.L', 'FLUS.TO', 'JGEP.L', 'XAW.TO', 'FSMB', 'XDWT.MI', 'XLE.MX', 'VWRA.L', 'ZBAL.TO', 'VDHG.AX', 'IGBH', '1660.T', 'SXLK.L', 'LESU.DE', 'FMI.MI', 'EMHY', 'SSAC.L', 'SWDA.MI', 'E15G.DE', 'SUSB', 'VEUR.AS', 'INTL.L', 'JREE.MI', 'IJPE.SW', 'XLUS.L', 'DBC', 'VWCE.MI', 'SCHG', 'FRNE.DE', 'SPHQ', 'IESG.L', 'XD5S.L', 'SBIO.AS', 'QQQE', 'FLBL', 'FTSD', 'SSAC.AS', 'SYBM.DE', 'IDEV', 'EXXT.DE', 'UBU9.DE', 'XD9C.SW', 'EXW1.DE', '1578.T', 'VWRL.SW', 'UC44.L', 'ISAC.L', 'IBGS.AS', 'HEWJ', '1476.T', 'VFEG.L', 'UBU9.AS', 'IBGL.MI', 'ICVT', 'SSAC.SW', 'XUS-U.TO', 'IEF', 'USFR.L', 'IQQG.DE', 'FDM', 'XESX.MI', 'CHCORP.SW', 'SCHR', 'IUSU.L', 'SXLY.MI', 'WCOS.L', 'CMBS', 'UC46.L', 'IOO.AX', 'BLES', 'ISWD.L', 'DWAS', 'IBCH.DE', '00657.TW', 'ZPDH.DE', 'TI5A.AS', '0050.TW', 'ETHO', 'JREE.SW', 'DRMU.TO', 'EXH5.DE', 'C4D.PA', 'PRFZ', 'EYLD', 'SWRD.MI', 'VGSH.MX', 'IYK', 'PSCI', 'SPAG.L', 'EZA', 'RTH', 'COWZ', 'SMH', 'VERE.MI', 'IEMB.L', 'VPU', 'EUN2.DE']

DOW_list=[
             "AAPL", "MSFT", "GOOGL", "AMZN", "NVDA", "META", "BRK-B", "TSLA", "LLY", "V", "TSM", "UNH", "NVO", "JPM", "WMT", "XOM", "AVGO", "MA", "JNJ", "PG", "ORCL", 
             "HD", "ADBE", "ASML", "CVX", "MRK", "TM", "COST", "KO", "ABBV", "BAC", "PEP", "FMX", "CRM", "SHEL", "ACN", "NFLX", "BABA", "MCD", "LIN", "AZN", "CSCO", "AMD", 
             "NVS", "INTC", "TMO", "ABT", "SAP", "DIS", "TMUS", "CMCSA", "PFE", "NKE", "TTE", "INTU", "DHR", "WFC", "BHP", "VZ", "PDD", "HSBC", "HDB", "AMGN", "QCOM", "PM", 
             "IBM", "TXN", "NOW", "UNP", "COP", "MS", "AMAT", "GE", "CAT", "SPGI", "BX", "HON", "UPS", "BA", "SBUX", "RY", "BUD", "UL", "LOW", "SNY", "AXP", "RTX", "NEE", 
             "T", "UBER", "GS", "TD", "SYK", "LMT", "RIO", "BKNG", "DE", "ELV", "SONY", "ISRG", "BLK", "BMY", "PLD", "PBR", "TJX", "MUFG", "MDT", "BP", "SCHW", "MMC", 
             "MDLZ", "PBR-A", "ADP", "EQNR", "PGR", "GILD", "AMT", "LRCX", "CB", "ETN", "ADI", "VRTX", "CVS", "SHOP", "C", "REGN", "MU", "CI", "UBS", "SNPS", "ZTS", "ABNB",
             "BSX", "DEO", "IBN", "SO", "CME", "PANW", "FI", "CNI", "SLB", "MELI", "KLAC", "KKR", "EQIX", "CDNS", "NTES", "MO", "ITW", "INFY", "GSK", "EOG", "NOC", "ENB",
             "BTI", "CNQ", "DUK", "RELX", "WM", "SHW", "BDX", "GD", "AON", "CP", "ANET", "VALE", "MCO", "VMW", "HCA", "SMFG", "FDX", "RACE", "SAN", "HUM", "ICE", "CL", "TRI",
             "CSX", "PYPL", "STLA", "CHTR", "APD", "ITUB", "WDAY", "MCK", "TGT", "MAR", "CMG", "USB", "ORLY", "SCCO", "BMO", "MNST", "EPD", "CTAS", "ROP", "BN", "AMX",
             "MPC", "PH", "TDG", "PXD", "APH", "OXY", "LULU", "SNOW", "BNS", "AJG", "MMM", "ECL", "MSI", "BBVA", "E", "DELL", "NXPI", "TT", "PNC", "FCX", "EMR", "RSG", "PSX",
             "HMC", "APO", "CRWD", "BSBR", "TEAM", "ING", "PCAR", "AFL", "CPRT", "MRVL", "NSC", "AZO", "NGG", "WELL", "ADSK", "MET", "PSA", "SRE", "SPG", "CCI", "PCG", "AIG",
             "MCHP", "KDP", "CARR", "JD", "ABEV", "EL", "HLT", "ODFL", "STZ", "HES", "DHI", "TAK", "PLTR", "WMB", "MFG", "SU", "PAYX", "TFC", "CRH", "VLO", "KHC", "MSCI", "ET",
             "LNG", "KMB", "STM", "TEL", "DLR", "EW", "AEP", "ROST", "SGEN", "HSY", "COF", "DXCM", "F", "FTNT", "COR", "GWW", "ADM", "TRV", "EXC", "D", "CNC", "LI", "IDXX",
             "NUE", "WDS", "BIDU", "OKE", "DASH", "GIS", "GM", "O", "LVS", "KVUE", "KMI", "IQV", "NU", "EA", "TRP", "DOW", "CM", "BCE", "YUM", "LEN", "BK", "SYY", "AME",
             "MPLX", "DDOG", "AMP", "JCI", "LHX", "ALL", "WCN", "FAST", "SQ", "VRSK", "OTIS", "CTSH", "MFC", "ALC", "SPOT", "PRU", "BBD", "FERG", "CSGP", "BKR", "XEL", "A", "CTVA",
             "HAL", "BIIB", "LYG", "MBLY", "CVE", "IT", "GEHC", "IBKR", "TTD", "URI", "FIS", "PPG", "QSR", "PEG", "CMI", "ORAN", "PUK", "ED", "ROK", "ACGL", "LYB", "GFS", "ARES",
             "NDAQ", "IMO", "KR", "DD", "ON", "VICI", "NEM", "SLF", "MRNA", "GPN", "CDW", "ARGX", "CHT", "MLM", "EXR", "ELP", "IR", "DVN", "CPNG", "NTR", "VMC", "VEEV", "BF-A",
             "CQP", "HPQ", "GOLD", "MDB", "CCEP", "ZS", "PKX", "FANG", "PWR", "RCL", "TTWO", "ANSS", "BCS", "RYAAY", "WEC", "DG", "MPWR", "TU", "FICO", "SBAC", "AWK", "WST", "CAH",
             "SPLK", "EIX", "WBD", "WTW", "DLTR", "VOD", "EFX", "XYL", "WIT", "EC", "GLW", "AEM", "AVB", "GIB", "BNTX", "DB", "CBRE", "FTV", "NET", "RBLX", "KEYS", "HUBS", "ZBH",
             "FNV", "COIN", "WY", "TEF", "HIG", "MTD", "CHD", "DAL", "TCOM", "GRMN", "TLK", "APTV", "RCI", "ICLR", "TSCO", "NWG", "RMD", "TROW", "TW", "SE", "RJF", "FCNCA", "EBAY",
             "STT", "VRSN", "BR", "DTE", "MOH", "FE", "HWM", "ALNY", "PINS", "DFS", "EQR", "LIFW", "ETR", "GMAB", "BRO", "WPM", "HEI-A", "HEI", "IX", "WAB", "INVH", "LYV", "MTB",
             "ULTA", "NVR", "HPE", "AEE", "STE", "ES", "FTS", "BGNE", "SYM", "CTRA", "SNAP", "TS", "SIRI", "MT", "ZTO", "NOK", "CCL", "GPC", "DOV", "BEKE", "ROL", "PPL", "PHG",
             "OWL", "ZM", "CCJ", "IFF", "UMC", "YUMC", "DRI", "EBR", "PHM", "TRGP", "AIU", "FITB", "JBHT", "TDY", "ASX", "MKL", "STLD", "PTC", "IRM", "TECK", "ARE", "EXPE", "BAX",
             "WRB", "VTR", "WBA", "LH", "K", "HRL", "KOF", "HOLX", "RKT", "MKC", "FDS", "TYL", "UWMC", "PBA", "CNP", "VLTO", "EXPD", "DKNG", "EG", "CLX", "VIV", "TSN", "FLT", "PFG",
             "ATO", "AKAM", "COO", "CMS", "J", "JBL", "LPLA", "BAH", "AXON", "BEP", "FSLR", "EQT", "CHKP", "BMRN", "WLK", "BALL", "VRT", "WMG", "ERIC", "BLDR", "NTAP", "RPRX", "SUI",
             "CINF", "DECK", "STX", "KB", "HBAN", "WAT", "BSY", "RS", "MGA", "ALGN", "HUBB", "ENTG", "NTRS", "RIVN", "BG", "CF", "OMC", "AVY", "DGX", "WSO", "RF", "ILMN", "TXT", "DT",
             "SMCI", "AMH", "L", "IEX", "WDC", "SWKS", "FWONA", "EPAM", "ERIE", "SNA", "TME", "BBY", "MRO", "RTO", "LUV", "LII", "LDOS", "ALB", "SHG", "AER", "IOT", "MAA", "MGM",
             "SUZ", "XPEV", "JHX", "ESS", "PKG", "TER", "FOXA", "CE", "LW", "SWK", "ELS", "MANH", "SSNC", "APP", "SQM", "POOL", "GGG", "CAG", "AMCR", "VFS", "CBOE", "MAS", "LOGI",
             "CSL", "CNHI", "NDSN", "NIO", "RPM", "BAM", "DPZ", "AVTR", "GEN", "GDDY", "UAL", "VST", "ROKU", "XP", "TAP", "LNT", "CFG", "BIP", "IHG", "GRAB", "ENPH", "TPL", "NICE",
             "CRBG", "NMR", "AGR", "CPB", "MOS", "LBRDK", "PODD", "WPC", "BEN", "GLPI", "NWSA", "INCY", "FNF", "ACI", "ACM", "LKQ", "HST", "OVV", "SYF", "RYAN", "UDR", "H", "KIM",
             "IP", "AZPN", "LEGN", "GFI", "EVRG", "MORN", "ZI", "OC", "FLEX", "HTHT", "IPG", "OKTA", "EDU", "SJM", "AES", "CG", "KEY", "FMS", "PSTG", "AOS", "RBA", "ARCC", "JKHY",
             "SNN", "TWLO", "LECO", "GL", "GFL", "REG", "U", "ZBRA", "EXAS", "RDY", "WSM", "VTRS", "APA", "TRU", "RVTY", "SAIA", "BCH", "CNA", "NRG", "RNR", "NI", "UTHR", "CHK",
             "TRMB", "NBIX", "WES", "CDAY", "CASY", "TPG", "USFD", "RGA", "PNR", "AEG", "OTEX", "UHAL-B", "PAG", "PATH", "KMX", "UHAL", "YPF", "CCK", "TFX", "XPO", "FIVE", "PAYC",
             "CX", "TFII", "REXR", "BAP", "EME", "LAMR", "CZR", "TEVA", "DKS", "WYNN", "DOX", "LCID", "CRL", "EMN", "COTY", "WRK", "TECH", "PFGC", "FFIV", "CHRW", "EQH", "FND",
             "DINO", "NTNX", "Z", "WPP", "CPT", "WMS", "WTRG", "HII", "UHS", "ALLE", "BLD", "AFG", "TOL", "QRVO", "SBS", "PEAK", "BJ", "BRKR", "QGEN", "CHDN", "HSIC", "DBX", "TTC",
             "CLH", "ITT", "CUBE", "SCI", "BSAC", "BIO", "CHE", "SNX", "NLY", "KEP", "FTI", "ESLT", "NVT", "GGB", "EWBC", "TTEK", "AGCO", "BXP", "DOCU", "DVA", "DUOL", "MEDP", "ETSY",
             "ONON", "SKM", "CLF", "CHWY", "PEN", "MTCH", "JNPR", "MKTX", "MTN", "RGEN", "PARA", "RHI", "TIMB", "AIZ", "MNSO", "BURL", "ALV", "DLB", "ATR", "PCTY", "PSO", "FBIN",
             "KNX", "FUTU", "VIPS", "PNW", "UNM", "ALLY", "KNSL", "MNDY", "LSCC", "RRC", "CW", "WWD", "ORI", "PCOR", "SSL", "SKX", "AAL", "BWA", "BIRK", "GWRE", "RL", "STVN", "CYBR",
             "STN", "FCN", "EGP", "LNW", "LEA", "GLOB", "OHI", "SRPT", "MUSA", "CNM", "COLD", "OLED", "JAZZ", "X", "SEIC", "AR", "GRFS", "PR", "RRX", "TOST", "FRT", "WCC", "BERY",
             "RGLD", "WSC", "GTLB", "YMM", "AFRM", "SWN", "HESM", "EDR", "WEX", "WF", "NOV", "LAD", "TXRH", "JLL", "VOYA", "WBS", "JEF", "PRI", "NYT", "ESTC", "HOOD", "DCI", "ARMK",
             "NFE", "HLI", "AU", "BZ", "TX", "CACI", "TREX", "CTLT", "CCCS", "RMBS", "GNTX", "NNN", "APPF", "MSTR", "IEP", "INFA", "BWXT", "KBR", "ROIV", "OGE", "UI", "RBC", "CSAN",
             "GNRC", "IONS", "CART", "FIX", "TPR", "DSGX", "PAC", "APG", "BBWI", "GPK", "ASR", "ACHC", "INGR", "UFPI", "FHN", "AXTA", "GXO", "FSV", "KVYO", "TPX", "CAR", "ARW",
             "WING", "MAT", "MTDR", "VFC", "EXEL", "KGC", "FMC", "CAE", "WFRD", "ONTO", "DAR", "MUR", "LBTYK", "SSD", "CIEN", "MIDD", "PSN", "NYCB", "CIB", "CIG", "HRB", "COKE",
             "BILL", "MSA", "CHRD", "ICL", "TKO", "GMED", "BRX", "FLR", "QLYS", "WTS", "SOFI", "SWAV", "WFG", "STAG", "EHC", "AMKR", "CIVI", "WH", "KRTX", "STWD", "KT"
]
#Futures
Futures_symbol_list=['ES=F','YM=F','NQ=F','ZB=F','ZN=F','ZF=F','ZT=F','GC=F','HG=F','SI=F','CL=F','NG=F','BZ=F','ZC=F','ZO=F','KE=F','ZR=F','ZM=F','ZL=F','ZS=F','GF=F','HE=F','HO=F','LE=F','CC=F','KC=F','CT=F','SB=F']
#Index
Index_symbol_list=['^N225','^KS11','^BSESN','^MXX','^IXIC','^GSPTSE','^DJI','^FCHI','^GDAXI','^FTSE','^IBEX','^N100','^GSPC','^RUT','^NYA','^STI']
#ETF
ETF_symbol_list=[
    "SPY", "IVV", "VOO", "VTI", "QQQ", "VEA", "VTV", "IEFA", "BND", "VUG", "AGG", "IWF", "VWO", "IEMG", "IJH", "VIG", "IJR", "VXUS", "GLD", "VO", "VGT", "BNDX", "IWM", "XLK", "IWD",
 "VYM", "SCHD", "EFA", "ITOT", "VB", "TLT", "BIL", "XLE", "RSP", "VCIT", "XLV", "VCSH", "MUB", "VEU", "IVW", "BSV", "SCHX", "QUAL", "SCHF", "IXUS", "VNQ", "JEPI", "IWB", "LQD", "VT",
 "XLF", "DIA", "VV", "IEF", "VTEB", "SHY", "USMV", "IWR", "MBB", "IAU", "IVE", "VBR", "DGRO", "JPST", "GOVT", "SCHB", "VGSH", "SHV", "IUSB", "DFAC", "IGSB", "SPLG", "TIP", "SCHG", "SDY", 
 "SPYG", "USFR", "VGIT", "SGOV", "VGK", "MDY", "DVY", "ACWI", "EFV", "VHT", "SPYV", "VMBS", "EEM", "XLP", "XLY", "SPDW", "COWZ", "TQQQ", "BIV", "VXF", "VOE", "QQQM", "MGK", "VONG", "XLI"
 ]

Special_symbol_list=[
             '^GSPC','^DJI','^RUT','^NYA','^STI','^IXIC','^FTSE','^FCHI','^GDAXI','^IBEX','^N100','^N225','^KS11','^BSESN','^MXX','^GSPTSE',
             'ES=F','YM=F','NQ=F','ZB=F','ZN=F','ZF=F','ZT=F','GC=F','HG=F','SI=F','CL=F','NG=F','BZ=F','ZC=F','ZO=F','KE=F','ZM=F','ZL=F','ZS=F','GF=F','HE=F','HO=F','LE=F','CC=F','KC=F','CT=F','SB=F',
             "SPY", "IVV", "VOO", "VTI", "QQQ", "VEA", "VTV", "IEFA", "BND", "VUG", "AGG", "IWF", "VWO", "IEMG", "IJH", "VIG", "IJR", "VXUS", "GLD", "VO", "VGT", "BNDX", "IWM", "XLK", "IWD",             
             ]

Test_symbol_list=[
            #  '^GSPC','^DJI','^RUT','^NYA','^STI','^IXIC',
                # "SPYG", "USFR", "VGIT",
    "1186.HK",
             ]

# stock_code_list = None
# stock_code_list = ["015760", "034020", "088980", "055550", "032640", "323410", "000270", "105560", "047040", "003490", "034220", "005940", "138930", "030200", "086790",]
# symbol_list = ["005930.KS", "015760.KS", "034020.KS", "088980.KS", "055550.KS", "032640.KS", "323410.KS", "000270.KS", "105560.KS", "047040.KS", "003490.KS", "034220.KS", "005940.KS", "138930.KS", "030200.KS", "086790.KS",]
KOSPI_list = [
"005935.KS", "005930.KS", "373220.KS", "000660.KS", "207940.KS", "005387.KS", "005389.KS", "005380.KS", "005385.KS", "068270.KS", "000270.KS", "035420.KS", "051915.KS", 
"005490.KS", "051910.KS", "006405.KS", "006400.KS", "035720.KS", "003670.KS", "028260.KS", "02826K.KS", "105560.KS", "012330.KS", "055550.KS", "086520.KQ", "066575.KS", 
"066570.KS", "011200.KS", "323410.KS", "018260.KS", "03473K.KS", "096770.KS", "096775.KS", "086790.KS", "015760.KS", "003555.KS", "003550.KS", "032830.KS", "138040.KS", 
"009155.KS", "009150.KS", "000810.KS", "017670.KS", "329180.KS", "000815.KS", "352820.KS", "033780.KS", "010130.KS", "034020.KS", "316140.KS", "259960.KS", "047050.KS", 
"024110.KS", "034730.KS", "003495.KS", "003490.KS", "030200.KS", "090430.KS", "090435.KS", "009540.KS", "010950.KS", "010955.KS", "326030.KS", "377300.KS", "012450.KS", 
"042660.KS", "402340.KS", "086280.KS", "009830.KS", "454910.KS", "009835.KS", "066970.KQ", "247540.KQ", "010140.KS", "028300.KQ", "361610.KS", "001570.KS", "011170.KS", 
"042700.KS", "161390.KS", "18064K.KS", "047810.KS", "011070.KS", "180640.KS", "302440.KS", "051905.KS", "307950.KS", "051900.KS", "028050.KS", "000105.KS", "034220.KS", 
"005830.KS", "251270.KS", "000100.KS", "267250.KS", "241560.KS", "271560.KS", "005070.KS", "068760.KQ", "128940.KS", "036570.KS", "00680K.KS", "097950.KS", "088980.KS", 
"004020.KS", "032640.KS", "097955.KS", "006800.KS", "006805.KS", "005945.KS", "021240.KS", "000720.KS", "000725.KS", "078930.KS", "078935.KS", "018880.KS", "272210.KS", 
"029780.KS", "267260.KS", "035900.KQ", "016360.KS", "035250.KS", "010620.KS", "058470.KQ", "011785.KS", "011780.KS", "383220.KS", "071050.KS", "071055.KS", "064350.KS", 
"011790.KS", "005940.KS", "112610.KS", "003410.KS", "079550.KS", "008930.KS", "000120.KS", "001045.KS", "052690.KS", "00104K.KS", "001040.KS", "282330.KS", "007070.KS", 
"001450.KS", "081660.KS", "039490.KS", "014680.KS", "008775.KS", "138930.KS", "008770.KS", "006260.KS", "293490.KQ", "004370.KS", "010060.KS", "005290.KQ", "328130.KQ", 
"002790.KS", "002795.KS", "000990.KS", "041510.KQ", "000880.KS", "00088K.KS", "175330.KS", "000885.KS", "010120.KS", "036460.KS", "009420.KS", "023530.KS", "039030.KQ", 
"088350.KS", "028670.KS", "111770.KS", "012750.KS", "030000.KS", "139480.KS", "00499K.KS", "004990.KS", "007660.KS", "204320.KS", "020150.KS", "047040.KS", "298040.KS", 
"026960.KS", "095340.KQ", "000250.KQ", "065350.KQ", "298050.KS", "002380.KS", "240810.KQ", "086900.KQ", "036930.KQ", "003230.KS", "011210.KS", "37550K.KS", "004170.KS", 
"073240.KS", "375500.KS", "042670.KS", "005850.KS", "003620.KS", "051600.KS", "348370.KQ", "006110.KS", "336260.KS", "000080.KS", "000240.KS", "33626K.KS", "000087.KS", 
"185750.KS", "017800.KS", "248070.KS", "298020.KS", "137310.KS", "35320K.KS", "002710.KS", "353200.KS", "007310.KS", "005420.KS", "253450.KQ", "000150.KS", "139130.KS", 
"035760.KQ", "000155.KS", "018670.KS", "005305.KS", "108320.KS", "005300.KS", "006280.KS", "069620.KS", "004000.KS", "067310.KQ", "196170.KQ", "336370.KS", "064760.KQ", 
"006360.KS", "192820.KS", "078600.KQ", "161890.KS", "38380K.KS", "121600.KQ", "365340.KQ", "383800.KS", "120115.KS", "120110.KS", "001440.KS", "004800.KS", "009520.KQ", 
"000210.KS", "140860.KQ", "237690.KQ", "440110.KQ", "100090.KS", "214450.KQ", "280360.KS", "285130.KS", "000215.KS", "006040.KS", "145720.KS", "101000.KQ", "298380.KQ", 
"003535.KS", "034230.KQ", "33637K.KS", "025900.KQ", "096530.KQ", "067160.KQ", "069960.KS", "28513K.KS", "395400.KS", "001740.KS", "067630.KQ", "166090.KQ", "089590.KS", 
"003690.KS", "213420.KQ", "093370.KS", "085660.KQ", "103140.KS", "000157.KS", "224060.KQ", "278280.KQ", "003280.KS", "195870.KS", "056190.KQ", "014820.KS", "014825.KS", 
"294870.KS", "089860.KS", "001120.KS", "084370.KQ", "112040.KQ", "009900.KS", "271940.KS", "048410.KQ", "003545.KS", "003547.KS", "012510.KS", "267270.KS", "192650.KS", 
"003540.KS", "039130.KS", "222800.KQ", "009970.KS", "036540.KQ", "001067.KS", "064550.KQ", "003030.KS", "000670.KS", "001800.KS", "006650.KS", "020560.KS", "001060.KS", 
"192080.KS", "005880.KS", "086450.KQ", "003530.KS", "074600.KQ", "214320.KS", "101490.KQ", "300720.KS", "348950.KS", "178920.KS", "009240.KS", "003090.KS", "001065.KS", 
"099190.KQ", "002840.KS", "058610.KQ", "287410.KQ", "114090.KS", "010780.KS", "004490.KS", "060280.KQ", "105630.KS", "001430.KS", "122870.KQ", "003850.KS", "069260.KS", 
"016380.KS", "365550.KS", "002355.KS", "456040.KS", "023590.KS", "137400.KQ", "181710.KS", "002350.KS", "006125.KS", "025540.KS", "000400.KS", "006120.KS", "330590.KS", 
"032350.KS", "007340.KS", "950160.KQ", "215200.KQ", "008730.KS", "005257.KS", "005250.KS", "352480.KQ", "015750.KQ", "145020.KQ", "036830.KQ", "210980.KS", "001680.KS", 
"082740.KS", "001685.KS", "003380.KQ", "009450.KS", "299900.KQ", "087010.KQ", "025320.KQ", "082640.KS", "079160.KS", "399720.KQ", "268280.KS", "272450.KS", "033240.KS", 
"179900.KQ", "161000.KS", "286940.KS", "019175.KS", "019170.KS", "344820.KS", "290650.KQ", "054780.KQ", "025980.KQ", "281820.KS", "229640.KS", "085620.KS", "091120.KQ", 
"049770.KS", "090460.KS", "053800.KQ", "080220.KQ", "094360.KQ", "031430.KS", "091810.KS", "000640.KS", "141080.KQ", "084850.KQ", "450140.KS", "018290.KQ", "039200.KQ", 
"007700.KS", "030520.KQ", "293940.KS", "170900.KS", "041190.KQ", "046890.KQ", "009415.KS", "237880.KQ", "950210.KS", "064960.KS", "079370.KQ", "017960.KS", "030190.KS", 
"460860.KS", "182360.KQ", "082270.KQ", "030610.KS", "031980.KQ", "095660.KQ", "45014K.KS", "005440.KS", "183300.KQ", "001530.KS", "403550.KS", "003475.KS", "032500.KQ", 
"189300.KQ", "078340.KQ", "381970.KS", "215000.KQ", "034120.KS", "000070.KS", "068240.KQ", "000075.KS", "003000.KS", "032190.KQ", "200130.KQ", "003470.KS", "093320.KQ", 
"003240.KS", "002240.KS", "005610.KS", "299030.KQ", "348340.KQ", "007390.KQ", "097520.KS", "241590.KS", "192400.KS", "095500.KQ", "001720.KS", "900140.KS", "001725.KS", 
"069080.KQ", "263750.KQ", "057050.KS", "144510.KQ", "007815.KS", "005180.KS", "131290.KQ", "394280.KQ", "200880.KS", "284740.KS", "060250.KQ", "00781K.KS", "007810.KS", 
"108860.KQ", "043150.KQ", "000370.KS", "052020.KQ", "003920.KS", "033640.KQ", "018250.KS", "049070.KQ", "357120.KS", "001470.KS", "003925.KS", "249420.KS", "377740.KS", 
"054450.KQ", "145995.KS", "068400.KS", "145990.KS", "034310.KS", "056080.KQ", "001390.KS", "020000.KS", "094170.KQ", "084690.KS", "118990.KQ", "091700.KQ", "319400.KQ", 
"010690.KS", "218410.KQ", "036810.KQ", "103590.KS", "014830.KS", "039440.KQ", "206640.KQ", "101160.KQ", "203690.KQ", "241710.KQ", "058650.KS", "084695.KS", "006730.KQ", 
"011930.KS", "089030.KQ", "095700.KQ", "050890.KQ", "047920.KQ", "151860.KQ", "000490.KS", "314930.KQ", "215600.KQ", "183190.KS", "263720.KQ", "003520.KS", "448730.KS", 
"232140.KQ", "004430.KS", "053610.KQ", "001820.KS", "041830.KQ", "082920.KQ", "033270.KS", "453340.KS", "108675.KS", "007575.KS", "123890.KS", "017810.KS", "194480.KQ", 
"110990.KQ", "108670.KS", "102710.KQ", "042600.KQ", "340570.KQ", "033100.KQ", "071970.KS", "304100.KQ", "001940.KS", "025860.KS", "104830.KQ", "298690.KS", "093050.KS", 
"005690.KS", "023160.KQ", "084680.KS", "064290.KQ", "297890.KQ", "094480.KQ", "088260.KS", "005090.KS", "108490.KQ", "095610.KQ", "253590.KQ", "225570.KQ", "002310.KS", 
"089010.KQ", "131970.KQ", "027410.KS", "451800.KS", "044490.KQ", "178320.KQ", "306200.KS", "092040.KQ", "011810.KS", "243070.KQ", "005810.KS", "100790.KQ", "083310.KQ", 
"330860.KQ", "000430.KS", "013030.KQ", "034950.KQ", "014620.KQ", "317330.KQ", "012630.KS", "092220.KS", "025770.KQ", "017940.KS", "033500.KQ", "039840.KQ", "027360.KQ", 
"054950.KQ", "126560.KS", "090360.KQ", "060150.KQ", "003300.KS", "136480.KQ", "089980.KQ", "003570.KS", "029960.KQ", "002320.KS", "119610.KQ", "035600.KQ", "102940.KQ", 
"007690.KS", "038500.KQ", "140410.KQ", "089970.KQ", "058820.KQ", "077360.KQ", "060980.KS", "322510.KQ", "001200.KS", "003220.KS", "322000.KS", "265520.KQ", "097230.KS", 
"004690.KS", "267980.KQ", "058430.KS", "200670.KQ", "001880.KS", "061970.KQ", "008490.KS", "036030.KQ", "139050.KQ", "060720.KQ", "071320.KS", "064260.KQ", "000680.KS", 
"298000.KS", "012030.KS", "002030.KS", "047310.KQ", "008355.KS", "013890.KS", "950140.KQ", "008350.KS", "307750.KQ", "001790.KS", "950130.KQ", "079810.KQ", 
"102460.KS", "084650.KQ", "005950.KS", "024720.KS", "002960.KS", "051500.KQ", "006390.KS", "072710.KS", "042000.KQ", "000020.KS", "206400.KQ", "001780.KS", "078160.KQ", 
"095720.KS", "382840.KQ", "084110.KQ", "243840.KQ", "086390.KQ", "009290.KS", "389260.KQ", "356860.KQ", "013120.KQ", "001630.KS", "138490.KS", "001465.KS", "080160.KQ", 
"029530.KS", "001460.KS", "059090.KQ", "048530.KQ", "272290.KQ", "004560.KS", "007460.KS", "108230.KQ", "126700.KQ", "036620.KQ", "297090.KQ", "026890.KS", "077970.KS", 
"005010.KS", "031330.KQ", "000545.KS", "003800.KQ", "037270.KS", "053210.KS", "036420.KS", "016800.KS", "001500.KS", "035890.KQ", "452260.KS", "100120.KQ", "018470.KS", 
"005500.KS", "037560.KS", "126720.KS", "045100.KQ", "272550.KS", "002900.KS", "117580.KS", "075580.KS", "053030.KQ", "010100.KS", "418420.KQ", "001525.KS", "051360.KQ", 
"122900.KS", "200710.KQ", "005390.KS", "007570.KS", "099320.KQ", "206650.KQ", "002150.KS", "214370.KQ", "001510.KS", "041960.KQ", "033160.KQ", "322310.KQ", "001230.KS", 
"115390.KS", "000480.KS", "011500.KS", "003960.KS", "063080.KQ", "089600.KQ", "049950.KQ", "051370.KQ", "093640.KQ", "092070.KQ", "040300.KQ", "114190.KQ", "006220.KS", 
"211270.KQ", "271980.KS", "213500.KS", "104700.KS", "048870.KQ", "462520.KS", "054210.KQ", "036530.KS", "060370.KQ", "030530.KQ", "315640.KQ", "123860.KQ", "083650.KQ", 
"171090.KQ", "143160.KQ", "293480.KS", "101390.KQ", "370090.KQ", "130660.KS", "096760.KS", "042420.KQ", "005160.KQ", "032620.KQ", "036200.KQ", "035150.KS", "194370.KS", 
"041020.KQ", "034830.KS", "074610.KS", "104460.KQ", "045390.KQ", "001515.KS", "388720.KQ", "078350.KQ", "017890.KQ", "037710.KS", "079940.KQ", "278650.KQ", "017390.KS", 
"038290.KQ", "034810.KQ", "023410.KQ", "206560.KQ", "083450.KQ", "321820.KQ", "029460.KS", "004360.KS", "016580.KS", "018310.KQ", "004365.KS", "348150.KQ", "030210.KS", 
"052400.KQ", "002025.KS", "071840.KS", "900290.KQ", "090710.KQ", "002020.KS", "092730.KQ", "267850.KS", "095570.KS", "060590.KQ", "084010.KS", "047560.KQ", "214430.KQ", 
"131370.KQ", "004380.KS", "036630.KQ", "051980.KQ", "000050.KS", "086980.KQ", "068290.KS", "044450.KS", "004255.KS", "006840.KS", "011760.KS", "122640.KQ", "020120.KS", 
"125210.KQ", "001250.KS", "088390.KQ", "011280.KS", "078070.KQ", "073490.KQ", "00806K.KS", "009160.KS", "004250.KS", "008060.KS", "226320.KS", "003075.KS", "003070.KS", 
"027970.KS", "002810.KS", "230240.KQ", "005725.KS", "102260.KS", "004710.KS", "149950.KQ", "081000.KS", "053300.KQ", "321550.KQ", "011000.KS", "094860.KQ", "294570.KQ", 
"078150.KQ", "004980.KS", "005720.KS", "143240.KQ", "004985.KS", "019680.KS", "102120.KQ", "063570.KQ", "179290.KQ", "073570.KQ", "065510.KQ", "001130.KS", "086890.KQ", 
"063170.KQ", "083790.KQ", "067000.KQ", "060310.KQ", "377190.KS", "214680.KQ", "011330.KS", "019685.KS", "091580.KQ", "053690.KS", "098460.KQ", "036800.KQ", "204620.KQ", 
"025000.KS", "046440.KQ", "007160.KS", "251630.KQ", "083930.KQ", "339770.KS", "067280.KQ", "078020.KQ", "035810.KQ", "092230.KS", "043610.KQ", "036890.KQ", "080420.KQ", 
"460850.KS", "032820.KQ", "291230.KQ", "078520.KS", "009580.KS", "043370.KQ", "035080.KQ", "234080.KS", "010580.KS", "382800.KQ", "016590.KS", "000540.KS", "003060.KS", 
"092200.KS", "084730.KQ", "084990.KQ", "001527.KS", "950170.KQ", "089890.KQ", "036560.KQ", "192440.KQ", "317400.KS", "000145.KS", "003580.KS", "223310.KQ", "013990.KQ", 
"036010.KQ", "005680.KS", "002390.KS", "000140.KS", "005860.KQ", "001520.KS", "115960.KQ", "072770.KQ", "019210.KQ", "139990.KS", "023760.KQ", "214390.KS", "027710.KQ", 
"003200.KS", "048550.KQ", "024850.KQ", "105840.KS", "092870.KQ", "092790.KS", "107590.KS", "122990.KQ", "092190.KQ", "293780.KQ", "300080.KQ", "015890.KS", "002100.KS", 
"119860.KQ", "016450.KS", "002995.KS", "225530.KQ", "100130.KQ", "002990.KS", "090350.KS", "377030.KQ", "001360.KS", "011040.KQ", "002620.KS", "015590.KS", "004700.KS", 
"024060.KQ", "030960.KQ", "028100.KS", "090355.KS", "045970.KQ", "104480.KQ", "009680.KS", "014580.KS", "334890.KS", "094820.KQ", "038540.KQ", "083420.KS", "196450.KQ", 
"080580.KQ", "060570.KQ", "042370.KQ", "015860.KS", "362320.KQ", "290550.KQ", "314130.KQ", "220260.KQ", "001020.KS", "025950.KQ", "039570.KS", "111380.KS", "100250.KS", 
"008700.KS", "007860.KS", "024900.KS", "214180.KQ", "068790.KQ", "008260.KS", "238090.KQ", "023910.KQ", "010820.KS", "365270.KQ", "092130.KQ", "035510.KS", "950190.KQ", 
"230360.KQ", "044340.KQ", "011700.KS", "376190.KQ", "085670.KQ", "123040.KQ", "033530.KS", "006910.KQ", "014280.KS", "355390.KQ", "282690.KS", "046210.KQ", "013310.KQ", 
"121440.KQ", "161580.KQ", "006620.KQ", "006060.KS", "002785.KS", "003160.KS", "196490.KQ", "002787.KS", "067080.KQ", "419530.KQ", "004090.KS", "003720.KS"
]
# def get_symbol_list(select_list):
#     if select_list == 'All_Train':
#         symbol_list = All_train_symbol_list
#     elif select_list == 'All_Valid':
#         symbol_list = All_valid_symbol_list
#     elif select_list == 'DOW':
#         symbol_list = DOW_symbol_list
#     elif select_list == 'KOSPI':
#         symbol_list = KOSPI_symbol_list
#     elif select_list == 'Special':
#         symbol_list = Special_symbol_list
#     elif select_list == 'ETF':
#         symbol_list = ETF_symbol_list
#     elif select_list == 'Futures':
#         symbol_list = Futures_symbol_list
#     elif select_list == 'Index':
#         symbol_list = Index_symbol_list
#     elif select_list == 'Test':
#         symbol_list = Test_symbol_list
#     return symbol_list

# def get_parameters(select_interval):
#     if  select_interval == '1wk':
#         n_predict = 1
#         n_last_sequence = 10
#         forward= 0
#         interval='1wk'
#         target_col = 'Close'
#     elif select_interval == '1mo':
#         n_predict = 1
#         n_last_sequence = 10
#         forward= 0
#         interval='1mo'
#         target_col = 'Close'
#     elif select_interval == '1d':
#         n_predict = 1
#         n_last_sequence = 60
#         forward= 0
#         interval='1d'
#         target_col = 'Return'
#     return n_predict, n_last_sequence, forward, interval, target_col

# start_date = '2011-01-01'
# end_date = '2021-12-31'
# start_date_valid = '2011-01-01'
# interval = '1d'
# # select_list='All_Train'
# # select_list='All_Valid'

# # symbol_list_train = get_symbol_list(select_list='Futures')
# # symbol_list_valid = get_symbol_list(select_list='Futures')



# symbol_list_train = World_list+ETF_list
# symbol_list_valid = SP_list
# target_col = 'Return'
# n_trials = 2
# n_top_models = 1
# n_predict = 1
# n_last_sequence = 100
# forward= -1

# save_directory = "/home/predict_price/stock_price/model_save/no_split/ver5.5/test_1"

# all_results_params_df = model_save(start_date=start_date, end_date=end_date, start_date_valid=start_date_valid, symbol_list_train=symbol_list_train, symbol_list_valid=symbol_list_valid, interval=interval, 
#                                    target_col=target_col, n_trials=n_trials, model_save=True, save_directory=save_directory, plot_loss=True, n_last_sequence=n_last_sequence, n_predict=n_predict, forward=forward)


# output_file_path = "/home/predict_price/stock_price/model_save/no_split/ver5.5/test_1/test_1"
# # Save results_df
# all_results_params_df.to_csv(f'{output_file_path}_all_results_params_df.csv', index=True)

In [3]:
US_list = ["MSFT", "GOOG", "GOOGL", "AMZN", "NVDA", "BRK-B", "AVGO", "V", "UNH", "MA", "XOM", "LVMUY", "BAC", "ORCL", "NSRGY", "ASML", "ABBV", "ADBE", "CRM", "TM", "CVX", "LRLCY", "PEP", "NVS", "MCD", "NFLX", "CSCO", "ABT", "LIN", "SAP", "TMUS", "CMCSA", "INTU", "WFC", "DHR", "DIS", "QCOM", "PFE", "TXN", "NKE", "NOW", "BHP", "TTE", "MS", "HSBC", "CAT", "GS", "PM", "HDB", "GE", "SPGI", "AMAT", "SIEGY", "UPS", "AXP", "HON", "BA", "SNY", "COP", "DTEGY", "BKNG", "BUD", "RTX", "SYK", "BLK", "PLD", "NEE", "LMT", "VRTX", "RIO", "TJX", "SBGSY", "DE", "SBUX", "PANW", "SHOP", "PBR", "BMY", "PBR-A", "PGR", "MDLZ", "AIQUY", "MMC", "ADP", "ETN", "CB", "ADI", "BP", "UBS", "ABNB", "BSX", "CI", "MELI", "IBN", "INFY", "EQNR", "GSK", "ANET", "CFRUY", "DEO", "SNPS", "RELX", "SAFRY", "SHW", "HCA", "WDAY", "IBDRY", "BYDDY", "SO", "KKR", "DUK", "CME", "PYPL", "SLB", "MCO", "CRWD", "MKKGY", "MAR", "BDX", "CNQ", "CSX", "GD", "GLNCY", "BTI", "UNCRY", "USB", "STLA", "NTDOY", "MCK", "EOG", "CMG", "TGT", "VWAGY", "LNSTY", "RACE", "SCCO", "AON", "BMO", "LULU", "ORLY", "PH", "PNC", "MNST", "VALE", "TDG", "HEINY", "EPD", "MPC", "APD", "AMX", "BNS", "ECL", "PSX", "TT", "NXPI", "APO", "FCX", "ISNPY", "CHTR", "EMR", "ADSK", "SPG", "HMC", "DHI", "AJG", "PSA", "IFNNY", "MET", "PXD", "E", "PCAR", "CARR", "TFC", "WELL", "OXY", "HUM", "CRH", "AFL", "NGG", "PPRUY", "COF", "GM", "DXCM", "MCHP", "TRV", "AIG", "CMPGY", "HLT", "AZO", "CCI", "FTNT", "CPRT", "ING", "STZ", "ET", "ROST", "TAK", "BAESY", "KHC", "EL", "F", "KDP", "PAYX", "MSCI", "VLO", "DANOY", "IDXX", "SXYAY", "BASFY", "PCG", "GWW", "DDOG", "DLR", "DASH", "ENGIY", "BK", "KMB", "TEL", "NUE", "O", "OKE", "SU", "STM", "ALL", "SQ", "ADYEY", "TRP", "SPOT", "NEM", "CNC", "IQV", "HSY", "URI", "AMP", "FERG", "LNG", "DSDVY", "ALC", "MRNA", "SYY", "D", "EXPGY", "IBKR", "AME", "PRU", "MPLX", "YUM", "EA", "DOW", "JCI", "PLTR", "ADM", "OTIS", "ARES", "ROK", "ZS", "QSR", "GPN", "PPG", "HENOY", "KR", "XEL", "TTD", "RCL", "CMI", "CTVA", "IR", "CCEP", "DD", "VICI", "FICO", "NGLOY", "MLM", "LZAGY", "BKR", "HAL", "CDW", "LYB", "MPWR", "EFX", "ACGL", "ASAZY", "VMC", "IMO", "COIN", "HXGBY", "SGAPY", "PWR", "HUBS", "DG", "MDB", "RYAAY", "HPQ", "LI", "PUK", "CVE", "TTWO", "GOLD", "NET", "FANUY", "FANG", "XYL", "TSCDY", "CPNG", "BF-B", "DVN", "CBRE", "CLLNY", "MTD", "CAH", "SPLK", "SDVKY", "ZBH", "AVB", "HIG", "NTR", "KMTUY", "RMD", "RCI", "GIB", "TLK", "PINS", "VONOY", "AEM", "CQP", "DFS", "RKT", "RBLX", "BR", "DNNGY", "EC", "DAL", "PKX", "SMCI", "STT", "RJF", "BNTX", "FITB", "MTB", "ALNY", "EQR", "TW", "HWM", "ULTA", "WAB", "PHM", "CCL", "APTV", "ARGX", "STE", "OWL", "MT", "ILMN", "ARE", "ICLR", "EBAY", "IMBBY", "MOH", "HEI-A", "BRO", "PHG", "BLDR", "FE", "LYV", "ETR", "ALPMY", "PTC", "DOV", "SE", "EXPE", "VRT", "HSNGY", "FLT", "ALGN", "CCJ", "IFF", "TECK", "GPC", "CUK", "CBOE", "WRB", "VTR", "DECK", "AXON", "LH", "COO", "PPL", "CABGY", "KOF", "DRI", "WBA", "GMAB", "TSN", "WMG", "PFG", "PBA", "EXPD", "K", "CHKP", "HBAN", "TRGP", "CTRA", "STLD", "AKAM", "LPLA", "NTAP", "LUV", "STX", "WDC", "WLK", "HUBB", "FDS", "HOLX", "DKNG", "CNP", "CINF", "RPRX", "OMC", "BMRN", "RF", "ATO", "BEP", "SWKS", "BGNE", "HRL", "J", "BAH", "CMS", "L", "RS", "JBL", "CE", "LII", "IHG", "LW", "IEX", "MGA", "FSLR", "TXT", "AER", "BBY", "SYF", "GDDY", "BAM", "SNA", "LDOS", "CSL", "ALLY", "CNHI", "SSNC", "POOL", "DGX", "DPZ", "PKG", "AMH", "MGM", "FWONK", "EQT", "NDSN", "FOX", "VST", "WPC", "HST", "GGG", "CG", "YUMC", "CF", "BIP", "SHG", "KB", "PODD", "ENPH", "RPM", "SUZ", "NWSA", "BEN", "APP", "INCY", "SQM", "SJM", "NICE", "VTRS", "KIM", "ALB", "TAP", "BG", "OKTA", "OC", "UDR", "TRU", "NTNX", "FNF", "TWLO", "WSM", "AKZOY", "KEY", "MRO", "AZPN", "DOCU", "CPB", "UAL", "AFRM", "IP", "LKQ", "ACI", "UHAL", "LECO", "U", "PATH", "BURL", "TEVA", "GFL", "LNT", "TRMB", "ROKU", "SNN", "IPG", "AOS", "CELH", "ACM", "ESTC", "AGR", "JNPR", "DKS", "SAIA", "KRTX", "EXAS", "BLD", "NRG", "PNR", "CNA", "REG", "TFX", "ARCC", "GL", "GFI", "ZI", "RYAN", "LBRDK", "LBRDA", "OTEX", "AES", "USFD", "EQH", "PAYC", "OVV", "RGA", "PFGC", "RNR", "SSDOY", "PAA", "CCK", "JBAXY", "DBX", "DOX", "SRPT", "PEAK", "COTY", "UHS", "SBS", "FFIV", "ALLE", "X", "WYNN", "CASY", "NI", "WES", "CPT", "MOS", "CDAY", "FIVE", "EMN", "QRVO", "WRK", "UTHR", "LSXMA", "HII", "CZR", "CHRW", "EWBC", "SKX", "FLEX", "BRKR", "SCI", "RGEN", "CHK", "PCOR", "WPP", "XPO", "GLOB", "MTCH", "ITT", "PEN", "NLY", "TTC", "NVT", "HSIC", "BBWI", "DVA", "LSCC", "MNDY", "YPF", "GWRE", "SNX", "HOOD", "KNSL", "APA", "CLH", "MEDP", "BIO", "UNM", "CLF", "TREX", "AAL", "RL", "WCC", "CTLT", "AIZ", "TPX", "BSAC", "CNM", "ELF", "TIMB", "WBS", "WEX", "PCTY", "RHI", "CW", "ALV", "MTN", "ETSY", "JEF", "FTI", "PSO", "ORI", "TPR", "SEIC", "COLD", "CHWY", "THC", "CIB", "JLL", "IMGN", "WSC", "WWD", "MSTR", "DLB", "AMKR", "SKM", "NTRA", "CVNA", "CYTK", "PRI", "TXRH", "ACHC", "LBTYK", "PNW", "LBTYA", "NYT", "RMBS", "HESM", "MUSA", "GMED", "BERY", "APG", "BILL", "BWA", "GPK", "NNN", "DCI", "IVZ", "NOV", "CERE", "CIEN", "MIDD", "LEA", "UI", "APLS", "NCLH", "XRAY", "GNTX", "ARMK", "VOYA", "CACI", "OHI", "ASND", "S", "EDR", "FN", "JAZZ", "IONS", "FIX", "WIX", "KBR", "PVH", "MKSI", "SITE", "ELAN", "UFPI", "IEP", "GNRC", "RRC", "INGR", "SF", "AXTA", "EXP", "CBSH", "WAL", "AYI", "EHC", "GPS", "ALTR", "OSK", "EXEL", "BRBR", "LSTR", "DAR", "CMA", "CSAN", "PSN", "WFRD", "AU", "CFR", "AIT", "RHP", "CFLT", "BRX", "HRB", "CR", "STWD", "OGE", "NFE", "CNXC", "W", "CAE", "PPC", "PLNT", "PNFP", "AR", "RLI", "ENSG", "MTSI", "HLNE", "EVR", "HR", "G", "MSA", "SIGI", "BBIO", "MTDR", "MHK", "MTH", "SUM", "ARW", "KT", "MAT", "ITCI", "FLR", "SMAR", "BYD", "HQY", "OLN", "FAF", "SSB", "PB", "FOUR", "WHR", "DDS", "ZION", "NE", "GLBE", "CROX", "SRC", "THO", "NSIT", "WTFC", "LFUS", "HXL", "AN", "BC", "CAR", "INSP", "ESNT", "TNET", "BPOP", "NXST", "IMVT", "MUR", "AM", "CMC", "TMHC", "FSK", "PCVX", "AAON", "REYN", "ATKR", "OPCH", "CIGI", "SON", "FIVN", "POST", "TKR", "ESI", "TENB", "SNV", "RTMVY", "MSM", "VNOM", "MTG", "IBP", "BOKF", "BECN", "OZK", "OMF", "ALGM", "GTLS", "EXLS", "ATI", "AMR", "FLS", "CADE", "PII", "BCC", "ENLC", "DOOO", "VLY", "MLI", "SFM", "WK", "PBF", "ST", "IGT", "COLB", "AMG", "CHX", "ALSN", "ANF", "BEPC", "FCFS", "AIRC", "FTAI", "R", "DLO", "SUN", "MMS", "MTZ", "LYFT", "HOMB", "PAAS", "TXG", "UBSI", "BPMC", "CRSP", "JHG", "ONB", "ACDVF", "CWEN-A", "AGI", "M", "VAL", "VMI", "KRG", "MOG-A", "FNB", "QDEL", "HOG", "COLM", "AXS", "NUVL", "VVV", "TDC", "MDC", "POWI", "LNC", "ASO", "ALKS", "KBH", "BMA", "UGI", "IRDM", "KRC", "OLLI", "EEFT", "SLGN", "SPXC", "MMSI", "GBCI", "BCPC", "FSS", "HALO", "HGV", "SRCL", "AL", "PRGO", "ACAD", "ALK", "GKOS", "DXC", "BMI", "RVMD", "ASGN", "BLKB", "AEL", "AWI", "SIG", "FIZZ", "NFG", "FELE", "SAM", "IPAR", "CC", "IAC", "ACLS", "AVT", "SLM", "NSP", "PFSI", "HAE", "SYNA", "SHC", "PTEN", "ABG", "COOP", "FRPT", "MATX", "FFIN", "GATX", "ARWR", "FUL", "SM", "NVMI", "CBT", "SMPL", "MRTX", "CAAP", "NJR", "HUN", "MGY", "SGRY", "HRI", "PAM", "ASAN", "ORA", "AXSM", "ASH", "INSM", "AEO", "YETI", "PEGA", "OGN", "WEN", "HWC", "ENS", "PAGP", "SEDG", "TFSL", "EPRT", "UMBF", "GGAL", "MDU", "PJT", "TEX", "CPA", "AAP", "STEP", "GPI", "BVN", "APLE", "BOX", "VRRM", "AB", "LOPE", "MHO", "MAN", "URBN", "MAIN", "BCO", "CLS", "RLJ", "CCOI", "BL", "RARE", "INST", "HCC", "LNTH", "NCNO", "KRYS", "LITE", "PYCR", "KMPR", "IRT", "OTTR", "FROG", "ABCB", "BHVN", "CUZ", "PENN", "FOLD", "MAC", "KWR", "RUSHA", "EVH", "CRC", "SFBS", "MOD", "ITGR", "IMCR", "PK", "CVLT", "AVAV", "STNG", "ITRI", "GTES", "AXNX", "SEM", "BTU", "TGS", "ARCH", "TPH", "BSM", "SPT", "NOG", "ALE", "IRTC", "TDOC", "RPD", "WIRE", "DY", "CXM", "HP", "IART", "WD", "TSEM", "FORM", "XENE", "KTB", "JOE", "OGS", "IBOC", "BHF", "SEAS", "CRS", "BNL", "ACIW", "DOC", "VC", "RNG", "ASB", "ERJ", "BUR", "NEA", "HI", "WNS", "DIOD", "SPR", "CBZ", "JBT", "CNX", "CNMD", "CEIX", "SR", "OLK", "ADNT", "CATY", "SHOO", "PNM", "VSH", "DFH", "AX", "CNO", "GTBIF", "GFF", "BDC", "TGNA", "FHI", "YOU", "KLIC", "FIBK", "CVI", "YELP", "TCBI", "DBRG", "DOCN", "NOMD", "AI", "VIRT", "MEOH", "VSAT", "GEF", "KFY", "PBH", "HUBG", "VAC", "LGIH", "LCII", "JWN", "TRIP", "SLG", "SANM", "LBRT", "SITC", "BRC", "GSHD", "SWTX", "RUN", "APAM", "CALX", "TNL", "SHAK", "DEI", "ERF", "LSPD", "IDYA", "ESRT", "GH", "CCS", "MP", "ARLP", "AMN", "ARCB", "HELE", "PDCO", "UPST", "CBAY", "FOXF", "ENV", "WOR", "ABR", "KSS", "SPB", "IDCC", "CSTM", "CWT", "WSFS", "BOH", "LIVN", "PLXS", "CRI", "TMDX", "COUR", "LRN", "HAYW", "HCM", "FTDR", "PPBI", "CVBF", "RAMP", "IIPR", "FULT", "AMPH", "CBU", "OR", "AUB", "CARG", "GBDC", "AVA", "PTVE", "BMBL", "ARCO", "DORM", "CERT", "EVTC", "HTGC", "EGO", "BE", "QTWO", "WERN", "HASI", "CORT", "WHD", "REZI", "ROCK", "FL", "BATRK", "ATGE", "NVG", "PRVA", "HIW", "ICFI", "PRFT", "FSLY", "NEP", "NMIH", "NABL", "USAC", "OSCR", "RYTM", "JAMF", "EBC", "FLYW", "LGF-A", "PGTI", "RCKT", "NHI", "AROC", "FA", "PD", "CRNX", "SFNC", "DNLI", "APPN", "MYRG", "PSMT", "STRL", "DNUT", "OI", "ZLAB", "GPOR", "ALKT", "CENTA", "VCTR", "PRMW", "GLNG", "NTLA", "TGTX", "ENR", "ICUI", "CWK", "NS", "THS", "VIAV", "MCY", "AY", "ROAD", "AMBA", "SHLS", "CPK", "AMK", "SBCF", "CCU", "OUT", "SHO", "ACVA", "BLMN", "MWA", "FCPT", "CLDX", "FFBC", "VKTX", "BKU", "EXTR", "PATK", "ROG", "HTH", "ADX", "DRVN", "TOWN", "MTX", "RIOT", "FRME", "PAX", "ATEC", "IBTX", "TBBK", "EQC", "TDS", "AIR", "GOGL", "PLUS", "CPE", "TGH", "SNEX", "PTCT", "PARR", "OII", "LZ", "DOOR", "JBI", "UTG", "TRN", "PAY", "MRUS", "NAVI", "FUN", "NEO", "ARVN", "LAUR", "SBLK", "GVA", "MGPI", "IRWD", "XRX", "BANC", "UPWK", "WGO", "SWI", "HURN", "ARRY", "PLAY", "PLAB", "PTY", "CMPR", "TWST", "PEB", "BEAM", "ETY", "ODP", "EVCM", "WAFD", "SDGR", "DAN", "VCEL", "HNI", "EPC", "INMD", "SPNT", "VCYT", "NBTB", "HRMY", "VECO", "BKE", "LOB", "VSCO", "YY", "MXL", "DVAX", "KROS", "DFIN", "VRNT", "MBIN", "SSTK", "ANDE", "RNST", "PRIM", "HEES", "WSBC", "FBK", "SNDX", "VET", "CNK", "SAH", "EAT", "CPRX", "VSTO", "LPG", "HPK", "TNC", "FOR", "OCSL", "SEMR", "BANR", "OFG", "ENVA", "TRMK", "CAKE", "SFL", "PRO", "BRP", "VICR", "RXST", "GSBD", "KYMR", "CBRL", "CUBI", "CODI", "EPAC", "LKFN", "COHU", "MRTN", "TNDM", "EFSC", "MSGE", "VGR", "JBGS", "INBX", "HOLI", "ATRC", "HTLF", "VRE", "TALO", "AMEH", "XMTR", "THRM", "B", "SAGE", "KW", "ZIM", "CSGS", "NWBI", "JACK", "SPNS", "KN", "KAR", "OXM", "NVCR", "LZB", "OMCL", "GLP", "AMWD", "TTMI", "FCF", "NGVT", "PCRX", "NHC", "AZZ", "DAC", "GTY", "NTB", "ETV", "LADR", "XHR", "MMI", "SAFE", "UVV", "XPEL", "ZIP", "OMI", "SCS", "PTGX", "FBNC", "JKS", "STAA", "GEL", "HE", "CNNE", "HOPE", "CRSR", "HMN", "PERI", "CRTO", "GBX", "ALGT", "GEO", "VRNA", "PHR", "GIII", "HWKN", "OSW", "AVDL", "GPRE", "DQ", "OEC", "CASH", "PRG", "LTC", "MORF", "NBHC", "FWRD", "KFRC", "UTZ", "CTS", "ETG", "BUSE", "RLAY", "HLIT", "CMRE", "ALEX", "PFS", "CSIQ", "CLMT", "PMT", "SBH", "LGND", "STBA", "PAR", "ACMR", "DEA", "HLF", "MIRM", "DYN", "AGIO", "GES", "PDFS", "CHEF", "FDP", "XNCR", "TGI", "ECPG", "SGH", "TASK", "APOG", "VBTX", "CARS", "MFA", "NVGS", "GOOS", "PRDO", "ALPN", "BASE", "LMND", "SASR", "RCUS", "SCHL", "CWH", "TRUP", "GDEN", "WNC", "AMRC", "SPH", "PEBO", "DOLE", "KURA", "EYPT", "DCPH", "SRRK", "MODN", "AGRO", "MBUU", "VRDN", "EIG", "CMCO", "EFC", "CAL", "PLYM", "THR", "OCFC", "INVA", "RVLV", "UMH", "REVG", "KOP", "BHLB", "TRS", "ETNB", "NX", "MATW", "BZH", "KRUS", "SP", "ATEN", "BVH", "KALU", "PLRX", "FVRR", "METC", "CPLP", "BY", "BCSF", "OPRA", "SBGI", "SCSC", "IMTX", "PWP", "VERV", "POWL", "DCOM", "GDYN", "TTEC", "CENX", "MNRO", "TTGT", "ESTA", "ERII", "BRKL", "ARR", "PRLB", "BHE", "AMSF", "ASPN", "ZNTL", "PRAA", "HAIN", "LBAI", "EVRI", "NMM", "EVBG", "SA", "TWI", "CNOB", "AWF", "CHY", "BLX", "AVNS", "ARCT", "CMP", "SXC", "UNFI", "TMST", "KREF", "SMP", "LBPH", "MRC", "HCI", "NUS", "ACEL", "DGII", "BSIG", "MEG", "LEU", "FBMS", "QTRX", "DLX", "CABA", "IDT", "BELFB", "SLRC", "RAPT", "EGBN", "PFC", "BLBD", "RNA", "CRNC", "SLCA", "TMCI", "BJRI", "TARS", "AMAL", "FLGT", "ICVX", "CCRN", "RGR", "BLFS", "IMAX", "SPTN", "AMCX", "ASTE", "REX", "AOSL", "EH", "HIBB", "SIBN", "HZO", "MYI", "UTI", "MGNX", "GSL", "DMRC", "IMXI", "FDMT", "VTOL", "GNK", "NBR", "DRQ", "CLB", "HA", "KELYA", "CHCT", "HFWA", "DX", "AMBC", "MAX", "CYRX", "SCVL", "DIN", "MYE", "PFLT", "FLWS", "TILE", "EOLS", "FFC", "PBT", "ASIX", "OSBC", "SNCY", "NVRO", "KIDS", "BCYC", "TCPC", "PRA", "IIIN", "THRY", "VERA", "ALXO", "SBOW", "IIIV", "CFB", "ASC", "WSR", "PCN", "MOV", "OLMA", "TRIN", "RILY", "PLOW", "IGMS", "CCAP", "LASR", "IMNM", "RGNX", "VITL", "SWBI", "AAOI", "UVSP", "HBNC", "TITN", "CHUY", "ATRO", "MED", "SILK", "DENN", "FPI", "CLW", "MODV", "NFBK", "FDUS", "HSII", "MITK", "ATEX", "HAFC", "XPOF", "CVRX", "ASLE", "MTW", "EGLE", "ORIC", "HDSN", "GOOD", "GNE", "PETQ", "CLDT", "HONE", "SD", "LQDT", "CSTL", "CPF", "MCB", "WASH", "LUNG", "OFIX", "DSU", "PRAX", "BALY", "NXJ", "GLAD", "CEVA", "UVE", "IRBT", "AEHR", "PNTG", "URGN", "XPER", "PAHC", "RGP", "TPB", "KALV", "HRZN", "LMB", "NTGR", "OSPN", "CRMT", "AMSWA", "FARO", "TPVG", "LYTS", "AVK", "CSTR", "HARP", "CLFD", "CTO", "ORTX", "ITOS", "PSTL", "EFR", "GRPN", "CELC", "SHYF", "EVER", "TREE", "MCFT", "RCEL", "LOVE", "CSV", "ALTG", "BAND", "ZYXI", "ZUMZ", "PLL", "BOOM", "EFT", "TCMD", "SCM", "BBW", "RMAX", "SAR", "JAKK", "SPOK", "AAN", "NEWT", "GCO", "FRST", "CPS", "HMST", "FLIC", "FENC", "JILL", "IPI", "ENTA", "AFCG", "SNBR", "AIF", "LWAY", "HEAR", "VIRC", "RRGB", "INTT", "CMT", "VTSI", "ANVS", "SSNT"
]

World_list = ["7203.T", "RELIANCE.BO", "RELIANCE.NS", "6758.T", "AZN.L", "6861.T", "SHEL.L", "SHELL.XC", "9432.T", "8306.T", "TCS.NS", "HDFCBANK.BO", "HDFCBANK.NS", "BHP.L", "8035.T", "9983.T", "4063.T", "HSBA.L", "HSBAL.XC", "6501.T", "9433.T", "8058.T", "6098.T", "8316.T", "4519.T", "8001.T", "7974.T", "ULVRL.XC", "ULVR.L", "9984.T", "4661.T", "9434.T", "8031.T", "RIOL.XC", "RIO.L", "4568.T", "BP.L", "BPL.XC", "7267.T", "8766.T", "6367.T", "ICICIBANK.NS", "ICICIBANK.BO", "4502.T", "INFY.NS", "2914.T", "6902.T", "8411.T", "GSK.L", "GSKL.XC", "BHARTIARTL.NS", "BHARTIARTL.BO", "7741.T", "DGE.L", "DGEL.XC", "HINDUNILVR.NS", "REL.L", "ITC.NS", "ITC.BO", "6981.T", "SBIN.BO", "SBIN.NS", "GLENL.XC", "GLEN.L", "LICI.NS", "7182.T", "BATS.L", "BATSL.XC", "BNC.L", "3382.T", "LSEG.L", "LT.NS", "BAJFINANCE.NS", "6503.T", "6178.T", "6723.T", "HCLTECH.NS", "6954.T", "5108.T", "6146.T", "8002.T", "8053.T", "RKT.L", "NG.L", "NGL.XC", "6702.T", "FEMSAUBD.MX", "6201.T", "6857.T", "7751.T", "4543.T", "CPG.L", "CPGL.XC", "4901.T", "CRH.L", "6301.T", "9022.T", "KOTAKBANK.NS", "4689.T", "BA.L", "BAL.XC", "ADANIENT.NS", "8801.T", "AXISBANK.NS", "6594.T", "8015.T", "TITAN.NS", "6752.T", "8591.T", "9020.T", "7011.T", "4578.T", "SUNPHARMA.NS", "6920.T", "MARUTI.NS", "8113.T", "4503.T", "5401.T", "ASIANPAINT.NS", "6971.T", "8725.T", "7269.T", "LLOY.L", "9613.T", "HLNL.XC", "HLN.L", "NTPC.BO", "NTPC.NS", "8750.T", "2802.T", "1925.T", "TATAMTRDVR.NS", "TATAMOTORS.BO", "TATAMOTORS.NS", "ONGC.BO", "ONGC.NS", "0QYP.IL", "2502.T", "0R2V.IL", "8267.T", "LLOYL.XC", "EXPN.L", "4452.T", "8802.T", "0700.HK", "6762.T", "4612.T", "ADANIGREEN.NS", "7733.T", "4307.T", "6326.T", "ADANIPORTS.NS", "BAJAJFINSV.NS", "1605.T", "WIPRO.NS", "RRL.XC", "RR.L", "8630.T", "AAL.L", "AALL.XC", "6701.T", "COALINDIA.BO", "COALINDIA.NS", "9735.T", "FLTR.L", "9101.T", "3659.T", "III.L", "PRUL.XC", "PRU.L", "POWERGRID.NS", "POWERGRID.BO", "7201.T", "BARCL.XC", "BARC.L", "8604.T", "1928.T", "9843.T", "4523.T", "8830.T", "AHT.L", "7270.T", "8309.T", "TSCOL.XC", "TSCO.L", "4507.T", "ADANIPOWER.BO", "ADANIPOWER.NS", "BAJAJ-AUTO.NS", "600519.SS", "M&M.NS", "HAL.NS", "7532.T", "JSWSTEEL.NS", "7832.T", "STAN.L", "DLF.NS", "NWG.L", "SSE.L", "0RYA.IL", "IOC.BO", "IOC.NS", "NWGL.XC", "5020.T", "9104.T", "2801.T", "VODL.XC", "VOD.L", "9503.T", "6869.T", "8308.T", "1398.HK", "2503.T", "ABF.L", "8697.T", "IRFC.BO", "IRFC.NS", "4091.T", "JIOFIN.BO", "9107.T", "IMB.L", "TATASTEEL.BO", "TATASTEEL.NS", "VBL.NS", "4911.T", "STANL.XC", "6988.T", "2413.T", "601398.SS", "ANTO.L", "6506.T", "2897.T", "3407.T", "CCL.L", "2587.T", "8601.T", "LGEN.L", "LGENL.XC", "9021.T", "7202.T", "9202.T", "5802.T", "9502.T", "0941.HK", "8593.T", "SBILIFE.NS", "7259.T", "9531.T", "9988.HK", "5411.T", "4755.T", "0857.HK", "1288.HK", "GRASIM.NS", "BEL.BO", "BEL.NS", "7911.T", "HINDZINC.NS", "6645.T", "PFC.NS", "PFC.BO", "9143.T", "7272.T", "9501.T", "4151.T", "HDFCLIFE.NS", "9532.T", "600941.SS", "INDUSINDBK.NS", "TECHM.NS", "601857.SS", "8795.T", "4188.T", "ADANIENSOL.NS", "4528.T", "601288.SS", "7701.T", "1812.T", "WALMEX.MX", "9201.T", "3003.T", "HINDALCO.NS", "9009.T", "7735.T", "6479.T", "INDIGO.NS", "BANKBARODA.BO", "BANKBARODA.NS", "3402.T", "IHG.L", "AV.L", "AVL.XC", "SGEL.XC", "SGE.L", "LODHA.NS", "9766.T", "0005.HK", "5713.T", "4062.T", "RECLTD.NS", "GODREJCP.NS", "RECLTD.BO", "5019.T", "BTL.XC", "0939.HK", "BT-A.L", "9697.T", "TATAPOWER.NS", "TATAPOWER.BO", "ATGL.NS", "ZOMATO.NS", "ZOMATO.BO", "3088.T", "5201.T", "TRENT.NS", "7550.T", "4704.T", "6383.T", "601988.SS", "6586.T", "BNZL.L", "SMT.L", "GAIL.BO", "GAIL.NS", "3988.HK", "PNB.NS", "PNB.BO", "7912.T", "9005.T", "CIPLA.NS", "CHOLAFIN.NS", "TATACONSUM.NS", "AMBUJACEM.NS", "601939.SS", "SGRO.L", "SGROL.XC", "RTOL.XC", "RTO.L", "7261.T", "EICHERMOT.NS", "INFL.XC", "INF.L", "4324.T", "6963.T", "2267.T", "7181.T", "VEDL.NS", "VEDL.BO", "1801.T", "BPCL.NS", "UNIONBANK.BO", "UNIONBANK.NS", "AMXB.MX", "DABUR.NS", "2269.T", "TSLA.NE", "3092.T", "8473.T", "1802.T", "TVSMOTOR.NS", "1911.T", "SNL.XC", "SN.L", "9719.T", "9064.T", "4204.T", "0M2Z.IL", "6504.T", "WISE.L", "HAVELLS.NS", "9041.T", "2702.T", "3038.T", "MANKIND.NS", "CCH.L", "HEROMOTOCO.NS", "SHRIRAMFIN.NS", "AIBG.L", "4021.T", "6305.T", "9007.T", "4183.T", "CANBK.NS", "4185.T", "IOB.BO", "IOB.NS", "S32.L", "HLMA.L", "7936.T", "2875.T", "CNA.L", "3626.T", "CNAL.XC", "7747.T", "IDEA.NS", "0XXT.IL", "3436.T", "0XXV.IL", "MCDOWELL-N.NS", "ADM.L", "IDEA.BO", "6525.T", "7186.T", "WPP.L", "WPPL.XC", "SKG.L", "SVT.L", "JSWENERGY.NS", "6841.T", "MRO.L", "MROL.XC", "8331.T", "IRCTC.NS", "5406.T", "3968.HK", "5334.T", "2768.T", "0R0X.IL", "ICICIPRULI.NS", "3769.T", "JINDALSTEL.NS", "3064.T", "6724.T", "MOTHERSON.BO", "MOTHERSON.NS", "6967.T", "IDBI.BO", "IDBI.NS", "IAG.L", "IAGL.XC", "1299.HK", "7752.T", "SBICARD.NS", "2212.T", "NHPC.NS", "NHPC.BO", "ZYDUSLIFE.NS", "1803.T", "3289.T", "3231.T", "4527.T", "UU.L", "4732.T", "5831.T", "9962.T", "YESBANK.NS", "YESBANK.BO", "1088.HK", "MAXHEALTH.NS", "BHEL.NS", "600036.SS", "BERGEPAINT.NS", "MNDI.L", "BHEL.BO", "5844.T", "9684.T", "CGPOWER.NS", "7211.T", "6753.T", "2318.HK", "7276.T", "MARICO.NS", "WTB.L", "SBRYL.XC", "PSON.L", "SBRY.L", "INDHOTEL.NS", "ICICIGI.NS", "SDR.L", "GMEXICOB.MX", "AUROPHARMA.NS", "0883.HK", "3291.T", "300750.SZ", "3099.T", "0386.HK", "5332.T", "7453.T", "2628.HK", "601088.SS", "8354.T", "6370.T", "LUPIN.NS", "AUTO.L", "GODREJPROP.NS", "AUTOL.XC", "2670.T", "HINDPETRO.NS", "601318.SS", "CRDA.L", "POLYCAB.NS", "9024.T", "4042.T", "6448.T", "600938.SS", "NMDC.NS", "IDFCFIRSTB.BO", "NMDC.BO", "IDFCFIRSTB.NS", "2002.T", "1211.HK", "ENT.L", "600028.SS", "ENTL.XC", "601628.SS", "4613.T", "6532.T", "SUZLON.NS", "BANKINDIA.BO", "SUZLON.BO", "BANKINDIA.NS", "PATANJALI.NS", "JDL.XC", "INDUSTOWER.NS", "JD.L", "4005.T", "BHARATFORG.NS", "SMIN.L", "INDIANB.NS", "7951.T", "3861.T", "PERSISTENT.NS", "600900.SS", "8572.T", "3635.T", "OBEROIRLTY.NS", "CUMMINSIND.NS", "9008.T", "002594.SZ", "BME.L", "BMEL.XC", "2501.T", "9401.T", "5021.T", "7012.T", "CONCOR.NS", "5101.T", "GICRE.NS", "PRESTIGE.NS", "6113.T", "4536.T", "3197.T", "1808.T", "3288.T", "4004.T", "5938.T", "2181.T", "5333.T", "3774.T", "BDEVL.XC", "BDEV.L", "MNG.L", "MNGL.XC", "OCDOL.XC", "0HTP.IL", "OCDO.L", "ASHOKLEY.NS", "8136.T", "ASHOKLEY.BO", "FACT.NS", "AUBANK.NS", "PHNX.L", "MKS.L", "MKSL.XC", "NYKAA.NS", "NYKAA.BO", "TW.L", "9142.T", "9508.T", "TWL.XC", "3132.T", "7731.T", "CTEC.L", "5929.T", "CTECL.XC", "LANDL.XC", "LAND.L", "3405.T", "4967.T", "GMRINFRA.NS", "GMRINFRA.BO", "6526.T", "9506.T", "TATACOMM.NS", "000858.SZ", "UCOBANK.BO", "UCOBANK.NS", "3141.T", "MPHASIS.NS", "0728.HK", "4182.T", "AAF.L", "TORNTPOWER.NS", "JSL.NS", "5947.T", "ASTRAL.NS", "8804.T", "WEIR.L", "GFNORTEO.MX", "TATATECH.NS", "SAIL.BO", "SAIL.NS", "MAZDOCK.NS", "6460.T", "AWL.NS", "3690.HK", "PSN.L", "ICP.L", "9633.HK", "ABCAPITAL.NS", "8252.T", "BRBY.L", "2871.T", "RVNL.BO", "RVNL.NS", "CENTRALBK.NS", "9999.HK", "CENTRALBK.BO", "601728.SS", "6976.T", "PAYTM.NS", "0N4T.IL", "RMV.L", "RMVL.XC", "4751.T", "UTG.L", "7167.T", "6923.T", "1658.HK", "9513.T", "JSWINFRA.NS", "JSWINFRA.BO", "6473.T", "DPH.L", "3116.T", "ACC.NS", "3923.T", "9404.T", "8253.T", "APLAPOLLO.NS", "OIL.NS", "7013.T", "1963.T", "5110.T", "HIK.L", "ACRO.JK", "UPL.NS", "HWDN.L", "L&TFH.BO", "L&TFH.NS", "2433.T", "6417.T", "4202.T", "2331.T", "KPITTECH.NS", "9468.T", "3328.HK", "4385.T", "8905.T", "601658.SS", "5076.T", "SMDS.L", "SMDSL.XC", "KGFL.XC", "UNOMINDA.NS", "SJVN.NS", "KGF.L", "SJVN.BO", "KALYANKJIL.NS", "6324.T", "000333.SZ", "6471.T", "5838.T", "4186.T", "FRES.L", "EZJL.XC", "EZJ.L", "POLICYBZR.NS", "POONAWALLA.NS", "4676.T", "9504.T", "ASLI.JK", "8359.T", "601328.SS", "4912.T", "5105.T", "DIXON.NS", "GUJGASLTD.NS", "BIMBOA.MX", "FEDERALBNK.BO", "8304.T", "FEDERALBNK.NS", "BANDHANBNK.BO", "BANDHANBNK.NS", "2579.T", "9006.T", "9076.T", "BLND.L", "BLNDL.XC", "HSX.L", "7180.T", "3086.T", "8410.T", "NIACL.NS", "MAHABANK.BO", "MAHABANK.NS", "6481.T", "1810.HK", "6141.T", "RS1.L", "6268.T", "7282.T", "HL.L", "SONACOMS.NS", "5233.T", "VOLTAS.NS", "JUBLFOOD.NS", "PETRONET.NS", "M&MFIN.NS", "2899.HK", "INVP.L", "5711.T", "STJ.L", "BEZ.L", "UKW.L", "4927.T", "3563.T", "2371.T", "300760.SZ", "BIOCON.NS", "2412.T", "VTY.L", "2222.T", "8233.T", "EMBASSY.BO", "IREDA.BO", "IREDA.NS", "BBOX.L", "0R87.IL", "BBOXL.XC", "ANGELONE.NS", "STARHEALTH.NS", "NLCINDIA.NS", "4666.T", "5471.T", "FORTIS.NS", "BDL.NS", "9831.T", "MFSL.NS", "NAM-INDIA.NS", "0388.HK", "LICHSGFIN.NS", "AC.MX", "5803.T", "601899.SS", "ABDNL.XC", "ABDN.L", "PSB.NS", "601166.SS", "6030.HK", "3549.T", "9618.HK", "BSE.NS", "ITI.NS", "IGL.NS", "DELHIVERY.NS", "3IN.L", "JMAT.L", "APOLLOTYRE.NS", "APOLLOTYRE.BO", "5830.T", "IGG.L", "0HBY.IL", "MRPL.BO", "MRPL.NS", "6952.T", "SYNGENE.NS", "KOFUBL.MX", "8418.T", "1860.T", "8570.T", "TUI.L", "0HC0.IL", "7337.T", "INCH.L", "7205.T", "4921.T", "002415.SZ", "TATACHEM.NS", "5214.T", "3994.T", "IRB.NS", "IRB.BO", "MSUMI.BO", "9888.HK", "MSUMI.NS", "GODREJIND.NS", "JET2.L", "4205.T", "8334.T", "TRIG.L", "0QR4.IL", "EXIDEIND.NS", "600030.SS", "SHC.L", "GFINBURO.MX", "SUNTV.NS", "0HBP.IL", "EMG.L", "ROR.L", "600276.SS", "HUDCO.NS", "HUDCO.BO", "3401.T", "0998.HK", "HICL.L", "6770.T", "GLENMARK.NS", "9552.T", "CYIENT.NS", "DARK.L", "INPP.L", "9783.T", "601138.SS", "IIFL.NS", "HINDCOPPER.NS", "TATE.L", "2531.T", "2127.T", "600809.SS", "NATIONALUM.NS", "1332.T", "NATIONALUM.BO", "HINDCOPPER.BO", "ITV.L", "ITVL.XC", "601998.SS", "3387.T", "IDS.L", "2015.HK", "8876.T", "JET.L", "EMAMILTD.NS", "3941.T", "1893.T", "360ONE.NS", "2175.T", "6755.T", "601816.SS", "2359.HK", "TRIDENT.NS", "4208.T", "TRIDENT.BO", "0H6T.IL", "8282.T", "ZEEL.NS", "ZEEL.BO", "3865.T", "7189.T", "000568.SZ", "7163.T", "0O87.IL", "HBR.L", "KIOCL.NS", "HBRL.XC", "0016.HK", "BYG.L", "LAURUSLABS.NS", "0YSU.L", "002475.SZ", "BRIGADE.NS", "DEVYANI.NS", "9507.T", "7762.T", "601225.SS", "DLG.L", "BAB.L", "AARTIIND.NS", "5301.T", "DLGL.XC", "2427.T", "600309.SS", "ABFRL.NS", "2328.HK", "0267.HK", "6925.T", "BSOFT.NS", "603259.SS", "9412.T", "0R7R.IL", "FIVESTAR.NS", "0QOK.IL", "BVIC.L", "6690.HK", "BPT.L", "LMP.L", "SONATSOFTW.NS", "4902.T", "5032.T", "2020.HK", "0QLR.IL", "PEL.NS", "NETW.L", "0NMK.IL", "CROMPTON.NS", "PNN.L", "ROO.L", "CTY.L", "DRX.L", "1339.HK", "2601.HK", "300059.SZ", "JGGI.L", "1024.HK", "COCHINSHIP.NS", "8628.T", "002714.SZ", "HGT.L", "2388.HK", "5423.T", "601668.SS", "VMUK.L", "VMUKL.XC", "IDFC.NS", "0981.HK", "ASTERDM.NS", "9090.T", "8698.T", "4506.T", "RDW.L", "8086.T", "7956.T", "600690.SS", "002352.SZ", "CDSL.NS", "LXI.L", "CELLO.NS", "RY.TO", "600000.SS", "603288.SS", "000651.SZ", "IRCON.BO", "IRCON.NS", "3105.T", "7518.T", "SHYAMMETL.NS", "8515.T", "CEMEXCPO.MX", "2760.T", "NOVNZ.XC", "GRI.L", "1719.T", "601601.SS", "9961.HK", "8242.T", "601319.SS", "PPLPHARMA.NS", "PPLPHARMA.BO", "1880.HK", "4180.T", "CIEINDIA.NS", "CESC.NS", "CESC.BO", "BBY.L", "SRP.L", "WWH.L", "688981.SS", "PHLL.L", "3186.T", "QQ.L", "JYOTHYLAB.NS", "4587.T", "GSPL.NS", "TTML.NS", "TTML.BO", "5631.T", "6818.HK", "EIHOTEL.NS", "ALOKINDS.NS", "4443.T", "2333.HK", "000001.SZ", "8358.T", "IWG.L", "RBLBANK.NS", "4478.T", "1109.HK", "1928.HK", "RBLBANK.BO", "7732.T", "CASTROLIND.NS", "ENOG.L", "0027.HK", "SSPG.L", "6754.T", "6425.T", "APTUS.NS", "1816.HK", "PNL.L", "2432.T", "6315.T", "0066.HK", "1876.HK", "FGT.L", "0G8C.IL", "MRC.L", "600887.SS", "GMXT.MX", "600406.SS", "ALOKINDS.BO", "0NW4.IL", "300999.SZ", "601888.SS", "6674.T", "1988.HK", "OSB.L", "5801.T", "5253.T", "INDV.L", "HSCL.NS", "601012.SS", "NBCC.NS", "NBCC.BO", "TEM.L", "JINDALSAW.NS", "601818.SS", "TPK.L", "601633.SS", "AML.L", "1919.HK", "0011.HK", "N91.L", "FINCABLES.NS", "BLS.NS", "MCX.NS", "CHAMBLFERT.NS", "TANLA.NS", "4680.T", "0002.HK", "1766.HK", "2928.S", "HAS.L", "ASHM.L", "0669.HK", "7004.T", "YCA.L", "0001.HK", "9505.T", "003816.SZ", "600025.SS", "MYI.L", "1171.HK", "PAG.L", "600104.SS", "0A28.IL", "688041.SS", "JWL.NS", "LIVEPOLC-1.MX", "0762.HK", "PIN.L", "HONASA.NS", "600016.SS", "3863.T", "200725.SZ", "NATCOPHARM.NS", "8609.T", "TD.TO", "601985.SS", "601919.SS", "PAGE.L", "LRE.L", "FINPIPE.NS", "PVRINOX.NS", "2337.T", "WOSG.L", "WTAN.L", "6160.HK", "INOXWIND.NS", "300124.SZ", "WELSPUNLIV.NS", "INOXWIND.BO", "NSLNISP.NS", "0Q11.IL", "NSLNISP.BO", "GMDCLTD.NS", "7148.T", "6871.T", "TRN.L", "6254.T", "MONY.L", "IEX.NS", "J&KBANK.NS", "IEX.BO", "002304.SZ", "6472.T", "MANAPPURAM.BO", "MANAPPURAM.NS", "QLT.L", "0390.HK", "TITAGARH.NS", "SWANENERGY.NS", "8368.T", "601766.SS", "TCAP.L", "FSL.NS", "BYIT.L", "DOM.L", "PETS.L", "RKFORGE.NS", "TTE.PA", "TTE.L", "200625.SZ", "0788.HK", "REDINGTON.NS", "0RH0.IL", "8616.T", "ASURB.MX", "600188.SS", "KARURVYSYA.NS", "AGR.L", "SHOP.TO", "HMSOL.XC", "AGRL.XC", "WELCORP.NS", "ARE&M.NS", "2160.T", "000725.SZ", "1898.HK", "BHMG.L", "HMSO.L", "601066.SS", "SEQI.L", "9509.T", "8524.T", "2338.HK", "DWLL.XC", "DWL.L", "GAPB.MX", "J&KBANK.BO", "0P6M.IL", "OLECTRA.NS", "GESHIP.NS", "300015.SZ", "9603.T", "0688.HK", "1113.HK", "MTO.L", "PHP.L", "2611.HK", "TRITURBINE.NS", "688235.SS", "ZENSARTECH.NS", "600436.SS", "601390.SS", "ONT.L", "600150.SS", "002142.SZ", "8848.T", "600019.SS", "300498.SZ", "5727.T", "300274.SZ", "600050.SS", "ASCL.L", "2269.HK", "HBLPOWER.NS", "000625.SZ", "NETWORK18.BO", "NETWORK18.NS", "HFCL.NS", "RITES.NS", "6890.T", "EQUITASBNK.NS", "600919.SS", "SPARC.NS", "ENGINERSIN.BO", "ENGINERSIN.NS", "0QWI.IL", "0992.HK", "AEGISCHEM.NS", "BNKR.L", "SMS.L", "601898.SS", "0902.HK", "HFCL.BO", "SOBHA.NS", "6886.HK", "4565.T", "CHENNPETRO.NS", "0O59.IL", "MGL.NS", "600018.SS", "0RJI.IL", "600905.SS", "000338.SZ", "3908.HK", "1519.HK", "0916.HK", "002371.SZ", "DBREALTY.NS", "NCC.NS", "0KVV.IL", "0914.HK", "ITX.MC", "3993.HK", "601211.SS", "PFD.L", "AJB.L", "2317.T", "ATT.L", "SRE.L", "300122.SZ", "SAN.PA", "BREE.L", "2202.HK", "RPOWER.BO", "JMG.L", "UJJIVANSFB.NS", "RPOWER.NS", "RAILTEL.BO", "RAILTEL.NS", "UJJIVANSFB.BO", "AIR.PA", "JPPOWER.NS", "JPPOWER.BO", "0763.HK", "601006.SS", "600438.SS", "0753.HK", "9866.HK", "ALOKTEXT.BO", "CUERVO.MX", "RTNINDIA.NS", "RTNINDIA.BO", "WG.L", "601688.SS", "GSFC.NS", "2313.HK", "601995.SS", "600011.SS", "603501.SS", "6099.HK", "6632.T", "FGP.L", "GSFC.BO", "GNFC.NS", "1038.HK", "COA.L", "600031.SS", "1929.HK", "0MPH.IL", "CUB.NS", "9166.T", "FUNO11.MX", "600585.SS", "1913.HK", "688111.SS", "001289.SZ", "3778.T", "0003.HK", "1800.HK", "603993.SS", "CEYL.XC", "CEY.L", "0823.HK", "IEM.L", "0012.HK", "0NWV.IL", "CNR.TO", "RAJESHEXPO.NS", "IBULHSGFIN.NS", "GRAPHITE.NS", "6806.HK", "IBULHSGFIN.BO", "TV18BRDCST.NS", "688271.SS", "LEMONTREE.NS", "600048.SS", "0H6X.IL", "TV18BRDCST.BO", "3606.HK", "SYRMA.NS", "SWSOLAR.NS", "000002.SZ", "SUPR.L", "6837.HK", "688036.SS", "7868.T", "ANANTRAJ.NS", "1776.HK", "JKTYRE.NS", "000063.SZ", "600989.SS", "601111.SS", "ENB.TO", "6727.T", "1055.HK", "0MCK.IL", "GRANULES.NS", "PCBL.NS", "4996.T", "MMTC.NS", "MMTC.BO", "SCHNEIDER.NS", "RICA.L", "GPE.L", "EIDPARRY.NS", "600999.SS", "3962.T", "6618.HK", "9901.HK", "GRSE.NS", "600760.SS", "2492.T", "0Q4U.IL", "2767.T", "CANFINHOME.NS", "0291.HK", "AGT.L", "601127.SS", "1787.HK", "601800.SS", "601169.SS", "PNCINFRA.NS", "BBGI.L", "5726.T", "600660.SS", "LATENTVIEW.NS", "RENUKA.NS", "000596.SZ", "6881.HK", "RENUKA.BO", "600886.SS", "4293.T", "SU.PA", "4776.T", "600837.SS", "000166.SZ", "PRAJIND.NS", "002493.SZ", "0006.HK", "CP.TO", "JMFINANCIL.NS", "GEN.L", "9519.T", "CNQ.TO", "KTKBANK.NS", "FCSS.L", "HTWS.L", "3498.T", "002050.SZ", "PRSMJOHNSN.NS", "000776.SZ", "601186.SS", "CHG.L", "2600.HK", "000895.SZ", "RCF.NS", "RCF.BO", "601989.SS", "USHAMART.NS", "SBFC.NS", "600015.SS", "MINDACORP.NS", "603392.SS", "8714.T", "1972.HK", "600029.SS", "EMIL.NS", "7014.T", "RELINFRA.NS", "002230.SZ", "AI.PA", "KFINTECH.NS", "600547.SS", "DOMS.NS", "FUTR.L", "4617.T", "HAPPYFORGE.NS", "6740.T", "7342.T", "601881.SS", "FIBRAPL14.MX", "JUBLPHARMA.NS", "JUST.L", "0168.HK", "1514.T", "600845.SS", "601229.SS", "SURYAROSNI.NS", "600893.SS", "6366.T", "0R3T.IL", "THG.L", "CAMPUS.NS", "000538.SZ", "TVSSCS.NS", "RWS.L", "SONG.L", "688223.SS", "VARROC.NS", "601600.SS", "RATEGAIN.NS", "0MHZ.IL", "002027.SZ", "BBH.L", "1772.HK", "002466.SZ", "688012.SS", "600346.SS", "7599.T", "EASEMYTRIP.BO", "EASEMYTRIP.NS", "9348.T", "0H2Z.IL", "MANINFRA.NS", "SQZ.L", "0670.HK", "0087.HK", "000792.SZ", "UKCM.L", "601669.SS", "2931.T", "0019.HK", "300308.SZ", "DEEPAKFERT.NS", "BN.TO", "AWE.L", "UBSGZ.XC", "300014.SZ", "BMO.TO", "600009.SS", "1179.HK", "9868.HK", "SCI.NS", "VIPIND.NS", "600600.SS", "0175.HK", "GPPL.NS", "GAEL.NS", "IFCI.NS", "IFCI.BO", "ALLCARGO.NS", "000100.SZ", "REDD.L", "INOXINDIA.NS", "603195.SS", "1336.HK", "6590.T", "INDIACEM.NS", "THOMASCOOK.NS", "601009.SS", "BALRAMCHIN.NS", "2238.HK", "2484.T", "SYNC.L", "TFIF.L", "3996.HK", "002736.SZ", "ELECTCAST.NS", "3692.HK", "MIDHANI.NS", "MGAM.L", "002460.SZ", "001979.SZ", "0836.HK", "6862.HK", "BNS.TO", "300033.SZ", "LXCHEM.NS", "QUESS.NS", "ARVIND.NS", "DOCS.L", "ACI.NS", "0HIT.IL", "BNP.PA", "CRN.L", "0316.HK", "8346.T", "1093.HK", "600115.SS", "600795.SS", "AVANTIFEED.NS", "601100.SS", "ATD.TO", "7241.T", "2319.HK", "8016.T", "INFIBEAM.BO", "INFIBEAM.NS", "TIFS.L", "1997.HK", "KNRCON.NS", "UJJIVAN.NS", "TRIVENI.NS", "601868.SS", "STARCEMENT.NS", "2016.HK", "601336.SS", "TEXRAIL.NS", "VAIBHAVGBL.NS", "0285.HK", "BSIF.L", "601238.SS", "9229.T", "6823.HK", "NAFTRACISHRS.MX", "ELM.L", "600089.SS", "IBE.MC", "SNR.L", "MARKSANS.NS", "0004.HK", "BBAJIOO.MX", "RELIGARE.NS", "LTFOODS.NS", "JAICORPLTD.NS", "3958.HK", "002179.SZ", "SITES1A-1.MX", "601018.SS", "605499.SS", "0NXX.IL", "SAF.PA", "600085.SS", "JKPAPER.NS", "EDELWEISS.NS", "601689.SS", "TRST.L", "EDELWEISS.BO", "7003.T", "ENEL.MI", "SEIT.L", "0NRE.IL", "600023.SS", "0NX2.IL", "OMAB.MX", "4384.T", "0NQM.IL", "000708.SZ", "600674.SS", "9696.HK", "002311.SZ", "NAVA.NS", "MEGACPO.MX", "SUNTECK.NS", "601699.SS", "8508.T", "UTKARSHBNK.NS", "300896.SZ", "UTKARSHBNK.BO", "GBG.L", "0QMU.IL", "0HAR.IL", "0960.HK", "601916.SS", "ABBNZ.XC", "SPT.L", "DG.PA", "1801.HK", "MSLH.L", "0083.HK", "000617.SZ", "JLEN.L", "CS.PA", "WOCKPHARMA.NS", "600039.SS", "2688.HK", "000425.SZ", "6178.HK", "600703.SS", "600111.SS", "300782.SZ", "2382.HK", "0288.HK", "600010.SS", "BORORENEW.NS", "0O0U.IL", "600958.SS", "2196.HK", "1099.HK", "ORBIA.MX", "GRID.L", "Q.MX", "1618.HK", "1138.HK", "1821.T", "0598.HK", "0QXR.IL", "002241.SZ", "688303.SS", "601698.SS", "EPL.NS", "6865.HK", "PARADEEP.NS", "MOIL.NS", "EWI.L", "002459.SZ", "000301.SZ", "BLUEJET.NS", "STAR.NS", "2883.HK", "SRC.L", "ALFAA.MX", "PZC.L", "001965.SZ", "0QE6.IL", "000963.SZ", "8600.T", "688008.SS", "ARVINDFASN.NS", "9204.T", "0AA9.IL", "601825.SS", "300979.SZ", "STLAP.PA", "STLAM.MI", "2727.HK", "RBA.NS", "GCP.L", "600926.SS", "601901.SS", "002920.SZ", "PTC.NS", "2607.HK", "0HB4.IL", "300433.SZ", "0135.HK", "0AAS.IL", "MSTCLTD.NS", "2170.T", "601598.SS", "CCR.L", "SHED.L", "000983.SZ", "688599.SS", "INDIASHLTR.NS", "FSFL.L", "KIMBERA.MX", "2353.T", "SAN.MC", "601788.SS", "GATEWAY.NS", "DSY.PA", "1071.HK", "NESF.L", "002129.SZ", "DBL.NS", "STLTECH.NS", "2618.HK", "CMSINFO.NS", "600026.SS", "600221.SS", "9164.T", "1177.HK", "CORD.L", "600196.SS", "601618.SS", "LTG.L", "SOUTHBANK.NS", "SOUTHBANK.BO", "603260.SS", "ESP.L", "ASTRAMICRO.NS", "300832.SZ", "601865.SS", "RTNPOWER.NS", "RTNPOWER.BO", "JSG.L", "601808.SS", "1157.HK", "0358.HK", "VESTA.MX", "0177.HK", "ORIENTCEM.NS", "600372.SS", "1378.HK", "SU.TO", "UCG.MI", "ICIL.NS", "1209.HK", "GENUSPOWER.NS", "5202.T", "601727.SS", "IBST.L", "ASKAUTOLTD.NS", "IPO.L", "9896.HK", "600803.SS", "000877.SZ", "002236.SZ", "PURVA.NS", "3856.T", "MOON.L", "GABRIEL.NS", "TRP.TO", "601360.SS", "603369.SS", "THRL.L", "CRST.L", "3679.T", "CHOICEIN.NS", "0151.HK", "MCG.L", "1193.HK", "601607.SS", "600426.SS", "0H3Q.IL", "0IAH.IL", "688396.SS", "HELD.L", "601872.SS", "USA.L", "600027.SS", "HCC.NS", "CURY.L", "601059.SS", "000768.SZ", "301269.SZ", "IBREALEST.BO", "IBREALEST.NS", "BCPT.L", "000157.SZ", "KESORAMIND.NS", "002001.SZ", "600741.SS", "6963.HK", "601021.SS", "SULA.NS", "0HBC.IL", "MFC.TO", "2800.HK", "600362.SS", "HCC.BO", "600377.SS", "ITDCEM.NS", "ORIT.L", "HOTC.L", "3182.T", "FEDFINA.NS", "300316.SZ", "002812.SZ", "603986.SS", "RA.MX", "0293.HK", "AO.L", "603799.SS", "KIE.L", "BCE.TO", "RAIN.NS", "600256.SS", "PATELENG.BO", "PATELENG.NS", "688256.SS", "002049.SZ", "RALLIS.NS", "000999.SZ", "TIPSINDLTD.NS", "TARC.NS", "000938.SZ", "ISP.MI", "0241.HK", "IIFLSEC.NS", "ALSEA.MX", "000661.SZ", "6066.HK", "300408.SZ", "HEMIPROP.NS", "NFL.NS", "NFL.BO", "LLOYDSENGG.NS", "7184.T", "688728.SS", "JPASSOCIAT.NS", "0N9S.IL", "JPASSOCIAT.BO", "3618.HK", "0874.HK", "9517.T", "002648.SZ", "TI.NS", "BVA.L", "ENI.MI", "BBVA.MC", "603198.SS", "0101.HK", "DCBBANK.NS", "3759.HK", "6619.T", "TLW.L", "603019.SS", "600161.SS", "603806.SS", "0322.HK", "HOC.L", "002252.SZ", "600588.SS", "7779.T", "6228.T", "PDG.L", "0RIC.IL", "0KED.IL", "PAISALO.NS", "MEGP.L", "5408.T", "600584.SS", "600745.SS", "600570.SS", "NOCIL.NS", "601377.SS", "000786.SZ", "QSR.TO", "PRSR.L", "PCFT.L", "002938.SZ", "ASC.L", "3898.HK", "1072.HK", "PRICOLLTD.NS", "0RLS.IL", "2331.HK", "GNC.L", "600489.SS", "601877.SS", "1821.HK", "600918.SS", "3969.HK", "4449.T", "ESNT.L", "JAMNAAUTO.NS", "601838.SS", "3347.HK", "002709.SZ", "IPL.NS", "6266.T", "600236.SS", "301236.SZ", "BOO.L", "002271.SZ", "3561.T", "601136.SS", "6198.HK", "JKIL.NS", "3808.HK", "PTAL.L", "000977.SZ", "601077.SS", "LACOMERUBC.MX", "BALMLAWRIE.NS", "JAYNECOIND.NS", "ASHOKA.NS", "4837.T", "600332.SS", "688126.SS", "3311.HK", "300759.SZ", "2638.HK", "600985.SS", "HATHWAY.NS", "1347.HK", "HATHWAY.BO", "GSF.L", "JTLIND.NS", "FXPO.L", "GROW.L", "0991.HK", "GALLANTT.NS", "0144.HK", "JUP.L", "JTEKTINDIA.NS", "CHRY.L", "XPS.L", "4592.T", "603659.SS", "688009.SS", "SAMHI.NS", "002032.SZ", "000408.SZ", "002028.SZ", "600160.SS", "IMO.TO", "600875.SS", "688082.SS", "0GRX.L", "0BFA.IL", "688187.SS", "600061.SS", "000630.SZ", "300418.SZ", "ABX.TO", "601298.SS", "600233.SS", "300413.SZ", "0384.HK", "TIMETECHNO.NS", "BCG.BO", "600183.SS", "MUTHOOTMF.NS", "BCG.NS", "300347.SZ", "0HAI.IL", "BGS.L", "300765.SZ", "PINT.L", "600515.SS", "3133.T", "603939.SS", "SINDHUTRAD.NS", "AZAD.NS", "DISHTV.NS", "002945.SZ", "603833.SS", "JISLJALEQS.BO", "JISLJALEQS.NS", "GWO.TO", "000975.SZ", "2282.HK", "INOXGREEN.NS", "0270.HK", "EBOX.L", "600415.SS", "DELTACORP.NS", "SLF.TO", "WABAG.NS", "DELTACORP.BO", "002422.SZ", "000876.SZ", "000807.SZ", "CVE.TO", "600482.SS", "002555.SZ", "0KFX.IL", "BN.PA", "0268.HK", "KALAMANDIR.NS", "0INB.IL", "001872.SZ", "SANGHIIND.NS", "CLI.L", "9863.HK", "600522.SS", "ENGI.PA", "0LD0.IL", "2150.T", "NCC.L", "002600.SZ", "GGP.L", "IHR.L", "HFD.L", "601991.SS", "002463.SZ", "9926.HK", "5563.T", "6817.T", "3998.HK", "2039.HK", "600079.SS", "000933.SZ", "002601.SZ", "CMCX.L", "DISHTV.BO", "1548.HK", "9522.T", "000564.SZ", "WHR.L", "002821.SZ", "SUNFLAG.NS", "605117.SS", "FCL.NS", "4475.T", "0881.HK", "300442.SZ", "TRIL.NS", "002603.SZ", "688169.SS", "9424.T", "2268.HK", "000800.SZ", "SHI.L", "ACA.PA", "601117.SS", "601058.SS", "002602.SZ", "601878.SS", "0780.HK", "0656.HK", "688047.SS", "600176.SS", "300450.SZ", "600925.SS", "000932.SZ", "2380.HK", "6110.HK", "603605.SS", "688188.SS", "002074.SZ", "888.L", "3315.T", "CARD.L", "0696.HK", "4714.T", "ESAFSFB.NS", "1585.HK", "GENTERA.MX", "3668.HK", "688469.SS", "601799.SS", "FLAIR.NS", "0392.HK", "SAGCEM.NS", "688072.SS", "MRL.L", "002007.SZ", "600350.SS", "0P6S.IL", "0968.HK", "STMMI.MI", "STMPA.PA", "WIX.L", "T.TO", "GREAVESCOT.NS", "PCTN.L", "6969.HK", "600348.SS", "2367.HK", "601061.SS", "DCXINDIA.NS", "0020.HK", "HMAAGRO.NS", "1877.HK", "000039.SZ", "BAJAJHIND.NS", "1308.HK", "6186.HK", "002078.SZ", "300037.SZ", "BAJAJHIND.BO", "600460.SS", "300142.SZ", "APOLLO.BO", "APOLLO.NS", "600546.SS", "7816.T", "AEM.TO", "0576.HK", "600663.SS", "600219.SS", "601168.SS", "002223.SZ", "9626.HK", "600066.SS", "601555.SS", "601108.SS", "300751.SZ", "688249.SS", "CPI.L", "600642.SS", "688114.SS", "0MPP.IL", "RCI-B.TO", "002294.SZ", "NA.TO", "601236.SS", "300628.SZ", "4477.T", "SEQUENT.NS", "BOMDYEING.NS", "300661.SZ", "DANHOS13.MX", "NTR.TO", "002064.SZ", "1910.HK", "002128.SZ", "600398.SS", "000423.SZ", "300502.SZ", "2880.HK", "0921.HK", "7803.T", "0412.HK", "300394.SZ", "1128.HK", "1044.HK", "6223.T", "601233.SS", "0973.HK", "300866.SZ", "3320.HK", "IMAGICAA.NS", "YATHARTH.NS", "688180.SS", "SHALBY.NS", "ITM.L", "SEPC.NS", "SEPC.BO", "0008.HK", "3660.T", "PRAKASH.NS", "002156.SZ", "688363.SS", "002916.SZ", "8550.T", "688538.SS", "4259.T", "COD.L", "600606.SS", "GMRP&UI.BO", "0868.HK", "GMRP&UI.NS", "CWR.L", "603688.SS", "SGO.PA", "BAJAJCON.NS", "1513.HK", "301308.SZ", "600109.SS", "PAF.L", "001286.SZ", "688777.SS", "300919.SZ", "601231.SS", "0867.HK", "5074.T", "GSEO.L", "GREENPLY.NS", "601216.SS", "SPICEJET.BO", "603899.SS", "GABI.L", "2208.HK", "6227.T", "601958.SS", "600177.SS", "600418.SS", "2930.T", "600733.SS", "600732.SS", "603658.SS", "DOTD.L", "600157.SS", "600153.SS", "0522.HK", "G.MI", "688349.SS", "600132.SS", "0772.HK", "2799.HK", "002939.SZ", "000009.SZ", "600298.SS", "601880.SS", "002465.SZ", "601567.SS", "300763.SZ", "3800.HK", "SATIN.NS", "AVCT.L", "600566.SS", "5017.T", "CAML.L", "0R9C.IL", "000921.SZ", "DCAL.NS", "2866.HK", "IDOX.L", "300496.SZ", "600873.SS", "000629.SZ", "CCO.TO", "603290.SS", "600702.SS", "000027.SZ", "DEN.NS", "1882.HK", "AVANTEL.BO", "601966.SS", "600968.SS", "002353.SZ", "1033.HK", "002756.SZ", "0ILK.IL", "002625.SZ", "3888.HK", "0HA0.IL", "INNOVACAP.NS", "600096.SS", "0OQV.IL", "000066.SZ", "688120.SS", "WPM.TO", "000513.SZ", "600995.SS", "ORA.PA", "0O5H.IL", "300957.SZ", "688301.SS", "000987.SZ", "CABK.MC", "6979.HK", "603568.SS", "BEPL.NS", "002180.SZ", "FMTY14.MX", "603816.SS", "002414.SZ", "PARACABLES.NS", "603233.SS", "688375.SS", "9995.HK", "UNITECH.BO", "TM17.L", "UNITECH.NS", "000883.SZ", "600754.SS", "DX.L", "9890.HK", "300474.SZ", "000728.SZ", "603160.SS", "601158.SS", "600118.SS", "HERITGFOOD.NS", "601577.SS", "002202.SZ", "600884.SS", "600998.SS", "0317.HK", "000783.SZ", "002738.SZ", "FNV.TO", "ISMTLTD.NS", "601990.SS", "603345.SS", "300601.SZ", "603156.SS", "0659.HK", "AMS.MC", "002841.SZ", "1818.HK", "1908.HK", "1359.HK", "YATRA.NS", "600895.SS", "0489.HK", "002739.SZ", "002262.SZ", "300146.SZ", "PFS.NS", "PFS.BO", "SKIPPER.NS", "1456.HK", "300003.SZ", "TERRA13.MX", "DOL.TO", "GIPCL.NS", "3331.HK", "ASLI.L", "1066.HK", "0017.HK", "603885.SS", "000733.SZ", "600816.SS", "CM.TO", "600487.SS", "DIVI.L", "1797.HK", "0EYG.IL", "688065.SS", "300223.SZ", "600060.SS", "600862.SS", "9992.HK", "600871.SS", "002673.SZ", "002532.SZ", "000831.SZ", "OTB.L", "FTS.TO", "600779.SS", "600885.SS", "BHARATWIRE.NS", "603000.SS", "000723.SZ", "300454.SZ", "601696.SS", "600583.SS", "002340.SZ", "600863.SS", "2471.T", "NIOX.L", "600517.SS", "000988.SZ", "603606.SS", "NCYF.L", "601866.SS", "603087.SS", "600612.SS", "000937.SZ", "600970.SS", "VPRPL.NS", "0956.HK", "601179.SS", "002384.SZ", "688819.SS", "TLEVISACPO.MX", "300144.SZ", "PCJEWELLER.BO", "TECK-B.TO", "PCJEWELLER.NS", "603338.SS", "688122.SS", "002653.SZ", "603786.SS", "600352.SS", "002444.SZ", "200539.SZ", "002608.SZ", "600705.SS", "ZAGGLE.NS", "002152.SZ", "601666.SS", "301267.SZ", "600685.SS", "ENQ.L", "2936.T", "0338.HK", "SUVEN.NS", "601162.SS", "CONFIPET.NS", "0564.HK", "0P2N.IL", "600518.SS", "3323.HK", "002439.SZ", "000959.SZ", "688234.SS", "ENQL.XC", "FIBRAMQ12.MX", "ADIG.L", "CABP.L", "600369.SS", "601615.SS", "3360.HK", "DGI9.L", "603596.SS", "600295.SS", "FER.MC", "0RI8.IL", "TIRUMALCHM.NS", "2018.HK", "300207.SZ", "SFOR.L", "002080.SZ", "002430.SZ", "600536.SS", "SHK.NS", "688331.SS", "600497.SS", "NRR.L", "GANDHAR.NS", "PPL.TO", "688295.SS", "603589.SS", "0QFT.IL", "6625.T", "ALPEKA.MX", "600072.SS", "601198.SS", "ALEMBICLTD.NS", "601456.SS", "EMSLIMITED.NS", "MTRO.L", "CLNX.MC", "601933.SS", "SUPRIYA.NS", "PARAGMILK.NS", "003022.SZ", "600637.SS", "002456.SZ", "603882.SS", "0023.HK", "600378.SS", "MOREPENLAB.BO", "002153.SZ", "MOREPENLAB.NS", "002607.SZ", "002410.SZ", "GKP.L", "002831.SZ", "002568.SZ", "002966.SZ", "002185.SZ", "300251.SZ", "POW.TO", "600535.SS", "000703.SZ", "000069.SZ", "2357.HK", "688220.SS", "600655.SS", "GTLINFRA.NS", "GTLINFRA.BO", "SOLG.L", "600848.SS", "SYNT.L", "002429.SZ", "600956.SS", "SOHO.L", "600486.SS", "603826.SS", "300724.SZ", "300487.SZ", "301358.SZ", "600688.SS", "KCP.NS", "000539.SZ", "600549.SS", "PANR.L", "6060.HK", "002157.SZ", "601155.SS", "600582.SS", "688390.SS", "2120.T", "300114.SZ", "000729.SZ", "601228.SS", "601099.SS", "688561.SS", "0347.HK", "601156.SS", "688297.SS", "1385.HK", "600879.SS", "CAPACITE.NS", "600906.SS", "600707.SS", "HINDOILEXP.NS", "FILATEX.NS", "1114.HK", "VTU.L", "601000.SS", "600021.SS", "600988.SS", "603979.SS", "003031.SZ", "603298.SS", "JAGRAN.NS", "0RP9.IL", "688032.SS", "603893.SS", "H.TO", "0RBI.IL", "KCT.L", "002797.SZ", "000050.SZ", "3010.T", "601928.SS", "002595.SZ", "GREENPOWER.NS", "VAKRANGEE.NS", "600765.SS", "GREENPOWER.BO", "VAKRANGEE.BO", "SALASAR.NS", "300888.SZ", "IQE.L", "603707.SS", "688475.SS", "601717.SS", "600704.SS", "600699.SS", "2602.HK", "000960.SZ", "002372.SZ", "603728.SS", "DHANI.NS", "0220.HK", "000738.SZ", "SUBEXLTD.NS", "0966.HK", "002056.SZ", "603737.SS", "688153.SS", "600499.SS", "EQLS.L", "300748.SZ", "3825.T", "MANINDS.NS", "6613.T", "000799.SZ", "002385.SZ", "LASITEB-1.MX", "0552.HK", "002176.SZ", "600380.SS", "MTNL.NS", "600763.SS", "PRECWIRE.NS", "600129.SS", "600511.SS", "600839.SS", "603179.SS", "SHRIRAMPPS.NS", "600004.SS", "0013.HK", "SEE.L", "002984.SZ", "ORIENTHOT.NS", "603486.SS", "MOL.NS", "688099.SS", "600977.SS", "688387.SS", "688520.SS", "SREI.L", "002409.SZ", "MTNL.BO", "600299.SS", "603712.SS", "0TDE.IL", "CAMLINFINE.NS", "RCH.L", "4344.T", "600100.SS", "002517.SZ", "000400.SZ", "000893.SZ", "SIGACHI.NS", "RIIL.NS", "SUBEXLTD.BO", "ML.PA", "600801.SS", "603529.SS", "000709.SZ", "0LNG.IL", "002244.SZ", "600282.SS", "688017.SS", "688276.SS", "0N9G.IL", "GVKPIL.NS", "002436.SZ", "000878.SZ", "GVKPIL.BO", "TEF.MC", "688521.SS", "000750.SZ", "600927.SS", "000021.SZ", "600909.SS", "KIRIINDUS.NS", "600901.SS", "600483.SS", "000591.SZ", "300001.SZ", "AEROFLEX.NS", "000898.SZ", "APH.L", "600038.SS", "300012.SZ", "002624.SZ", "0467.HK", "000825.SZ", "MARS.L", "0123.HK", "600166.SS", "603868.SS", "000683.SZ", "TOU.TO", "3475.T", "002850.SZ", "688361.SS", "MOTISONS.NS", "300604.SZ", "0NY8.IL", "600872.SS", "688213.SS", "CVCG.L", "002472.SZ", "600598.SS", "MG.TO", "002558.SZ", "688385.SS", "600390.SS", "002203.SZ", "ELE.MC", "1963.HK", "601965.SS", "600329.SS", "601001.SS", "000958.SZ", "VIE.PA", "600170.SS", "002044.SZ", "300699.SZ", "UDAICEMENT.BO", "002240.SZ", "601611.SS", "300390.SZ", "600361.SS", "688052.SS", "688278.SS", "0631.HK", "300017.SZ", "688772.SS", "688425.SS", "1208.HK", "0323.HK", "000088.SZ", "1258.HK", "6078.HK", "000032.SZ", "600095.SS", "1735.HK", "RAMASTEEL.NS", "603077.SS", "301526.SZ", "600578.SS", "BAM.TO", "600563.SS", "300285.SZ", "002508.SZ", "600777.SS", "8338.T", "600867.SS", "601399.SS", "PNBGILTS.NS", "603225.SS", "600673.SS", "300573.SZ", "600521.SS", "600516.SS", "COST.L", "HITECH.NS", "2007.HK", "002120.SZ", "9690.HK", "688172.SS", "300373.SZ", "GENL.L", "1199.HK", "002299.SZ", "SMIF.L", "0371.HK", "HLVLTD.NS", "300339.SZ", "300136.SZ", "0525.HK", "002405.SZ", "000785.SZ", "API.L", "9668.HK", "600008.SS", "002008.SZ", "GEOJITFSL.NS", "002281.SZ", "002138.SZ", "ANDREWYU.BO", "0NV5.IL", "601212.SS", "000155.SZ", "600141.SS", "300054.SZ", "0OF7.IL", "301165.SZ", "601139.SS", "300558.SZ", "000997.SZ", "0548.HK", "NAVF.L", "002318.SZ", "1999.HK", "603456.SS", "0136.HK", "2009.HK", "6098.HK", "002926.SZ", "300118.SZ", "600325.SS", "BCLIND.NS", "600098.SS", "000818.SZ", "9688.HK", "0148.HK", "SPCENET.NS", "300567.SZ", "JLP.L", "300073.SZ", "002432.SZ", "601997.SS", "600143.SS", "601019.SS", "601016.SS", "601568.SS", "600675.SS", "601106.SS", "603379.SS", "BOLSAA.MX", "0683.HK", "688063.SS", "0995.HK", "601098.SS", "601963.SS", "600032.SS", "600559.SS", "000830.SZ", "002025.SZ", "7048.T", "688516.SS", "300803.SZ", "600720.SS", "000403.SZ", "BIP-UN.TO", "300568.SZ", "600383.SS", "DCW.NS", "300140.SZ", "DPSCLTD.NS", "001213.SZ", "SURYODAY.NS", "HVO.L", "PENIND.NS", "601665.SS", "003035.SZ", "5852.T", "600820.SS", "GLE.PA", "300212.SZ", "1839.HK", "002015.SZ", "600808.SS", "600062.SS", "688578.SS", "301035.SZ", "VASCONEQ.NS", "300633.SZ", "001308.SZ", "002500.SZ", "688567.SS", "300595.SZ", "301301.SZ", "002867.SZ", "601128.SS", "9553.T", "2669.HK", "600007.SS", "002670.SZ", "600208.SS", "002531.SZ", "600373.SS", "000537.SZ", "TRAXIONA.MX", "000766.SZ", "GOLDIAM.NS", "1833.HK", "HPL.NS", "1508.HK", "600155.SS", "601333.SS", "0069.HK", "600529.SS", "600416.SS", "603883.SS", "NDTV.NS", "600271.SS", "601636.SS", "NAVKARCORP.NS", "300676.SZ", "3900.HK", "000519.SZ", "002065.SZ", "603858.SS", "688333.SS", "7383.T", "000034.SZ", "603185.SS", "600859.SS", "2386.HK", "000998.SZ", "601118.SS", "600498.SS", "688676.SS", "603477.SS", "0HAF.IL", "300070.SZ", "600548.SS", "002266.SZ", "600933.SS", "PENINLAND.NS", "688192.SS", "688639.SS", "MADRASFERT.NS", "301015.SZ", "1888.HK", "603650.SS", "600312.SS", "GEPIL.NS", "0OO9.IL", "000598.SZ", "ATL.NS", "1606.HK", "688005.SS", "301487.SZ", "002261.SZ", "000429.SZ", "000426.SZ", "MUFTI.NS", "601992.SS", "002832.SZ", "000628.SZ", "DHAMPURSUG.NS", "0HXB.IL", "300058.SZ", "SHALPAINTS.NS", "601858.SS", "688525.SS", "002461.SZ", "300002.SZ", "600131.SS", "1415.HK", "600737.SS", "002557.SZ", "6196.HK", "600012.SS", "1958.HK", "605589.SS", "600126.SS", "600258.SS", "002019.SZ", "301498.SZ", "000516.SZ", "600739.SS", "002273.SZ", "0636.HK", "IVN.TO", "688050.SS", "002389.SZ", "603228.SS", "002865.SZ", "002060.SZ", "605358.SS", "000623.SZ", "002407.SZ", "603730.SS", "605090.SS", "002024.SZ", "301039.SZ", "600123.SS", "300024.SZ", "TEN.MI", "000739.SZ", "ACLGATI.NS", "1252.HK", "APTECHT.NS", "3369.HK", "MONARCH.NS", "600339.SS", "002145.SZ", "SPIC.NS", "603565.SS", "600726.SS", "VOLARA.MX", "HIMATSEIDE.NS", "300298.SZ", "3903.T", "0257.HK", "TIGT.L", "6127.HK", "300761.SZ", "601608.SS", "600392.SS", "688608.SS", "688778.SS", "600764.SS", "0880.HK", "3396.HK", "601921.SS", "0570.HK", "NIITLTD.NS", "0NQP.IL", "000552.SZ", "601702.SS", "002268.SZ", "688002.SS", "0NQG.IL", "000401.SZ", "2096.HK", "3866.HK", "SRG.MI", "000951.SZ", "600528.SS", "688439.SS", "688029.SS", "0152.HK", "JSE.L", "RGL.L", "REP.MC", "300009.SZ", "300529.SZ", "688281.SS", "BOL.PA", "JYOTISTRUC.BO", "SHREDIGCEM.NS", "DWARKESH.NS", "300383.SZ", "JYOTISTRUC.NS", "688568.SS", "000543.SZ", "688536.SS", "000559.SZ", "600916.SS", "605296.SS", "000927.SZ", "000686.SZ", "KETL.L", "1530.HK", "600056.SS", "002151.SZ", "002773.SZ", "688146.SS", "002195.SZ", "002221.SZ", "002368.SZ", "300456.SZ", "002507.SZ", "CREO.L", "300364.SZ", "300679.SZ", "300769.SZ", "002563.SZ", "300395.SZ", "688147.SS", "600562.SS", "688037.SS", "FOXT.L", "600399.SS", "000060.SZ", "603927.SS", "002936.SZ", "SDY.L", "600827.SS", "603027.SS", "SURAJEST.NS", "300459.SZ", "BLKASHYAP.NS", "002373.SZ", "002506.SZ", "301371.SZ", "0425.HK", "600755.SS", "0354.HK", "603613.SS", "603505.SS", "002518.SZ", "603556.SS", "688177.SS", "TRN.MI", "603305.SS", "300294.SZ", "002901.SZ", "0639.HK", "603033.SS", "603517.SS", "601975.SS", "000967.SZ", "TFCILTD.NS", "000563.SZ", "001203.SZ", "601828.SS", "000810.SZ", "601777.SS", "1375.HK", "301090.SZ", "2252.HK", "301207.SZ", "603444.SS", "7859.T", "688366.SS", "000553.SZ", "300735.SZ", "688327.SS", "300257.SZ", "300776.SZ", "002690.SZ", "601326.SS", "300677.SZ", "603129.SS", "300682.SZ", "6049.HK", "688343.SS", "600057.SS", "AWHCL.NS", "002423.SZ", "600928.SS", "600481.SS", "603056.SS", "600877.SS", "MANGCHEFER.NS", "1164.HK", "002487.SZ", "2689.HK", "000981.SZ", "7238.T", "3633.HK", "688318.SS", "600500.SS", "002948.SZ", "001227.SZ", "688161.SS", "STEELXIND.BO", "STEELXIND.NS", "000778.SZ", "EKC.NS", "600959.SS", "688055.SS", "600435.SS", "300861.SZ", "000887.SZ", "002572.SZ", "SYNCOMF.BO", "601918.SS", "601022.SS", "SYNCOMF.NS", "002131.SZ", "688352.SS", "002958.SZ", "600248.SS", "OTEX.TO", "603127.SS", "000158.SZ", "600422.SS", "002675.SZ", "200012.SZ", "0014.HK", "300627.SZ", "301200.SZ", "002737.SZ", "000028.SZ", "688016.SS", "4978.T", "002092.SZ", "0QGU.IL", "000860.SZ", "002192.SZ", "9993.HK", "000762.SZ", "EPIC.L", "0754.HK", "300775.SZ", "600022.SS", "6255.T", "300672.SZ", "INDORAMA.NS", "600580.SS", "300476.SZ", "000902.SZ", "MONC.MI", "688484.SS", "600641.SS", "SATIA.NS", "002497.SZ", "300034.SZ", "603997.SS", "002906.SZ", "AA4.L", "000636.SZ", "WJG.L", "001287.SZ", "2005.HK", "601949.SS", "300376.SZ", "688798.SS", "300088.SZ", "002747.SZ", "ALL.L", "6869.HK", "2099.HK", "0811.HK", "002837.SZ", "AVTNPL.NS", "000410.SZ", "688432.SS", "000581.SZ", "300475.SZ", "600116.SS", "600761.SS", "002408.SZ", "601519.SS", "601369.SS", "DUKE.L", "AGSTRA.NS", "TEXINFRA.NS", "300438.SZ", "4563.T", "EMA.TO", "1799.HK", "688100.SS", "002683.SZ", "1060.HK", "600363.SS", "0010.HK", "MANALIPETC.NS", "1907.HK", "688088.SS", "MERC.L", "COFFEEDAY.BO", "002847.SZ", "3899.HK", "COFFEEDAY.NS", "002698.SZ", "300244.SZ", "2375.T", "002270.SZ", "BLISSGVS.NS", "LABB.MX", "600975.SS", "601952.SS", "601375.SS", "AFC.L", "000980.SZ", "688019.SS", "002155.SZ", "600456.SS", "301510.SZ", "SHG.L", "600315.SS", "002011.SZ", "601187.SS", "003021.SZ", "0038.HK", "603317.SS", "000582.SZ", "600667.SS", "600621.SS", "CENTRUM.NS", "688348.SS", "002126.SZ", "000567.SZ", "601801.SS", "600971.SS", "002643.SZ", "688006.SS", "600595.SS", "688336.SS", "600395.SS", "300346.SZ", "ARX.TO", "0142.HK", "0512.HK", "002351.SZ", "002387.SZ", "300253.SZ", "000627.SZ", "GICHSGFIN.NS", "000156.SZ", "600246.SS", "000012.SZ", "600320.SS", "688110.SS", "301095.SZ", "0853.HK", "600750.SS", "600967.SS", "600297.SS", "002096.SZ", "300296.SZ", "300666.SZ", "603599.SS", "301238.SZ", "301363.SZ", "600167.SS", "002484.SZ", "EN.PA", "0189.HK", "002925.SZ", "002399.SZ", "300357.SZ", "0551.HK", "002498.SZ", "000935.SZ", "688248.SS", "600633.SS", "4571.T", "600850.SS", "601811.SS", "301153.SZ", "000528.SZ", "688779.SS", "002030.SZ", "300482.SZ", "603171.SS", "000761.SZ", "688409.SS", "688266.SS", "3933.HK", "000089.SZ", "301029.SZ", "300115.SZ", "1053.HK", "300315.SZ", "2778.HK", "002727.SZ", "5244.T", "XCHANGING.NS", "603393.SS", "PST.MI", "000966.SZ", "601827.SS", "002320.SZ", "300638.SZ", "I3E.L", "2186.HK", "600452.SS", "600323.SS", "688200.SS", "600388.SS", "2068.HK", "0107.HK", "603283.SS", "603871.SS", "601677.SS", "601222.SS", "601969.SS", "002182.SZ", "688107.SS", "603132.SS", "002010.SZ", "601869.SS", "002541.SZ", "TREL.NS", "PFC.L", "601882.SS", "603515.SS", "KOPRAN.NS", "603005.SS", "600711.SS", "688739.SS", "600185.SS", "688575.SS", "600740.SS", "PFCL.XC", "002312.SZ", "600623.SS", "600316.SS", "688001.SS", "300045.SZ", "300723.SZ", "KAMOPAINTS.NS", "002317.SZ", "1798.HK", "1357.HK", "601778.SS", "INOV.L", "8789.T", "000415.SZ", "ONMOBILE.NS", "002511.SZ", "RICOAUTO.NS", "002791.SZ", "300171.SZ", "300446.SZ", "GMS.L", "688166.SS", "2343.HK", "000550.SZ", "600420.SS", "000031.SZ", "002091.SZ", "600997.SS", "300741.SZ", "0ROY.IL", "0FGH.IL", "AXTELCPO.MX", "300623.SZ", "300857.SZ", "688498.SS", "TRACXN.NS", "300363.SZ", "000869.SZ", "0579.HK", "002003.SZ", "300182.SZ", "300026.SZ", "601126.SS", "300458.SZ", "600602.SS", "603638.SS", "600335.SS", "601718.SS", "3918.HK", "002978.SZ", "900926.SS", "0O8F.IL", "0NPH.IL", "600572.SS", "688322.SS", "300130.SZ", "688139.SS", "002324.SZ", "603300.SS", "603866.SS", "002402.SZ", "000547.SZ", "1216.HK", "000930.SZ", "0RDM.IL", "688062.SS", "603025.SS", "001914.SZ", "WEBELSOLAR.NS", "002840.SZ", "601038.SS", "301511.SZ", "301262.SZ", "300457.SZ", "603236.SS", "SNOWMAN.NS", "002544.SZ", "002335.SZ", "688556.SS", "600717.SS", "0HAH.IL", "601163.SS", "600771.SS", "603355.SS", "600596.SS", "001323.SZ", "600120.SS", "300492.SZ", "601137.SS", "001322.SZ", "000875.SZ", "301239.SZ", "ONEPOINT.NS", "9969.HK", "300773.SZ", "000657.SZ", "688097.SS", "1448.HK", "300747.SZ", "601068.SS", "600864.SS", "200413.SZ", "300910.SZ", "601005.SS", "603983.SS", "301205.SZ", "300049.SZ", "NEMAKA.MX", "300428.SZ", "000712.SZ", "4593.T", "601107.SS", "002911.SZ", "0839.HK", "000970.SZ", "002237.SZ", "601512.SS", "600225.SS", "5621.T", "002468.SZ", "600729.SS", "600597.SS", "603218.SS", "002229.SZ", "605111.SS", "002745.SZ", "0B67.IL", "2162.HK", "688433.SS", "600821.SS", "002332.SZ", "ACRL.L", "300777.SZ", "002249.SZ", "603279.SS", "600195.SS", "0NQF.IL", "002597.SZ", "002100.SZ", "603919.SS", "002992.SZ", "600273.SS", "688789.SS", "600428.SS", "688105.SS", "301177.SZ", "600557.SS", "300229.SZ", "002851.SZ", "301381.SZ", "5586.T", "2128.HK", "5955.T", "000503.SZ", "603888.SS", "688700.SS", "002416.SZ", "3656.T", "002216.SZ", "002617.SZ", "605365.SS", "300068.SZ", "688289.SS", "600601.SS", "600935.SS", "600587.SS", "CA.PA", "2273.HK", "002424.SZ", "1579.HK", "000035.SZ", "002226.SZ", "600211.SS", "600201.SS", "300406.SZ", "002139.SZ", "002254.SZ", "0719.HK", "002705.SZ", "300856.SZ", "603103.SS", "688686.SS", "MUNJALAU.NS", "300393.SZ", "600409.SS", "002292.SZ", "002158.SZ", "301286.SZ", "300432.SZ", "605599.SS", "2285.HK", "603699.SS", "603197.SS", "1918.HK", "002183.SZ", "3777.T", "301219.SZ", "2193.T", "002458.SZ", "002222.SZ", "002287.SZ", "603043.SS", "3989.T", "000688.SZ", "0694.HK", "600055.SS", "002085.SZ", "600199.SS", "6699.HK", "600787.SS", "300188.SZ", "0586.HK", "6993.HK", "603108.SS", "300624.SZ", "1896.HK", "300705.SZ", "600728.SS", "ANIC.L", "600908.SS", "301559.SZ", "301155.SZ", "002265.SZ", "600552.SS", "300199.SZ", "002610.SZ", "600662.SS", "603567.SS", "600006.SS", "0MH1.IL", "603678.SS", "PRY.MI", "300168.SZ", "0E4Q.IL", "300726.SZ", "002099.SZ", "000682.SZ", "000620.SZ", "000062.SZ", "2370.T", "SMCGLOBAL.NS", "000685.SZ", "0546.HK", "600328.SS", "DHANBANK.NS", "1883.HK", "6808.HK", "600267.SS", "2982.T", "6185.HK", "688428.SS", "600475.SS", "000676.SZ", "601900.SS", "0697.HK", "INW.MI", "000413.SZ", "601908.SS", "000065.SZ", "603162.SS", "0CXC.IL", "601069.SS", "BODALCHEM.NS", "002041.SZ", "000727.SZ", "300260.SZ", "603733.SS", "001337.SZ", "688326.SS", "301268.SZ", "300820.SZ", "001269.SZ", "002326.SZ", "000061.SZ", "1368.HK", "000422.SZ", "3868.HK", "000756.SZ", "PEBB.L", "002174.SZ", "603063.SS", "CHAR.L", "600782.SS", "000828.SZ", "BVI.PA", "FUM.L", "600459.SS", "600963.SS", "1083.HK", "000402.SZ", "600861.SS", "002761.SZ", "ADSL.NS", "002063.SZ", "7777.T", "300255.SZ", "688351.SS", "000923.SZ", "600776.SS", "3709.HK", "603277.SS", "300850.SZ", "300101.SZ", "1316.HK", "001339.SZ", "0Q0Y.IL", "688141.SS", "002701.SZ", "VIKASLIFE.NS", "600259.SS", "002212.SZ", "688007.SS", "ACS.MC", "600507.SS", "688083.SS", "600835.SS", "688408.SS", "600186.SS", "600064.SS", "605011.SS", "600461.SS", "VIKASLIFE.BO", "603766.SS", "9979.HK", "000821.SZ", "0179.HK", "603039.SS", "SUTLEJTEX.NS", "600266.SS", "300133.SZ", "CPR.MI", "600338.SS", "600331.SS", "603337.SS", "ZEEMEDIA.NS", "600179.SS", "300080.SZ", "600508.SS", "601595.SS", "688106.SS", "300674.SZ", "002567.SZ", "600718.SS", "9698.HK", "ZEEMEDIA.BO", "ARB.L", "605577.SS", "0604.HK", "300827.SZ", "300323.SZ", "603267.SS", "000029.SZ", "603358.SS", "000737.SZ", "GSLSU.NS", "KELLTONTEC.NS", "600759.SS", "601208.SS", "RNO.PA", "600037.SS", "000811.SZ", "300811.SZ", "002539.SZ", "1313.HK", "002048.SZ", "002985.SZ", "VIV.PA", "603690.SS", "603876.SS", "300525.SZ", "8918.T", "6927.T", "0IIF.IL", "603235.SS", "0H59.IL", "600917.SS", "688185.SS", "600639.SS", "605116.SS", "600929.SS", "688279.SS", "002960.SZ", "TRR.L", "002020.SZ", "300620.SZ", "000690.SZ", "1302.HK", "600648.SS", "600216.SS", "002882.SZ", "601200.SS", "ORIENTPPR.NS", "601528.SS", "002929.SZ", "600657.SS", "002366.SZ", "000990.SZ", "600133.SS", "603100.SS", "603055.SS"]

ETF_list = ["FXN", "PXE", "FCG", "FTXN", "VDE", "IEO", "FENY", "IYE", "FEMS", "XLE", "REZ", "IXC", "INDS", "EWJ", "JPXN", "BBJP", "FLJP", "FILL", "EMCR", "JPEM", "VIGI", "XLRE", "FRI", "BBRE", "HACK", "HEWJ", "ICF", "USRT", "RWR", "IGRO", "IPAC", "PFFA", "PSR", "FLJH", "CIBR", "IYR", "IMTM", "DFE", "JHEM", "FREL", "FNDE", "VNQ", "DNL", "DGRE", "EMXC", "DEM", "IHDG", "EEMV", "PGX", "FDLO", "PFFD", "PXH", "EWJV", "VPL", "FTCS", "EMGF", "DBEU", "EWH", "HAWX", "EFG", "TDIV", "IQDG", "GEM", "XCEM", "ESGD", "HEFA", "VWO", "LGLV", "JHMD", "DOL", "DBEF", "SPLV", "HEEM", "SCZ", "FYLD", "QEFA", "IDEV", "SPEM", "EELV", "FGD", "SCHC", "CWI", "EFA", "SCHE", "IEFA", "JQUA", "IDHQ", "IEMG", "SPDW", "IETC", "PDN", "IXN", "XMHQ", "IQLT", "VEU", "DDWM", "TLTD", "VSS", "FVD", "IXUS", "KIE", "PIE", "HFXI", "EEM", "SCHF", "VEA", "SPHD", "VXUS", "PXF", "BAB", "GSIE", "BBEU", "HSCZ", "DGS", "GWX", "XSOE", "VYMI", "ESGE", "ACWX", "XLK", "QARP", "FTLS", "MFEM", "OUSA", "TLTE", "IGV", "FPE", "HEZU", "DWM", "IQIN", "PFM", "DDLS", "CWS", "DXJ", "VGK", "SCHD", "DIVI", "FDVV", "IYW", "SPHQ", "DGRO", "LVHI", "FTEC", "FDL", "IDV", "EFV", "ADME", "ISCF", "QUS", "MFDX", "QDF", "QUAL", "FNDF", "IEUR", "SPEU", "EYLD", "SDY", "ICOW", "VCLT", "EWT", "QINT", "VGT", "PFF", "FIVA", "HDEF", "IEV", "SPLB", "VIG", "USMF", "DTD", "EWX", "FQAL", "IUSG", "DGRW", "IGM", "IUS", "IVLU", "SCJ", "DLN", "DHS", "IAK", "USSG", "IWX", "NOBL", "MDYG", "VOOG", "IGLB", "EZU", "FXL", "FLEE", "LVHD", "MGK", "VSDA", "FDRR", "FIVG", "QGRO", "MGC", "IWL", "PGF", "AIQ", "VYM", "FEZ", "QLD", "TTAC", "VONE", "BBUS", "IJK", "BIZD", "SPYG", "JPUS", "XSW", "VTHR", "PWV", "USMC", "PID", "PFXF", "OEF", "IWF", "SCHX", "SUSA", "CFA", "CHGX", "DIVB", "JMOM", "SPYX", "IWY", "SPTM", "IDMO", "FPXI", "EEMS", "CDC", "ESGU", "VONG", "IVW", "IVV", "FEP", "IWV", "XLG", "VUG", "ILCV", "ILCG", "TMFC", "EPS", "SCHV", "SPLG", "SPY", "FLQL", "REGL", "XSHQ", "BBH", "GSLC", "QQQ", "VOO", "IVOG", "SLYG", "SPGP", "IUSV", "SCHB", "ITOT", "EDOW", "CATH", "PBUS", "JHML", "VV", "ONEQ", "SCHK", "IWB", "VTV", "VTI", "SPMO", "DSI", "MGV", "MOAT", "PSCH", "NUDM", "VOOV", "SYLD", "QQEW", "DES", "FPEI", "VFQY", "IJT", "VONV", "KBWP", "IWC", "QTUM", "EQWL", "PTNQ", "JPSE", "ESGV", "SPYV", "FSMD", "DFJ", "IYY", "OUSM", "PRF", "FNDX", "IVE", "MDY", "IJH", "IDOG", "FVAL", "MOTI", "DVY", "COWZ", "IWD", "DGRS", "HEDJ", "FTC", "SCHG", "SPYD", "WTV", "IMCV", "SECT", "EQAL", "FNDB", "FM", "QQQE", "FYC", "JSML", "PTH", "DSTL", "QTEC", "XMMO", "GSSC", "DWAS", "DJD", "IWN", "VIOG", "VIOO", "IXJ", "MVV", "DXJS", "IMCG", "VRP", "SPMD", "FDM", "EUSA", "JSMD", "IVOO", "IWS", "JHSC", "SNSR", "VOE", "IWP", "XSMO", "NULG", "ESML", "JVAL", "SCHM", "DON", "IYH", "GSEW", "FLQM", "FNDA", "VO", "IWM", "VTWV", "CNYA", "MDYV", "IWR", "VTWO", "IMCB", "SIZE", "IJJ", "ISCG", "RPG", "JHMM", "XLF", "IVOV", "FTA", "FYX", "FEX", "NUSC", "ONEY", "IJR", "PRFZ", "SPSM", "SDVY", "RSP", "KGRN", "VOT", "SCHA", "CALF", "SMMD", "VHT", "XLV", "KBA", "SDOG", "VFH", "NUMG", "IYF", "IHI", "XSVM", "VBK", "FTXL", "VB", "SMLF", "EZM", "NUEM", "RWL", "VFVA", "FHLC", "VXF", "ITEQ", "RWK", "FXH", "OMFS", "VBR", "DIA", "RWJ", "IJS", "SMH", "FNCL", "GXC", "FNX", "RDVY", "PEY", "FNK", "SLYV", "ASHR", "VIOV", "CHIQ", "FXO", "FYT", "EES", "SOXX", "ROBT", "FV", "IAI", "PSI", "SPVU", "CSB", "FCTR", "OMFL", "XHE", "FBT", "RZV", "RDIV", "RFV", "PTF", "CXSE", "XSD", "KOMP", "KBWB", "SPHB", "CQQQ", "CNRG", "RPV", "KRE", "IHF", "IAT", "PGJ",]

SP_list = ["MSFT", "AAPL", "GOOG", "GOOGL", "AMZN", "NVDA", "META", "BRK-B", "TSLA", "LLY", "JPM", "V", "TSM", "AVGO", "UNH", "WMT", "MA", "XOM", "JNJ", "PG", "HD", "TCEHY", "BAC", "COST", "MRK", "ORCL", "ABBV", "ASML", "CVX", "ADBE", "CRM", "KO", "AMD", "FMX", "PEP", "ACN", "NVS", "NFLX", "MCD", "TMO", "SHEL", "CSCO", "INTC", "LIN", "ABT", "PDD", "TMUS", "SAP", "BABA", "CMCSA", "INTU", "WFC", "DHR", "HDB", "DIS", "AMGN", "VZ", "PFE", "BHP", "NKE", "MS", "QCOM", "TTE", "IBM", "HSBC", "TXN", "NOW", "PM", "CAT", "UNP", "BX", "GS", "GE", "RY", "SPGI", "UPS", "AXP", "COP", "HON", "C", "BA", "SNY", "UBER", "BUD", "ISRG", "LOW", "AMAT", "NEE", "RTX", "SONY", "PLD", "SCHW", "SYK", "BLK", "MDT", "LMT", "RIO", "VRTX", "ELV", "TD", "TJX", "DE", "GILD", "SBUX", "SHOP", "PANW", "BMY", "LRCX", "BP", "MDLZ", "PGR", "CVS", "AMT", "ADP", "ETN", "MMC", "UBS", "ADI", "CB", "AAGIY", "ZTS", "MU", "EQNR", "CI", "ABNB", "BSX", "IBN", "GSK", "FI", "CNI", "DEO", "ANET", "ENB", "SO", "RELX", "SHW", "ITW", "KLAC", "DUK", "HCA", "SNPS", "WDAY", "MO", "WM", "CDNS", "KKR", "ICE", "NOC", "CP", "SLB", "CME", "CNQ", "GD", "MCO", "CSX", "BDX", "CRWD", "EOG", "BTI", "STLA", "CL", "MAR", "PYPL", "USB", "TGT", "MCK", "SCCO", "SNOW", "FDX", "MNST", "TEAM", "BN", "PNC", "LULU", "BMO", "AON", "MPC", "MMM", "PH", "NTES", "FCX", "APD", "EPD", "PSX", "BNS", "APH", "ROP", "MRVL", "ECL", "DELL", "TT", "SPG", "EMR", "APO", "HUM", "CHTR", "NXPI", "MSI", "HMC", "RSG", "PXD", "PSA", "NSC", "ADSK", "DHI", "OXY", "MET", "WELL", "AJG", "CARR", "PCAR", "TFC", "CCI", "COF", "CRH", "AFL", "GM", "EL", "FTNT", "DXCM", "SRE", "AIG", "STZ", "HLT", "IDXX", "KHC", "MCHP", "ROST", "CPRT", "EW", "VLO", "TRV", "KDP", "COR", "HES", "NEM", "PAYX", "AEP", "LEN", "O", "SU", "WMB", "ODFL", "DLR", "CNC", "KMB", "BK", "NUE", "DASH", "OKE", "TRP", "TEL", "SQ", "BIDU", "DDOG", "MRNA", "WDS", "SPOT", "D", "JD", "STM", "ALL", "LHX", "ALC", "LNG", "CTSH", "IQV", "HSY", "MFC", "WCN", "JCI", "A", "SYY", "AMP", "FERG", "URI", "AME", "DOW", "MPLX", "LVS", "BCE", "PRU", "ADM", "EA", "IBKR", "FIS", "FAST", "ARES", "YUM", "EXC", "IT", "OTIS", "QSR", "ROK", "VRSK", "PPG", "ZS", "CSGP", "GPN", "XEL", "CMI", "VEEV", "EXR", "KR", "NDAQ", "TTD", "CTVA", "DD", "LI", "VICI", "ON", "BKR", "GFS", "ED", "IMO", "IR", "COIN", "RCL", "CCEP", "HAL", "LYB", "ANSS", "PEG", "EFX", "SLF", "VMC", "DLTR", "DG", "HPQ", "PWR", "CDW", "PUK", "ACGL", "MDB", "HUBS", "RYAAY", "MPWR", "FANG", "TTWO", "DVN", "DFS", "EIX", "XYL", "BF-B", "NET", "KEYS", "AEM", "WEC", "GLW", "CAH", "CBRE", "AVB", "SBAC", "AWK", "SPLK", "ZBH", "WTW", "RMD", "RBLX", "NTR", "FTV", "PINS", "HIG", "DAL", "TROW", "BNTX", "WY", "TCOM", "ALNY", "TSCO", "CHD", "BR", "GRMN", "STT", "EQR", "SYM", "ULTA", "FITB", "WAB", "RJF", "APTV", "HWM", "PHM", "TW", "DTE", "MTB", "FE", "WPM", "ARE", "ILMN", "PHG", "MT", "ETR", "CCJ", "EBAY", "BRO", "ROL", "FNV", "ICLR", "LYV", "VRSN", "ALGN", "INVH", "GMAB", "BLDR", "EXPE", "DOV", "PTC", "CM", "IFF", "SE", "BAX", "WBA", "PPL", "ES", "TECK", "JBHT", "VTR", "IRM", "TSN", "GPC", "CTRA", "CBOE", "K", "LH", "PBA", "SMCI", "AEE", "VRT", "TS", "WRB", "PFG", "DRI", "TRGP", "CHKP", "EXPD", "WMG", "STLD", "WAT", "CNP", "IOT", "NTAP", "AKAM", "CLX", "MKC", "BALL", "BMRN", "HUBB", "OMC", "SUI", "ATO", "LPLA", "HOLX", "HRL", "NTRS", "STX", "FSLR", "LUV", "RPRX", "CMS", "J", "CINF", "ENTG", "BAH", "DT", "JBL", "SWKS", "WDC", "MAA", "CE", "TER", "MGA", "BBY", "L", "TXT", "EQT", "LW", "DKNG", "YUMC", "DGX", "BAM", "SYF", "LDOS", "MAS", "ENPH", "AER", "SSNC", "PKG", "AVTR", "MGM", "GEN", "ALB", "AMH", "GDDY", "FWONK", "CFG", "CF", "WPC", "SWK", "FOX", "FOXA", "BSY", "DPZ", "GGG", "CG", "XP", "BEN", "INCY", "VST", "NWSA", "NWS", "APP", "PODD", "CELH", "BIP", "ALLY", "SQM", "KIM", "OKTA", "CAG", "RPM", "BG", "SJM", "ELS", "TAP", "UDR", "MRO", "RVTY", "RTO", "FNF", "TRU", "OC", "TWLO", "H", "U", "WSM", "ACI", "UAL", "IP", "NBIX", "DOCU", "CPB", "EDU", "LNT", "TRMB", "AGR", "AFRM", "LKQ", "PATH", "GLPI", "ZG", "Z", "BURL", "EVRG", "ACM", "ROKU", "IPG", "NTNX", "JKHY", "EXAS", "GFL", "AOS", "JNPR", "KRTX", "SNN", "REXR", "REG", "FMS", "RBA", "PNR", "ARCC", "PSTG", "DKS", "OTEX", "NRG", "TECH", "PEAK", "PAYC", "USFD", "GL", "OVV", "FND", "ESTC", "LBRDK", "RYAN", "EQH", "NI", "KMX", "BXP", "DINO", "CCK", "PFGC", "SRPT", "LAMR", "WES", "CRL", "CHK", "UHS", "MOS", "WRK", "DOX", "DBX", "WYNN", "CPT", "X", "TOL", "HTHT", "FIVE", "FFIV", "ALLE", "WTRG", "EMN", "QGEN", "CUBE", "CDAY", "CHRW", "EWBC", "FLEX", "SCI", "XPO", "MTCH", "LSXMA", "DVA", "QRVO", "LSXMK", "PCOR", "APA", "BRKR", "FBIN", "SKX", "RGEN", "GTLB", "HSIC", "CZR", "BBWI", "CYBR", "NVT", "TTC", "GWRE", "MNDY", "SNX", "RRX", "LSCC", "KNX", "AGCO", "RL", "ONON", "CTLT", "DUOL", "TREX", "UNM", "BJ", "CNM", "ELF", "PCTY", "ALV", "TPX", "WBS", "RHI", "FRT", "INFA", "COLD", "ETSY", "JEF", "CYTK", "SEIC", "ORI", "TPR", "APLS", "IMGN", "THC", "WSC", "MSTR", "NYT", "PNW", "BILL", "SWAV", "NNN", "ACHC", "HLI", "HESM", "GPK", "TXRH", "GMED", "XRAY", "CERE", "BWA", "OHI", "IONS", "LBTYK", "APG", "LEA", "CVNA", "GNRC", "S", "BERY", "ARMK", "GNTX", "GPS", "JAZZ", "RRC", "ASND", "AMKR", "KBR", "VOYA", "NFE", "BWXT", "NTRA", "FMC", "AXTA", "FR", "DAR", "LNW", "CIEN", "WIX", "STAG", "PVH", "CMA", "RMBS", "MNSO", "ONTO", "EDR", "BRBR", "SF", "EXEL", "AR", "FRSH", "OSK", "MKSI", "EHC", "CBSH", "OGE", "GXO", "BRX", "HAS", "CFLT", "PSN", "WAL", "WFRD", "BBIO", "STWD", "PPC", "FUTU", "WH", "CIVI", "MTDR", "SMAR", "CGNX", "HRB", "GLBE", "MHK", "PLNT", "FLR", "FAF", "ITCI", "SRC", "OLN", "ADC", "COHR", "G", "NE", "WHR", "CROX", "DV", "ARW", "MASI", "ZION", "HQY", "FOUR", "IMVT", "BYD", "PB", "MUR", "THO", "W", "BC", "CPRI", "CMC", "CAR", "CHH", "OMF", "PCVX", "FSK", "GTLS", "VNO", "MTZ", "OPCH", "FIVN", "POST", "FYBR", "VNOM", "TMHC", "TXG", "AZEK", "TENB", "CWAN", "DOCS", "INSP", "ATI", "GIL", "ESI", "TRNO", "AA", "VNT", "BECN", "BRZE", "OZK", "FLS", "SEE", "AIRC", "DTM", "SNV", "PBF", "NSA", "COLB", "BEPC", "CHX", "ST", "CADE", "MLI", "IGT", "SUN", "CRSP", "ALSN", "SFM", "EXLS", "CWEN", "VRNS", "BROS", "ALGM", "BPMC", "PII", "LPX", "UGI", "KRG", "PWSC", "UBSI", "RH", "ANF", "QDEL", "VAL", "ZWS", "FTAI", "KD", "RYN", "JHG", "KRC", "HOMB", "FLO", "PECO", "HOG", "IRDM", "AXS", "ACAD", "LNC", "ALKS", "DXC", "TDC", "NUVL", "RVMD", "SLGN", "BIPC", "ASO", "KBH", "MMSI", "HALO", "VVV", "ALK", "ARWR", "CC", "PRGO", "AL", "GBCI", "RDN", "OLLI", "HCP", "AEL", "HGV", "AWI", "SM", "SQSP", "GKOS", "IAC", "POR", "AVT", "SIG", "WOLF", "SNDR", "MGY", "SUM", "SEDG", "SMPL", "MRTX", "HUN", "NVST", "AEO", "FRPT", "SGRY", "COOP", "INSM", "AXSM", "EPRT", "FFIN", "YETI", "NCNO", "JXN", "HWC", "ESMT", "LNTH", "TEX", "RARE", "FLNC", "ACLS", "PGNY", "AAP", "BXMT", "CUZ", "BHVN", "VRRM", "BKH", "BOX", "AZTA", "URBN", "EVH", "VERX", "NOG", "EPR", "MC", "CRC", "PENN", "BL", "MLTX", "AXNX", "TDW", "MANU", "FROG", "RUSHA", "TDOC", "STNG", "UCBI", "AYX", "CLS", "LEG", "LITE", "XENE", "SEM", "RPD", "HP", "OGS", "TPH", "IART", "MOD", "SPT", "SMG", "NVEI", "LAZ", "CRS", "NARI", "YOU", "HCC", "TSEM", "ACIW", "PNM", "SEAS", "CNX", "ENOV", "ASB", "SPR", "AI", "SHOO", "BTU", "OLK", "CVI", "VSH", "RNG", "DOCN", "ADNT", "AX", "FORM", "VSAT", "CNO", "FIBK", "YELP", "VAC", "CEIX", "SWTX", "SLG", "GH", "CDP", "SKT", "AMN", "FHI", "APAM", "KSS", "SGML", "KLIC", "HASI", "TNL", "CALX", "SHAK", "AVA", "LIVN", "FHB", "ENV", "SPB", "PDCO", "IDYA", "UPST", "WOR", "NEP", "CBAY", "FL", "CRI", "CTRE", "TMDX", "PPBI", "WHD", "PINC", "FTDR", "RCKT", "CARG", "WERN", "LRN", "FLYW", "PRVA", "HIW", "CRNX", "QTWO", "NTLA", "GLNG", "PD", "RYTM", "CORT", "PGTI", "PRCT", "AY", "PZZA", "USAC", "ENR", "CLDX", "FCPT", "AMBA", "BKU", "CPE", "BOOT", "BLMN", "VKTX", "ARVN", "PARR", "TWST", "BEAM", "SIX", "PTCT", "SDGR", "TGH"]